In [1]:
# I guess it is best to use Julia 1.9 because it works with the packages in my case
# IN GENERAL: Lets try to use self explaining variable names all the time

using Pkg
# you need to create an environment first, then activate it (in this case mine is called "optimization19")
Pkg.activate("1stEnv")
# - - #
# Install all the packages mentioned below via the following commands
# using Pkg
# Pkg.add("Optim") etc.

# Load packages and obtain the solver LICENSE
using Optim, TrafficAssignment, LinearAlgebra, JuMP, DelimitedFiles, PrettyTables, CSV, LsqFit, Printf

#Include the needed functions (alphabetically) - make sure that functions are stored in the right directory
include("moco_functions/daily_allocated_mocos.jl")      # compute amount of daily mocos (MobilityCoins) needed based on maintaining traffic patterns without       
include("moco_functions/allocated_mocos.jl")     # compute amount of total mocos (MobilityCoins) needed based on maintaining traffic patterns without coins
include("moco_functions/daily_consumed_mocos.jl")       # amount of mocos consumed per day based on the last traffic assignment
include("moco_functions/consumed_mocos.jl")             # amount of mocos consumed at the end of the considered period 
include("moco_functions/emissions.jl")
include("moco_functions/load_network.jl")               # load network demand and supply (status quo "Visum Munich Transportation Model")
include("moco_functions/moco_link_costs_wo_mp.jl")      # compute the link costs without market price
include("moco_functions/mode_choice_tt.jl")             # compute mode choice model based on current travel times
include("moco_functions/shortest_path.jl")              # compute shortest paths OD matrix
include("moco_functions/shortest_traveltime.jl")        # compute shortest travel time for mode choice function 
include("moco_functions/transportation_model.jl")       # includes mode choice and traffic assignment
println("all functions included")


# Load the SiouxFalls network from the TrafficAssignment repository. We will use the SiouxFalls network for development and debugging
ta_data = load_ta_network("SiouxFalls")
ta_datac = load_ta_network("SiouxFalls")
ta_datab = load_ta_network("SiouxFalls")
ta_datar = load_ta_network("SiouxFalls")
# We might use the network below later. Could be challenging to make it work
# ta_data = load_ta_network("ring24total")
# ta_datac = load_ta_network("ring24c")
# ta_datab = load_ta_network("ring24b")
# ta_datar = load_ta_network("ring24r")


sqcardemand = ta_datac.travel_demand 
sqbusdemand = ta_datab.travel_demand 
sqraddemand = ta_datar.travel_demand 

println("car demand status quo = ",sum(sqcardemand))
println("bus demand status quo = ",sum(sqbusdemand))
println("bike demand status quo = ",sum(sqraddemand))

  Activating project at `~/MyThesisCode-main/1stEnv`


all functions included
car demand status quo = 360600.0
bus demand status quo = 360600.0
bike demand status quo = 360600.0


In [2]:
# SETTING UP MOCO EMISSION AND INCENTIVE PARAMETER
# specific avg externalities/incentives per mode
# https://www.nachhaltigkeitsstrategie.de/strategie/projekte/detail-1/mit-dem-oepnv-das-klima-schuetzen?utm_source=chatgpt.com
# There’s no official public number for CO₂/pkm specifically for the MVV region. However, MVV roughly reflects the German averages:
# Public transport around 65–75 g/pkm Car around 150 g/pkm Cycling/walking essentially zero CO₂.
# Additionally, on average: Cars cause 147 g/pkm (assuming 1.5 passengers), Public transport (bus/train) averages 65 g/pkm
# emissions ------------------------------
carext = 0.147     # kgCO2 per Pkm
busext = 0.065     # kgCO2 per Pkm 
# bikeext = 0      # kgCO2 per Pkm
# ----------------------------------------

# moco price per emission ----------------
mocperkgCO2 = 1   # MoCo per kg CO2 for the start
# ----------------------------------------

# bike incentive -------------------------
bikeincentive = 0 # -> no incentive for the start
# ----------------------------------------

# mocomp ---------------------------------
mocomp = 1 # market price equals one at the start
# ---------------------------------------------------------------------------

println("done 1/10")


# LOAD NETWORK DATA >>> function load_network(basenet, carnet, busnet, bikenet)
nodes, nnodes, zones, nzones, n, arcs, demandc, demandb, demandr, totaldemand, destinations, arcdest, demand_dict, isod, bpr = @time load_network(ta_data, ta_datac, ta_datab, ta_datar)
println("done 2/10")



# first traffic assignment car w/o moco -> output: link travel times car
link_flowc, link_travel_timecsq, objectivec =
@time ta_frank_wolfe(ta_datac, method=:cfw, max_iter_no=50, step=:exact, log=:off, tol=1e-1)

# link travel times bus ("linearly" congested)
ta_datab.power .= 1                                     
link_travel_timebsq = ta_datab.free_flow_time # .+ 1/60 * 5 # 5 minute waiting time at the bus stop included in mode-choice

# link travel times bike ("linearly" congested)
ta_datar.power .= 1                                     
link_travel_timersq = ta_datar.link_length / 15  # 15km/h avg bike velocity
ta_datar.capacity .= 100
println("done 3/10")



# COMPUTE SHORTEST PATHS
dist = @time shortest_path(ta_data)
println("done 4/10")

# COMPUTE SHORTEST TRAVELTIMES 
ttsc, ttsb, ttsr = @time shortest_traveltime(ta_data, link_travel_timecsq, link_travel_timebsq, link_travel_timersq)
println("done 5/10")

# COMPUTE MOCO COSTS WITHOUT MARKET PRICE
mpdictwomp = moco_link_costs_wo_mp(dist, carext, mocperkgCO2)

println("done 6/10")

# ENABLE TOLL BPR TERM -> that is activating the toll term of the TrafficAssignment package. we use it for the mobilitycoin charges
ta_datac.toll_factor = 1
ta_datab.toll_factor = 1
ta_datar.toll_factor = 1
println("done 7/10")

# pluggin in VOT via free flow travel time
# that is important because we are using the TrafficAssignment package which is why I dont change the cost function itself
ta_datac.free_flow_time = ta_datac.free_flow_time .* 30 # 30 EUR VOT for car passengers
ta_datab.free_flow_time = ta_datab.free_flow_time .* 20 # 20 EUR VOT for bus passengers
ta_datar.free_flow_time = ta_datar.free_flow_time .* 20 # 20 EUR VOT for cyclists
println("done 8/10")

# prepare bike links for car and bus mode (the following code doesnt apply to the SiouxFalls case)
# Iterate over the range from 1 to number_of_links to adjust the capacity of bike links for mode car
for i in 1:ta_datac.number_of_links
    # Check if the link_type is 10 (bike link)
    if ta_datac.link_type[i] == 10
        # Adjust the capacity to 0.00001
        ta_datac.free_flow_time[i] = 100
    end
end

for i in 1:ta_datab.number_of_links
    # Check if the link_type is 10 (bike link)
    if ta_datab.link_type[i] == 10
        # Adjust the capacity to 0.00001
        ta_datab.free_flow_time[i] = 100
    end
end
println("done 9/10")


# Store status quo link travel times in a CSV file
dftt = DataFrame(link_travel_timec = link_travel_timecsq,
link_travel_timeb = link_travel_timebsq,
link_travel_timer = link_travel_timersq)
# Write the DataFrame to a CSV file
CSV.write("traveltimes.csv", dftt)

println("done 10/10")

done 1/10
  0.763446 seconds (1.73 M allocations: 91.472 MiB, 2.68% gc time, 99.81% compilation time)
done 2/10
  2.418668 seconds (4.14 M allocations: 221.049 MiB, 4.04% gc time, 99.46% compilation time)
done 3/10
  0.140483 seconds (411.50 k allocations: 21.439 MiB, 99.84% compilation time)
done 4/10
  0.381538 seconds (1.60 M allocations: 88.642 MiB, 5.61% gc time, 99.82% compilation time)
done 5/10
done 6/10
done 7/10
done 8/10
done 9/10
done 10/10


In [3]:
# preparing the arrays for recording results
mocperkgCO2arr = []
MParr = []
coinsallocatedarr = []
coinsneededarr = []
deltaarr = []
mocarr = []
cardemarr = []
busdemarr = []
bikedemarr = []
flowscararr = []
flowsbusarr = []
flowsbikearr = []
mocoscararr = []
mocosbusarr = []
mocosbikearr = []
emissionscararr = []
emissionsbusarr = []
totaltt = []
totalttc = []
totalttb = []
totalttr = []
totaldcarr = []
totaldbarr = []
totaldrarr = []
#row_index_arr = []  # New array to track row index

# adding headers to the arrays
push!(mocperkgCO2arr, "mocperkgCO2")
push!(MParr, "mocomp")
push!(coinsallocatedarr, "coins allocated")
push!(coinsneededarr, "coins consumed")
push!(deltaarr, "market delta")
push!(cardemarr, "cardem")
push!(busdemarr, "busdem")
push!(bikedemarr, "bikedem")
push!(flowscararr, "flows car")
push!(flowsbusarr, "flows bus")
push!(flowsbikearr, "flows bike")
push!(mocoscararr, "mocoscar needed")
push!(mocosbusarr, "mocosbus needed")
push!(mocosbikearr, "mocosbike needed")
push!(emissionscararr, "car emissions")
push!(emissionsbusarr, "bus emissions")
push!(totaltt, "total_tt")
push!(totalttc, "total_tt_c")
push!(totalttb, "total_tt_b")
push!(totalttr, "total_tt_r")
push!(totaldcarr, "total dist car")
push!(totaldbarr, "total dist bus")
push!(totaldrarr, "total dist rad")
#push!(row_index_arr, "Row Index")  # Header for row index

# Row counter
row_index = 1  # Start at row 1 after the headers

function roundarray(arr)
    return [ (x isa Number && !isa(x, Int)) ? round(x, digits=2) : x for x in arr ]
end

# definition of objective function for brent algorithm
function objective_function_1(initialMP, mocperkgCO2)
    global row_index  # Ensure row_index is modified inside function
    mocomp = initialMP #[1]
    
    
    coinsneeded, 
    cardem, busdem, bikedem, 
    mocoscar, mocosbus, mocosbike, 
    flowscar, flowsbus, flowsbike, 
    totalemissions, caremissions, busemissions, 
    total_tt, total_tt_c, total_tt_b, total_tt_r, 
    link_travel_timec, link_travel_timeb, link_travel_timer, 
    totaldistcar, totaldistbus, totaldistrad = transportation_model(
        ta_data, 
        mocomp, 
        0,                      # third argument = bikeincentive = 0
        mocperkgCO2, 
        ttsc, 
        ttsb, 
        ttsr
    ) 
    



    # Objective is to minimize the absolute difference between coinsallocated and coinsneeded
    delta = abs(coinsallocated - coinsneeded)
    
    push!(mocperkgCO2arr, mocperkgCO2)
    push!(MParr, mocomp)
    push!(coinsallocatedarr, coinsallocated)
    push!(coinsneededarr, coinsneeded)
    push!(deltaarr, delta)
    push!(cardemarr, cardem)
    push!(busdemarr, busdem)
    push!(bikedemarr, bikedem)
    push!(flowscararr, flowscar)
    push!(flowsbusarr, flowsbus)
    push!(flowsbikearr, flowsbike)
    push!(mocoscararr, mocoscar)
    push!(mocosbusarr, mocosbus)
    push!(mocosbikearr, mocosbike)
    push!(emissionscararr, caremissions)
    push!(emissionsbusarr, busemissions)
    push!(totaltt, total_tt)
    push!(totalttc, total_tt_c)
    push!(totalttb, total_tt_b)
    push!(totalttr, total_tt_r)
    push!(totaldcarr, totaldistcar)
    push!(totaldbarr, totaldistbus)
    push!(totaldrarr, totaldistrad)

# Round arrays individually
rounded_columns = [
    roundarray(mocperkgCO2arr),
    roundarray(MParr),
    roundarray(coinsallocatedarr),
    roundarray(coinsneededarr),
    roundarray(deltaarr),
    roundarray(cardemarr),
    roundarray(busdemarr),
    roundarray(bikedemarr),
    roundarray(flowscararr),
    roundarray(flowsbusarr),
    roundarray(flowsbikearr),
    roundarray(mocoscararr),
    roundarray(mocosbusarr),
    roundarray(mocosbikearr),
    roundarray(emissionscararr),
    roundarray(emissionsbusarr),
    roundarray(totaltt),
    roundarray(totalttc),
    roundarray(totalttb),
    roundarray(totalttr),
    roundarray(totaldcarr),
    roundarray(totaldbarr),
    roundarray(totaldrarr)
]

result_matrix = hcat(rounded_columns...)
writedlm("results.csv", result_matrix, ',')
    return (delta)
end

objective_function_1 (generic function with 1 method)

In [4]:
# in case we change something, we can reload the transportation model
include("moco_functions/transportation_model.jl")       # includes mode choice and traffic assignment

transportation_model (generic function with 1 method)

In [5]:
# in case we change something we can reload the mode-choice model
include("moco_functions/mode_choice_tt.jl")             # compute mode choice model based on current travel times

mode_choice (generic function with 1 method)

In [6]:
coinsallocated = 655040 #655030 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6

mocperkgCO2 = 1 # 1.05
initialMP = 1

# in case we change the borders for the solution space
# borders = [0.0,
#            0.1,
#            0.2,
#            0.5,
#            1.0,
#            1.5,
#            2.0,
#            2.5,
#            3.0]#,
    #    1.4,
    #    1.6,
    #    1.8,
    #    2.0,
    #    2.2,
    #    2.4,
    #    2.6,
    #    2.8,
    #    3.0] #16

# in case we change the allocated coins
# coinsallocatedarr1 = [770000]

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]
final_result_row = ()

#for coinsallocated in coinsallocatedarr1
for mocperkgCO2 in mocperkg
    
    local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )


    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)
    # Find the row where delta is minimized
    min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
    println("Optimization happened at row: ", min_delta_index) 

    # Save the row of minimum delta in a separate file
    final_result_row = ( 
        mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
        mocomp = MParr[min_delta_index], 
        coins_allocated = coinsallocatedarr[min_delta_index], 
        coins_needed = coinsneededarr[min_delta_index], 
        market_delta = deltaarr[min_delta_index], 
        cardem = cardemarr[min_delta_index], 
        busdem = busdemarr[min_delta_index], 
        bikedem = bikedemarr[min_delta_index], 
        flows_car = flowscararr[min_delta_index], 
        flows_bus = flowsbusarr[min_delta_index], 
        flows_bike = flowsbikearr[min_delta_index], 
        mocos_car = mocoscararr[min_delta_index], 
        mocos_bus = mocosbusarr[min_delta_index], 
        mocos_bike = mocosbikearr[min_delta_index], 
        emissions_car = emissionscararr[min_delta_index], 
        emissions_bus = emissionsbusarr[min_delta_index], 
        total_tt = totaltt[min_delta_index], 
        total_tt_c = totalttc[min_delta_index], 
        total_tt_b = totalttb[min_delta_index], 
        total_tt_r = totalttr[min_delta_index], 
        total_dist_car = totaldcarr[min_delta_index], 
        total_dist_bus = totaldbarr[min_delta_index], 
        total_dist_rad = totaldrarr[min_delta_index]
    )
   
    # Write header and data to CSV
    writedlm("Final_Result.csv", final_result_row, ',')
    
end
println(final_result_row)

     0     9.487045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.593022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.298590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.164937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.277848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     6.994080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     6.994080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.203913e+02   

In [7]:
using CSV
using DataFrames

# Load the CSV
df = CSV.read("results.csv", DataFrame)

# Show the column names to verify
println("Column names: ", names(df))

# Extract relevant columns using their names
allocated     = df[!, Symbol("coins allocated")]
consumed      = df[!, Symbol("coins consumed")]
market_delta  = df[!, Symbol("market delta")]

# Compute ratios directly (no parsing needed!)
ratio_allocated = market_delta ./ allocated
percentage_allocated = ratio_allocated .* 100
ratio_consumed  = market_delta ./ consumed
percentage_consumed = ratio_consumed .* 100

# Add new columns with clear names
df[!, Symbol("market delta/allocated ratio")] = percentage_allocated
df[!, Symbol("market delta/consumed ratio")]  = percentage_consumed

# Save to new CSV
CSV.write("results_updated.csv", df)

println("Ratios calculated and saved to 'results_updated.csv'")


Column names: [:mocperkgCO2, :mocomp, Symbol("coins allocated"), Symbol("coins consumed"), Symbol("market delta"), :cardem, :busdem, :bikedem, Symbol("flows car"), Symbol("flows bus"), Symbol("flows bike"), Symbol("mocoscar needed"), Symbol("mocosbus needed"), Symbol("mocosbike needed"), Symbol("car emissions"), Symbol("bus emissions"), :total_tt, :total_tt_c, :total_tt_b, :total_tt_r, Symbol("total dist car"), Symbol("total dist bus"), Symbol("total dist rad")]
Ratios calculated and saved to 'results_updated.csv'


In [8]:
using CSV, DataFrames

df = CSV.read("results.csv", DataFrame)

# Open a tex file
open("table.tex", "w") do io
    println(io, "\\begin{tabular}{|" * "r|"^(size(df,2)) * "}")
    println(io, "\\hline")
    println(io, join(names(df), " & ") * " \\\\ \\hline")
    for row in eachrow(df)
        println(io, join(row, " & ") * " \\\\ \\hline")
    end
    println(io, "\\end{tabular}")
end

In [9]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
allocated = df[!, Symbol("coins allocated")]
consumed = df[!, Symbol("coins consumed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
max_val = max(maximum(allocated), maximum(consumed))
ytick_positions = 654000:1000:665000
ytick_labels = [mod(y, 1000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    allocated,
    label = "Allocated Coins",
    xlabel = "Number of Iterations",
    ylabel = "Number of Consumed Coins",
    lw = 2,
    legend = :topright,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (654000, 665000),
    minorgrid = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
)

# Add Consumed Coins as a second line
plot!(
    plt,
    x_values,
    consumed,
    label = "Consumed Coins",
    lw = 2,
    color = :red,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    allocated,
    label = "Allocated Data Points",
    marker = (:circle, 5),
    color = :blue,
)

scatter!(
    plt,
    x_values,
    consumed,
    label = "Consumed Data Points",
    marker = (:diamond, 5),
    color = :red,
)

# Save the plot
savefig(plt, "time_series_allocated_consumed.png")
println("Plot saved as time_series_allocated_consumed.png")

Plot saved as time_series_allocated_consumed.png


In [9]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
mocomp = df[!, Symbol("mocomp")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
max_val = max(maximum(allocated), maximum(consumed))
ytick_positions = 1:1:10
ytick_labels = [mod(y, 1) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    mocomp,
    label = "Market Price",
    xlabel = "Number of Iterations",
    ylabel = "Mobility Coin Market Price",
    lw = 2,
    legend = :right,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (0, 10),
    minorgrid = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    #grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    mocomp,
    label = "MoCo Market Price Points",
    marker = (:circle, 5),
    color = :red,
)

# Save the plot
savefig(plt, "time_series_mocomp.png")
println("Plot saved as time_series_mocomp.png")

Plot saved as time_series_mocomp.png


In [10]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
car_demand = df[!, Symbol("cardem")]
pt_demand = df[!, Symbol("busdem")]
bike_demand = df[!, Symbol("bikedem")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
max_val = max(maximum(allocated), maximum(consumed))
ytick_positions = 240000:10000:570000
ytick_labels = [mod(y, 20000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    car_demand,
    label = "Car Demand",
    xlabel = "Number of Iterations",
    ylabel = "Different Modes of Transport Demand (Car, PT, Bike)",
    lw = 2,
    legend = :horizontal,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (240000, 570000),
    minorgrid = true,
    size = (1000, 700),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line
plot!(
    plt,
    x_values,
    pt_demand,
    label = "PT Demand",
    lw = 2,
    color = :blue,
)

# Add bike Demand as a second line
plot!(
    plt,
    x_values,
    bike_demand,
    label = "Bike Demand",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    car_demand,
    label = "Car Demand Data",
    marker = (:circle, 5),
    color = :red,
)

scatter!(
    plt,
    x_values,
    pt_demand,
    label = "PT Demand Data",
    marker = (:diamond, 5),
    color = :blue,
)

scatter!(
    plt,
    x_values,
    bike_demand,
    label = "Bike Demand Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Save the plot
savefig(plt, "time_series_modes_demand.png")
println("Plot saved as time_series_modes_demand.png")

Plot saved as time_series_modes_demand.png


In [21]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
mocoscar = df[!, Symbol("mocoscar needed")]
mocosbus = df[!, Symbol("mocosbus needed")]
#mocosbike = df[!, Symbol("mocosbike needed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 321000:500:334000
ytick_labels = [mod(y, 500) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    mocoscar,
    label = "Car Consumed MoCos",
    xlabel = "Number of Iterations",
    ylabel = "Transport MoCos Consumption by Motorized Modes (Car and PT)",
    lw = 2,
    legend = :horizontal,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (321000, 334000),
    minorgrid = true,
    size = (1000, 700),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line
plot!(
    plt,
    x_values,
    mocosbus,
    label = "PT Consumed MoCos",
    lw = 2,
    color = :blue,
)

# Add bike Demand as a second line
#plot!(
 #   plt,
  #  x_values,
   # bike_demand,
   # label = "Bike Demand",
   # lw = 2,
   # color = :green,
#)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    mocoscar,
    label = "Car MoCos Consumption",
    marker = (:circle, 5),
    color = :red,
)

scatter!(
    plt,
    x_values,
    mocosbus,
    label = "PT MoCos Consumption",
    marker = (:diamond, 5),
    color = :blue,
)

#scatter!(
  #  plt,
   # x_values,
   # bike_demand,
   # label = "Bike Demand Data",
   # marker = (:utriangle, 5),
   # color = :forestgreen,
#)

# Save the plot
savefig(plt, "time_series_modes_mocos.png")
println("Plot saved as time_series_modes_mocos.png")

Plot saved as time_series_modes_mocos.png


In [19]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
car_emission = df[!, Symbol("car emissions")]
pt_emission = df[!, Symbol("bus emissions")]
#bike_emission = df[!, Symbol("bike emissions")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 324500:500:336500
ytick_labels = [mod(y, 500) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    car_emission,
    label = "Car Emissions",
    xlabel = "Number of Iterations",
    ylabel = "Transport Emissions Generation by Motorized Modes (Car and PT)",
    lw = 2,
    legend = :horizontal,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (324500,336500),
    minorgrid = true,
    size = (1000, 700),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line
plot!(
    plt,
    x_values,
    pt_emission,
    label = "PT Emissions",
    lw = 2,
    color = :blue,
)

# Add bike Demand as a second line
#plot!(
 #   plt,
  #  x_values,
   # bike_demand,
   # label = "Bike Demand",
   # lw = 2,
   # color = :green,
#)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    car_emission,
    label = "Car Emissions Data",
    marker = (:circle, 5),
    color = :red,
)

scatter!(
    plt,
    x_values,
    pt_emission,
    label = "PT Emissions Data",
    marker = (:diamond, 5),
    color = :blue,
)

#scatter!(
 #   plt,
 #   x_values,
 #   bike_demand,
 #   label = "Bike Demand Data",
 #   marker = (:utriangle, 5),
 #   color = :forestgreen,
#)

# Save the plot
savefig(plt, "time_series_modes_emission.png")
println("Plot saved as time_series_modes_emission.png")

Plot saved as time_series_modes_emission.png


In [29]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
cartd = df[!, Symbol("total dist car")]
pttd = df[!, Symbol("total dist bus")]
biketd = df[!, Symbol("total dist rad")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 2200000:100000:5200000
ytick_labels = [mod(y, 200000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)


# Create the plot
plt = plot(
    x_values,
    cartd,
    label = "Car Total Distance",
    xlabel = "Number of Iterations",
    ylabel = "Different Modes of Transport Total Distances (Car, PT, Bike)",
    lw = 2,
    legend = :horizontal,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (2200000, 5200000),
    minorgrid = true,
    size = (1000, 700),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line
plot!(
    plt,
    x_values,
    pttd,
    label = "PT Total Distance",
    lw = 2,
    color = :blue,
)

# Add bike Demand as a second line
plot!(
    plt,
    x_values,
    biketd,
    label = "Bike Total Distance",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    cartd,
    label = "Car Total Distance Data",
    marker = (:circle, 5),
    color = :red,
)

scatter!(
    plt,
    x_values,
    pttd,
    label = "PT Total Distance Data",
    marker = (:diamond, 5),
    color = :blue,
)

scatter!(
    plt,
    x_values,
    biketd,
    label = "Bike Total Distance Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Save the plot
savefig(plt, "time_series_modes_distance.png")
println("Plot saved as time_series_modes_distance.png")

Plot saved as time_series_modes_distance.png


In [ ]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = df[!, Symbol("mocomp")]
market_delta = df[!, Symbol("market delta")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 0:500:10000
ytick_labels = [mod(y, 500) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 3:1:10
xtick_labels = [mod(x, 1) == 0 ? string(x) : "" for x in xtick_positions]
xticks = (xtick_positions, xtick_labels)


# Create the plot
plt = plot(
    x_values,
    market_delta,
    label = "Variation between Market Delta and Market Price",
    xlabel = "MobilityCoin Market Price",
    ylabel = "The Absolute Difference between Allocated and Consumed Coins",
    lw = 2,
    legend = :topright,
    title = "",
    xticks = xticks,
    yticks = yticks,
    ylims = (0, 10000),
    xlims = (3, 10),
    minorgrid = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    market_delta,
    label = "",
    marker = (:circle, 2.7),
    color = :red,
)

# Save the plot
savefig(plt, "time_series_mocomp_marketdelta.png")
println("Plot saved as time_series_mocomp_marketdelta.png")

Plot saved as time_series_mocomp_marketdelta.png


In [30]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df = CSV.read("results_updated.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
x_values = 1:16
allocated = df[!, Symbol("market delta/allocated ratio")]
consumed = df[!, Symbol("market delta/consumed ratio")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)

# Major and minor tick positions
y_minor = 0:0.05:2           # minor ticks every 0.01 (gridlines, thin ticks)
y_major = 0:0.1:2           # major ticks every 0.05 (labeled)

# Build a label vector: show labels only at the major positions
ytick_labels = [isapprox(mod(y, 0.05), 0; atol=1e-8) ? string(round(y, digits = 2)) : ""
                for y in y_minor]

yticks = (y_minor, ytick_labels)   # minor + major ticks; labels only on majors

# Create the plot
plt = plot(
    x_values,
    allocated,
    label = "Market Delta/Allocated Coins Ratio",
    xlabel = "Number of Iterations",
    ylabel = "Ratio of Market Delta to Allocated and Consumed Coins (%)",
    lw = 2,
    legend = :topright,
    title = "",
    xticks = 1:1:16,
    yticks = yticks,
    ylims = (0, 2),
    minorgrid = true,
    #minorgridalpha  = 0.3,
    yminorgridstyle = :dot,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
)

# Add Consumed Coins as a second line
plot!(
    plt,
    x_values,
    consumed,
    label = "Market Delta/Consumed Coins ratio",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Optionally add scatter markers
scatter!(
    plt,
    x_values,
    allocated,
    label = "Market Delta/Allocated Coins Data Points",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    x_values,
    consumed,
    label = "Market Delta/Consumed Coins Data Points",
    marker = (:diamond, 4),
    color = :red,
)

# Save the plot
savefig(plt, "time_series_market_allocated_consumed.png")
println("Plot saved as time_series_market_allocated_consumed.png")

Plot saved as time_series_market_allocated_consumed.png


In [7]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

println("Estimated parameters:")
println("L = $(round(L_fit, digits=6)), k = $(round(k_fit, digits=6)), x0 = $(round(x0_fit, digits=6))")
println("Best-fit equation: y = $L_fit / (1 + exp(-$k_fit * (x - $x0_fit)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model(x_values, fit.param)

# Print the results
println("\nModel Predictions:")
for (x, y) in zip(x_values, y_values)
    println("x = $(lpad(x, 2)) → y = $(round(y, digits=6))")
end

# Calculate and print the sum of all y-values
total_y = sum(y_values)
println("\nSum of all predicted y-values from x = 1 to 20: $(round(total_y, digits=6))")

Estimated parameters:
L = 897.273055, k = -0.22186, x0 = -32.686391
Best-fit equation: y = 897.273055249895 / (1 + exp(--0.2218600685464287 * (x - -32.68639136317905)))

Model Predictions:
x =  1 → y = 0.509221
x =  2 → y = 0.407946
x =  3 → y = 0.326806
x =  4 → y = 0.261799
x =  5 → y = 0.209721
x =  6 → y = 0.168
x =  7 → y = 0.134577
x =  8 → y = 0.107803
x =  9 → y = 0.086356
x = 10 → y = 0.069175
x = 11 → y = 0.055412
x = 12 → y = 0.044387
x = 13 → y = 0.035555
x = 14 → y = 0.028481
x = 15 → y = 0.022814
x = 16 → y = 0.018275
x = 17 → y = 0.014639
x = 18 → y = 0.011726
x = 19 → y = 0.009393
x = 20 → y = 0.007524

Sum of all predicted y-values from x = 1 to 20: 2.52961


In [3]:
using LsqFit

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

# Round parameters for display
L_fit_rnd = round(L_fit, digits=2)
k_fit_rnd = round(k_fit, digits=2)
x0_fit_rnd = round(x0_fit, digits=2)

println("Estimated parameters (rounded to 2 decimals):")
println("L = $L_fit_rnd, k = $k_fit_rnd, x0 = $x0_fit_rnd")

println("Best-fit equation: y = $L_fit_rnd / (1 + exp(-$k_fit_rnd * (x - $x0_fit_rnd)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model(x_values, fit.param)

# Print rounded results
println("\nModel Predictions (rounded to 2 decimals):")
for (x, y) in zip(x_values, y_values)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3))")
end

# Calculate and print the sum of all y-values, rounded
total_y = sum(y_values)
println("\nSum of all predicted y-values from x = 1 to 20: $(round(total_y, digits=3))")

Estimated parameters (rounded to 2 decimals):
L = 897.27, k = -0.22, x0 = -32.69
Best-fit equation: y = 897.27 / (1 + exp(--0.22 * (x - -32.69)))

Model Predictions (rounded to 2 decimals):
x =   1 → y = 0.509
x =   2 → y = 0.408
x =   3 → y = 0.327
x =   4 → y = 0.262
x =   5 → y = 0.21
x =   6 → y = 0.168
x =   7 → y = 0.135
x =   8 → y = 0.108
x =   9 → y = 0.086
x =  10 → y = 0.069
x =  11 → y = 0.055
x =  12 → y = 0.044
x =  13 → y = 0.036
x =  14 → y = 0.028
x =  15 → y = 0.023
x =  16 → y = 0.018
x =  17 → y = 0.015
x =  18 → y = 0.012
x =  19 → y = 0.009
x =  20 → y = 0.008

Sum of all predicted y-values from x = 1 to 20: 2.53


In [ ]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (in fraction)

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

# Round parameters for display
L_fit_rnd = round(L_fit, digits=2)
k_fit_rnd = round(k_fit, digits=2)
x0_fit_rnd = round(x0_fit, digits=2)

println("Estimated parameters (rounded to 2 decimals):")
println("L = $L_fit_rnd, k = $k_fit_rnd, x0 = $x0_fit_rnd")

println("Best-fit equation: y = $L_fit_rnd / (1 + exp(-$k_fit_rnd * (x - $x0_fit_rnd)))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_frac = logistic_model(x_values, fit.param)

# Convert to percentage
y_values_pct = y_values_frac .* 100
y_data_pct = y_data .* 100

# Print rounded results
println("\nModel Predictions (rounded to 2 decimals):")
for (x, y) in zip(x_values, y_values_pct)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=2)) %")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_frac)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt = plot(
    x_values,
    y_values_pct,
    label = "Fitted Logistic Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Logistic Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern"
)

scatter!(
    plt,
    x_data,
    y_data_pct,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt, "logistic_fit_crowdfunding_percentage100.png")
println("Plot saved as logistic_fit_crowdfunding_percentage.png")

Estimated parameters (rounded to 2 decimals):
L = 897.27, k = -0.22, x0 = -32.69
Best-fit equation: y = 897.27 / (1 + exp(--0.22 * (x - -32.69)))

Model Predictions (rounded to 2 decimals):
x =   1 → y = 50.92 %
x =   2 → y = 40.79 %
x =   3 → y = 32.68 %
x =   4 → y = 26.18 %
x =   5 → y = 20.97 %
x =   6 → y = 16.8 %
x =   7 → y = 13.46 %
x =   8 → y = 10.78 %
x =   9 → y = 8.64 %
x =  10 → y = 6.92 %
x =  11 → y = 5.54 %
x =  12 → y = 4.44 %
x =  13 → y = 3.56 %
x =  14 → y = 2.85 %
x =  15 → y = 2.28 %
x =  16 → y = 1.83 %
x =  17 → y = 1.46 %
x =  18 → y = 1.17 %
x =  19 → y = 0.94 %
x =  20 → y = 0.75 %

Sum of all predicted y-values from x = 1 to 20 (fraction): 2.53
Plot saved as logistic_fit_crowdfunding_percentage.png


In [17]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (in fraction)

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Round parameters for display
a_fit_rnd = round(a_fit, digits=2)
b_fit_rnd = round(b_fit, digits=2)

println("Exact parameters (rounded to 2 decimals): a = $a_fit_rnd, b = $b_fit_rnd")
println("Best-fit equation: y = $a_fit_rnd * exp($b_fit_rnd * x)")

# Define the exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_func = exp_model.(x_values)

# Clamp values between 0 and 1
y_values_func = clamp.(y_values_func, 0.0, 1.0)

# Convert to percentage
y_values_perc = y_values_func .* 100
y_data_perc = y_data .* 100

# Print rounded predictions
println("\nModel Predictions (rounded to 3 decimals):")
for (x, y) in zip(x_values, y_values_perc)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3)) %")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_func)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt2 = plot(
    x_values,
    y_values_perc,
    label = "Exponential Model Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Exponential Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefontsize = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern"
)

scatter!(
    plt2,
    x_data,
    y_data_perc,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt2, "exponential_fit_crowdfunding100.png")
println("Plot saved as exponential_fit_crowdfunding.png")

Exact parameters (rounded to 2 decimals): a = 0.58, b = -0.18
Best-fit equation: y = 0.58 * exp(-0.18 * x)

Model Predictions (rounded to 3 decimals):
x =   1 → y = 48.446 %
x =   2 → y = 40.639 %
x =   3 → y = 34.09 %
x =   4 → y = 28.597 %
x =   5 → y = 23.988 %
x =   6 → y = 20.123 %
x =   7 → y = 16.88 %
x =   8 → y = 14.16 %
x =   9 → y = 11.878 %
x =  10 → y = 9.964 %
x =  11 → y = 8.358 %
x =  12 → y = 7.011 %
x =  13 → y = 5.882 %
x =  14 → y = 4.934 %
x =  15 → y = 4.139 %
x =  16 → y = 3.472 %
x =  17 → y = 2.912 %
x =  18 → y = 2.443 %
x =  19 → y = 2.049 %
x =  20 → y = 1.719 %

Sum of all predicted y-values from x = 1 to 20 (fraction): 2.917
Plot saved as exponential_fit_crowdfunding.png


In [11]:
using LsqFit
using Plots

# ==========================================
# Data (same for both models)
# ==========================================
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (fraction)

# ==========================================
# Exponential Fit
# ==========================================
# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Exponential model function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute predictions
x_values = 1:20
y_exp = exp_model.(x_values)

# Clamp between 0 and 1
y_exp = clamp.(y_exp, 0.0, 1.0)

# Convert to percentages
y_exp_pct = y_exp .* 100

# ==========================================
# Logistic Fit
# ==========================================
# Generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess
p0 = [1.0, 0.5, 10.0]

# Curve fit
fit = curve_fit(logistic_model, x_data, y_data, p0)

L_fit, k_fit, x0_fit = fit.param

# Compute predictions
y_logistic = logistic_model(x_values, fit.param)

# Convert to percentages
y_logistic_pct = y_logistic .* 100

# ==========================================
# Prepare ticks
# ==========================================
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# ==========================================
# Plot both fits and data points
# ==========================================
plt_combined = plot(
    x_values,
    y_exp_pct,
    label = "Exponential Model Curve",
    lw = 2,
    color = :blue,
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    legend = :topright,
    title = "",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black
)

# Add logistic curve
plot!(
    plt_combined,
    x_values,
    y_logistic_pct,
    label = "Logistic Model Curve",
    lw = 2,
    color = :green,
)

# Add data points
scatter!(
    plt_combined,
    x_data,
    y_data .* 100,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save combined plot
savefig(plt_combined, "combined_fit_crowdfunding.png")
println("Combined plot saved as combined_fit_crowdfunding.png")

# Optional: display in a window if running interactively
# display(plt_combined)


Combined plot saved as combined_fit_crowdfunding.png


In [8]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Define the generalized logistic function
logistic_model(x, p) = p[1] ./ (1 .+ exp.(-p[2] .* (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.5, 10.0]

# Fit the model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract estimated parameters
L_fit, k_fit, x0_fit = fit.param

println("Estimated parameters: L = ", L_fit, ", k = ", k_fit, ", x0 = ", x0_fit)
println("Best-fit equation: y = ", L_fit, " / (1 + exp(-", k_fit, " * (x - ", x0_fit, ")))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model(x_values, fit.param)

# Print the results
println("\nModel Predictions:")
for (x, y) in zip(x_values, y_values)
    println("x = $x → y = $y")
end

# Calculate and print the sum of all y-values
total_y = sum(y_values)
println("\nSum of all predicted y-values from x = 1 to 20: $total_y")

Estimated parameters: L = 897.2730544491106, k = -0.22186006853514756, x0 = -32.68639136103844
Best-fit equation: y = 897.2730544491106 / (1 + exp(--0.22186006853514756 * (x - -32.68639136103844)))

Model Predictions:
x = 1 → y = 0.5092213147353761
x = 2 → y = 0.4079463146533796
x = 3 → y = 0.3268057581706648
x = 4 → y = 0.2617993542200777
x = 5 → y = 0.20972064318944014
x = 6 → y = 0.16799980408159657
x = 7 → y = 0.1345774674972492
x = 8 → y = 0.10780346216966578
x = 9 → y = 0.086355596300971
x = 10 → y = 0.06917452736659695
x = 11 → y = 0.055411544938075194
x = 12 → y = 0.04438671369780725
x = 13 → y = 0.03555532585095742
x = 14 → y = 0.028481016396139047
x = 15 → y = 0.022814219895845107
x = 16 → y = 0.018274908737578983
x = 17 → y = 0.014638762785335156
x = 18 → y = 0.011726088823310974
x = 19 → y = 0.009392943399471526
x = 20 → y = 0.0075240211282578415

Sum of all predicted y-values from x = 1 to 20: 2.529609788037796


In [9]:
# First Month - First Group
# Initial setup
initial_x = 20
z = 645860 * initial_x * 1.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]

println("Iteration Results:")
println("x\t   y\t\t\t    t\t\t     r\t\t     updated_z")

for x in 30:-1:11
    y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
    t = y * 645860
    r = 645860 - (t)
    z -= (r + t)

    @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\n", x, y, t, r, z)

    # Store values
    push!(x_vals, x)
    push!(y_vals, y)
    push!(t_vals, t)
    push!(r_vals, r)
    push!(z_vals, z)
end

# Create a DataFrame from the results
results_df = DataFrame(x = x_vals, y = y_vals, t = t_vals, r = r_vals, z = z_vals)

Iteration Results:
x	   y			    t		     r		     updated_z
30	0.000818	528.53	645331.47	18729940.00
29	0.001022	659.81	645200.19	18084080.00
28	0.001275	823.70	645036.30	17438220.00
27	0.001592	1028.31	644831.69	16792360.00
26	0.001988	1283.74	644576.26	16146500.00
25	0.002481	1602.61	644257.39	15500640.00
24	0.003098	2000.69	643859.31	14854780.00
23	0.003867	2497.65	643362.35	14208920.00
22	0.004828	3118.06	642741.94	13563060.00
21	0.006027	3892.57	641967.43	12917200.00
20	0.007524	4859.46	641000.54	12271340.00
19	0.009393	6066.53	639793.47	11625480.00
18	0.011726	7573.41	638286.59	10979620.00
17	0.014639	9454.59	636405.41	10333760.00
16	0.018275	11803.03	634056.97	9687900.00
15	0.022814	14734.79	631125.21	9042040.00
14	0.028481	18394.75	627465.25	8396180.00
13	0.035555	22963.76	622896.24	7750320.00
12	0.044387	28667.60	617192.40	7104460.00
11	0.055412	35788.10	610071.90	6458600.00


,x,y,t,r,z
,Int64,Float64,Float64,Float64,Float64
1,30,0.000818328,528.525,6.45331e5,1.87299e7
2,29,0.0010216,659.809,6.452e5,1.80841e7
3,28,0.00127536,823.703,6.45036e5,1.74382e7
4,27,0.00159215,1028.31,6.44832e5,1.67924e7
5,26,0.00198764,1283.74,6.44576e5,1.61465e7
6,25,0.00248136,1602.61,6.44257e5,1.55006e7
7,24,0.00309772,2000.69,6.43859e5,1.48548e7
8,23,0.00386718,2497.65,6.43362e5,1.42089e7
9,22,0.00482776,3118.06,6.42742e5,1.35631e7


In [11]:
# First Month - First Group -Second Version
# Initial setup
initial_x = 20
z = 645860 * initial_x * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]

println("Iteration Results:")
println("x\t   y\t\t\t    t\t\t     r\t\t     updated_z")

for x in 10:-1:1
    y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
    t = y * 645860
    r = 645860 - (t)
    z -= (r + t)

    @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\n", x, y, t, r, z)

    # Store values
    push!(x_vals, x)
    push!(y_vals, y)
    push!(t_vals, t)
    push!(r_vals, r)
    push!(z_vals, z)
end

# Create a DataFrame from the results
results_df = DataFrame(x = x_vals, y = y_vals, t = t_vals, r = r_vals, z = z_vals)

Iteration Results:
x	   y			    t		     r		     updated_z
10	0.069175	44677.06	601182.94	5812740.00
 9	0.086356	55773.63	590086.37	5166880.00
 8	0.107803	69625.94	576234.06	4521020.00
 7	0.134577	86918.20	558941.80	3875160.00
 6	0.168000	108504.35	537355.65	3229300.00
 5	0.209721	135450.17	510409.83	2583440.00
 4	0.261799	169085.73	476774.27	1937580.00
 3	0.326806	211070.77	434789.23	1291720.00
 2	0.407946	263476.21	382383.79	645860.00
 1	0.509221	328885.68	316974.32	0.00


,x,y,t,r,z
,Int64,Float64,Float64,Float64,Float64
1,10,0.0691745,44677.1,6.01183e5,5.81274e6
2,9,0.0863556,55773.6,5.90086e5,5.16688e6
3,8,0.107803,69625.9,5.76234e5,4.52102e6
4,7,0.134577,86918.2,5.58942e5,3.87516e6
5,6,0.168,1.08504e5,5.37356e5,3.2293e6
6,5,0.209721,1.3545e5,5.1041e5,2.58344e6
7,4,0.261799,1.69086e5,4.76774e5,1.93758e6
8,3,0.326806,2.11071e5,4.34789e5,1.29172e6
9,2,0.407946,2.63476e5,3.82384e5,645860.0


In [10]:
# All Months - Second Group
# Initial setup
initial_x = 20
z = 645860 * initial_x * 1 

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]

println("Iteration Results:")
println("x\t   y\t\t\t    t\t\t     r\t\t     updated_z")

for x in 20:-1:1
    y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
    t = y * 645860
    r = 645860 - (t)
    z -= (r + t)

    @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\n", x, y, t, r, z)

    # Store values
    push!(x_vals, x)
    push!(y_vals, y)
    push!(t_vals, t)
    push!(r_vals, r)
    push!(z_vals, z)
end

# Create a DataFrame from the results
results_df = DataFrame(x = x_vals, y = y_vals, t = t_vals, r = r_vals, z = z_vals)

Iteration Results:
x	   y			    t		     r		     updated_z
20	0.007524	4859.46	641000.54	12271340.00
19	0.009393	6066.53	639793.47	11625480.00
18	0.011726	7573.41	638286.59	10979620.00
17	0.014639	9454.59	636405.41	10333760.00
16	0.018275	11803.03	634056.97	9687900.00
15	0.022814	14734.79	631125.21	9042040.00
14	0.028481	18394.75	627465.25	8396180.00
13	0.035555	22963.76	622896.24	7750320.00
12	0.044387	28667.60	617192.40	7104460.00
11	0.055412	35788.10	610071.90	6458600.00
10	0.069175	44677.06	601182.94	5812740.00
 9	0.086356	55773.63	590086.37	5166880.00
 8	0.107803	69625.94	576234.06	4521020.00
 7	0.134577	86918.20	558941.80	3875160.00
 6	0.168000	108504.35	537355.65	3229300.00
 5	0.209721	135450.17	510409.83	2583440.00
 4	0.261799	169085.73	476774.27	1937580.00
 3	0.326806	211070.77	434789.23	1291720.00
 2	0.407946	263476.21	382383.79	645860.00
 1	0.509221	328885.68	316974.32	0.00


,x,y,t,r,z
,Int64,Float64,Float64,Float64,Float64
1,20,0.00752402,4859.46,6.41001e5,1.22713e7
2,19,0.00939294,6066.53,6.39793e5,1.16255e7
3,18,0.0117261,7573.41,6.38287e5,1.09796e7
4,17,0.0146388,9454.59,6.36405e5,1.03338e7
5,16,0.0182749,11803.0,634057.0,9.6879e6
6,15,0.0228142,14734.8,6.31125e5,9.04204e6
7,14,0.028481,18394.7,6.27465e5,8.39618e6
8,13,0.0355553,22963.8,6.22896e5,7.75032e6
9,12,0.0443867,28667.6,6.17192e5,7.10446e6


In [11]:
#Second Month to the last month- First Group
# Initial setup
initial_x = 20
z1 = 645860 * initial_x * 0.5 
z2 = 645860 * initial_x * 1

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]
j_vals = Int[]

println("Iteration Results:")
println("x\t  y\t\t      t\t\t     r\t\t     z\t\t     j")

    j = 21  # Total iterations

    # Phase 1 (j = 21 to 11): x goes from 10 to 1
    for x in 10:-1:1
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860 
        r = 645860 - t
        z1 -= (r + t)
        z = z1
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

    # Phase 2 (j = 10 to 1): x goes from 20 to 11
    for x in 20:-1:11
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860 
        r = 645860 - t
        z2 -= (r + t)
        z = z2
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

# Create a DataFrame from the results
results_df = DataFrame(
    x = x_vals,
    y = y_vals,
    t = t_vals,
    r = r_vals,
    z = z_vals,
    j = j_vals
)

Iteration Results:
x	  y		      t		     r		     z		     j
10	0.069175	44677.06	601182.94	5812740.00	20
 9	0.086356	55773.63	590086.37	5166880.00	19
 8	0.107803	69625.94	576234.06	4521020.00	18
 7	0.134577	86918.20	558941.80	3875160.00	17
 6	0.168000	108504.35	537355.65	3229300.00	16
 5	0.209721	135450.17	510409.83	2583440.00	15
 4	0.261799	169085.73	476774.27	1937580.00	14
 3	0.326806	211070.77	434789.23	1291720.00	13
 2	0.407946	263476.21	382383.79	645860.00	12
 1	0.509221	328885.68	316974.32	0.00	11
20	0.007524	4859.46	641000.54	12271340.00	10
19	0.009393	6066.53	639793.47	11625480.00	 9
18	0.011726	7573.41	638286.59	10979620.00	 8
17	0.014639	9454.59	636405.41	10333760.00	 7
16	0.018275	11803.03	634056.97	9687900.00	 6
15	0.022814	14734.79	631125.21	9042040.00	 5
14	0.028481	18394.75	627465.25	8396180.00	 4
13	0.035555	22963.76	622896.24	7750320.00	 3
12	0.044387	28667.60	617192.40	7104460.00	 2
11	0.055412	35788.10	610071.90	6458600.00	 1


,x,y,t,r,z,j
,Int64,Float64,Float64,Float64,Float64,Int64
1,10,0.0691745,44677.1,6.01183e5,5.81274e6,20
2,9,0.0863556,55773.6,5.90086e5,5.16688e6,19
3,8,0.107803,69625.9,5.76234e5,4.52102e6,18
4,7,0.134577,86918.2,5.58942e5,3.87516e6,17
5,6,0.168,1.08504e5,5.37356e5,3.2293e6,16
6,5,0.209721,1.3545e5,5.1041e5,2.58344e6,15
7,4,0.261799,1.69086e5,4.76774e5,1.93758e6,14
8,3,0.326806,2.11071e5,4.34789e5,1.29172e6,13
9,2,0.407946,2.63476e5,3.82384e5,645860.0,12


In [12]:
#Second Month to the last month- First Group -Second Version
# Initial setup
initial_x = 20
z1 = 645860 * initial_x * 0.5
z2 = 645860 * initial_x * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]
j_vals = Int[]

println("Iteration Results:")
println("x\t  y\t\t      t\t\t     r\t\t     z\t\t     j")

    j = 21  # Total iterations

    # Phase 1 (j = 21 to 11): x goes from 10 to 1
    for x in 20:-1:11
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860 * 0.5
        r = 645860 * 0.5 - t
        z1 -= (r + t)
        z = z1
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

    # Phase 2 (j = 10 to 1): x goes from 20 to 11
    for x in 10:-1:1
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860
        r = 645860 - t
        z2 -= (r + t)
        z = z2
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

# Create a DataFrame from the results
results_df = DataFrame(
    x = x_vals,
    y = y_vals,
    t = t_vals,
    r = r_vals,
    z = z_vals,
    j = j_vals
)

Iteration Results:
x	  y		      t		     r		     z		     j
20	0.007524	2429.73	320500.27	6135670.00	20
19	0.009393	3033.26	319896.74	5812740.00	19
18	0.011726	3786.71	319143.29	5489810.00	18
17	0.014639	4727.30	318202.70	5166880.00	17
16	0.018275	5901.52	317028.48	4843950.00	16
15	0.022814	7367.40	315562.60	4521020.00	15
14	0.028481	9197.37	313732.63	4198090.00	14
13	0.035555	11481.88	311448.12	3875160.00	13
12	0.044387	14333.80	308596.20	3552230.00	12
11	0.055412	17894.05	305035.95	3229300.00	11
10	0.069175	44677.06	601182.94	5812740.00	10
 9	0.086356	55773.63	590086.37	5166880.00	 9
 8	0.107803	69625.94	576234.06	4521020.00	 8
 7	0.134577	86918.20	558941.80	3875160.00	 7
 6	0.168000	108504.35	537355.65	3229300.00	 6
 5	0.209721	135450.17	510409.83	2583440.00	 5
 4	0.261799	169085.73	476774.27	1937580.00	 4
 3	0.326806	211070.77	434789.23	1291720.00	 3
 2	0.407946	263476.21	382383.79	645860.00	 2
 1	0.509221	328885.68	316974.32	0.00	 1


,x,y,t,r,z,j
,Int64,Float64,Float64,Float64,Float64,Int64
1,20,0.00752402,2429.73,3.205e5,6.13567e6,20
2,19,0.00939294,3033.26,3.19897e5,5.81274e6,19
3,18,0.0117261,3786.71,3.19143e5,5.48981e6,18
4,17,0.0146388,4727.3,3.18203e5,5.16688e6,17
5,16,0.0182749,5901.52,3.17028e5,4.84395e6,16
6,15,0.0228142,7367.4,3.15563e5,4.52102e6,15
7,14,0.028481,9197.37,3.13733e5,4.19809e6,14
8,13,0.0355553,11481.9,3.11448e5,3.87516e6,13
9,12,0.0443867,14333.8,3.08596e5,3.55223e6,12


In [13]:
#last month- First Group
# Initial setup
initial_x = 20
z1 = 645860 * initial_x * 0.5
z2 = 645860 * initial_x * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]
j_vals = Int[]

println("Iteration Results:")
println("x\t  y\t\t      t\t\t     r\t\t     z\t\t     j")

    j = 21  # Total iterations

    # Phase 1 (j = 21 to 11): x goes from 10 to 1
    for x in 10:-1:1
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860
        r = 645860 - t
        z1 -= (r + t)
        z = z1
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

    # Phase 2 (j = 10 to 1): x goes from 20 to 11
    for x in 10:-1:1
        y = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844)))
        t = y * 645860
        r = 645860 - t
        z2 -= (r + t)
        z = z2
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

# Create a DataFrame from the results
results_df = DataFrame(
    x = x_vals,
    y = y_vals,
    t = t_vals,
    r = r_vals,
    z = z_vals,
    j = j_vals
)

Iteration Results:
x	  y		      t		     r		     z		     j
10	0.069175	44677.06	601182.94	5812740.00	20
 9	0.086356	55773.63	590086.37	5166880.00	19
 8	0.107803	69625.94	576234.06	4521020.00	18
 7	0.134577	86918.20	558941.80	3875160.00	17
 6	0.168000	108504.35	537355.65	3229300.00	16
 5	0.209721	135450.17	510409.83	2583440.00	15
 4	0.261799	169085.73	476774.27	1937580.00	14
 3	0.326806	211070.77	434789.23	1291720.00	13
 2	0.407946	263476.21	382383.79	645860.00	12
 1	0.509221	328885.68	316974.32	0.00	11
10	0.069175	44677.06	601182.94	5812740.00	10
 9	0.086356	55773.63	590086.37	5166880.00	 9
 8	0.107803	69625.94	576234.06	4521020.00	 8
 7	0.134577	86918.20	558941.80	3875160.00	 7
 6	0.168000	108504.35	537355.65	3229300.00	 6
 5	0.209721	135450.17	510409.83	2583440.00	 5
 4	0.261799	169085.73	476774.27	1937580.00	 4
 3	0.326806	211070.77	434789.23	1291720.00	 3
 2	0.407946	263476.21	382383.79	645860.00	 2
 1	0.509221	328885.68	316974.32	0.00	 1


,x,y,t,r,z,j
,Int64,Float64,Float64,Float64,Float64,Int64
1,10,0.0691745,44677.1,6.01183e5,5.81274e6,20
2,9,0.0863556,55773.6,5.90086e5,5.16688e6,19
3,8,0.107803,69625.9,5.76234e5,4.52102e6,18
4,7,0.134577,86918.2,5.58942e5,3.87516e6,17
5,6,0.168,1.08504e5,5.37356e5,3.2293e6,16
6,5,0.209721,1.3545e5,5.1041e5,2.58344e6,15
7,4,0.261799,1.69086e5,4.76774e5,1.93758e6,14
8,3,0.326806,2.11071e5,4.34789e5,1.29172e6,13
9,2,0.407946,2.63476e5,3.82384e5,645860.0,12


In [14]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

println("Exact parameters: a = ", a_fit, ", b = ", b_fit)
println("Best-fit equation: y = ", a_fit, " * exp(", b_fit, " * x)")

# Define the exact exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = exp_model.(x_values)

# Ensure all values remain between 0 and 1
y_values = clamp.(y_values, 0.0, 1.0)

# Print the results
println("\nModel Predictions:")
for (x, y) in zip(x_values, y_values)
    println("x = $x → y = $y")
end

Exact parameters: a = 0.5775211856783368, b = -0.17571864924550018
Best-fit equation: y = 0.5775211856783368 * exp(-0.17571864924550018 * x)

Model Predictions:
x = 1 → y = 0.48445593493827627
x = 2 → y = 0.4063877805993413
x = 3 → y = 0.3408999999999999
x = 4 → y = 0.2859653157597631
x = 5 → y = 0.23988313821525675
x = 6 → y = 0.20122692098905487
x = 7 → y = 0.16879999999999998
x = 8 → y = 0.14159854884202994
x = 9 → y = 0.11878050375692384
x = 10 → y = 0.09963949622455986
x = 11 → y = 0.08358298621296567
x = 12 → y = 0.07011391916847948
x = 13 → y = 0.05881533890926589
x = 14 → y = 0.04933748009007248
x = 15 → y = 0.041386940665146976
x = 16 → y = 0.03471759916585314
x = 17 → y = 0.02912299562300992
x = 18 → y = 0.024429940273406368
x = 19 → y = 0.02049315219793725
x = 20 → y = 0.017190761921959546


In [8]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Round parameters for display
a_fit_rnd = round(a_fit, digits=2)
b_fit_rnd = round(b_fit, digits=2)

println("Exact parameters (rounded to 2 decimals): a = $a_fit_rnd, b = $b_fit_rnd")
println("Best-fit equation: y = $a_fit_rnd * exp($b_fit_rnd * x)")

# Define the exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = exp_model.(x_values)

# Clamp values between 0 and 1
y_values = clamp.(y_values, 0.0, 1.0)

# Print rounded predictions
println("\nModel Predictions (rounded to 3 decimals):")
for (x, y) in zip(x_values, y_values)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3))")
end

Exact parameters (rounded to 2 decimals): a = 0.58, b = -0.18
Best-fit equation: y = 0.58 * exp(-0.18 * x)

Model Predictions (rounded to 3 decimals):
x =   1 → y = 0.484
x =   2 → y = 0.406
x =   3 → y = 0.341
x =   4 → y = 0.286
x =   5 → y = 0.24
x =   6 → y = 0.201
x =   7 → y = 0.169
x =   8 → y = 0.142
x =   9 → y = 0.119
x =  10 → y = 0.1
x =  11 → y = 0.084
x =  12 → y = 0.07
x =  13 → y = 0.059
x =  14 → y = 0.049
x =  15 → y = 0.041
x =  16 → y = 0.035
x =  17 → y = 0.029
x =  18 → y = 0.024
x =  19 → y = 0.02
x =  20 → y = 0.017


In [ ]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Round parameters for display
a_fit_rnd = round(a_fit, digits=2)
b_fit_rnd = round(b_fit, digits=2)

println("Exact parameters (rounded to 2 decimals): a = $a_fit_rnd, b = $b_fit_rnd")
println("Best-fit equation: y = $a_fit_rnd * exp($b_fit_rnd * x)")

# Define the exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_func = exp_model.(x_values)

# Clamp values between 0 and 1
y_values_func = clamp.(y_values, 0.0, 1.0)

# Convert to percentage
y_values_perc = y_values_func .* 100
y_data_perc = y_data .* 100

# Print rounded predictions
println("\nModel Predictions (rounded to 3 decimals):")
for (x, y) in zip(x_values, y_values_perc)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3))")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_func)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:60
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt2 = plot(
    x_values,
    y_values_pct,
    label = "Exponential Model Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Exponential Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 60),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefont = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,

)

scatter!(
    plt2,
    x_data,
    y_data_pct,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt2, "exponential_fit_crowdfunding.png")
println("Plot saved as exponential_fit_crowdfunding60.png")

Exact parameters (rounded to 2 decimals): a = 0.58, b = -0.18
Best-fit equation: y = 0.58 * exp(-0.18 * x)

Model Predictions (rounded to 3 decimals):
x =   1 → y = 48.446 %
x =   2 → y = 40.639 %
x =   3 → y = 34.09 %
x =   4 → y = 28.597 %
x =   5 → y = 23.988 %
x =   6 → y = 20.123 %
x =   7 → y = 16.88 %
x =   8 → y = 14.16 %
x =   9 → y = 11.878 %
x =  10 → y = 9.964 %
x =  11 → y = 8.358 %
x =  12 → y = 7.011 %
x =  13 → y = 5.882 %
x =  14 → y = 4.934 %
x =  15 → y = 4.139 %
x =  16 → y = 3.472 %
x =  17 → y = 2.912 %
x =  18 → y = 2.443 %
x =  19 → y = 2.049 %
x =  20 → y = 1.719 %

Sum of all predicted y-values from x = 1 to 20 (fraction): 2.917


LoadError: LoadError: UndefVarError: @L_str not defined
in expression starting at /home/ge48bax/MyThesisCode-main/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X24sdnNjb2RlLXJlbW90ZQ==.jl:53

In [65]:
using LsqFit
using Plots

# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount (in fraction)

# Solve for exact parameters a and b using two points
b_fit = log(y_data[3] / y_data[1]) / (x_data[3] - x_data[1])
a_fit = y_data[1] / exp(b_fit * x_data[1])

# Round parameters for display
a_fit_rnd = round(a_fit, digits=2)
b_fit_rnd = round(b_fit, digits=2)

println("Exact parameters (rounded to 2 decimals): a = $a_fit_rnd, b = $b_fit_rnd")
println("Best-fit equation: y = $a_fit_rnd * exp($b_fit_rnd * x)")

# Define the exponential function
exp_model(x) = a_fit * exp(b_fit * x)

# Compute model values for x = 1 to 20
x_values = 1:20
y_values_func = exp_model.(x_values)

# Clamp values between 0 and 1
y_values_func = clamp.(y_values_func, 0.0, 1.0)

# Convert to percentage
y_values_perc = y_values_func .* 100
y_data_perc = y_data .* 100

# Print rounded predictions
println("\nModel Predictions (rounded to 3 decimals):")
for (x, y) in zip(x_values, y_values_perc)
    println("x = $(lpad(x, 3)) → y = $(round(y, digits=3)) %")
end

# Calculate and print the sum of all y-values (fraction)
total_y = sum(y_values_func)
println("\nSum of all predicted y-values from x = 1 to 20 (fraction): $(round(total_y, digits=3))")

# Create tick labels:
ytick_positions = 0:1:100
ytick_labels = [mod(y,10)==0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Plot the fitted curve and original data
plt2 = plot(
    x_values,
    y_values_perc,
    label = "Exponential Model Curve",
    xlabel = "Remaining Working Days of Designated Period (Month)",
    ylabel = "Crowdfunded MobilityCoins of Users' Remaining Budget (%)",
    lw = 2,
    legend = :topright,
    title = "Exponential Fit to Individuals' Crowdfunding Willingness",
    xticks = 1:1:20,
    yticks = yticks,
    ylims = (0, 100),
    minorgrid = true,
    xflip = true,
    size = (750, 650),
    titlefontsize = 14,
    guidefontsize = 12,
    tickfontsize = 10,
    legendfontsize = 10,
    margin = 3Plots.mm,
)

scatter!(
    plt2,
    x_data,
    y_data_perc,
    label = "Surveyed Data",
    marker = (:circle, 8),
    color = :red,
)

# Save plot as PNG
savefig(plt2, "exponential_fit_crowdfunding100.png")
println("Plot saved as exponential_fit_crowdfunding.png")


Exact parameters (rounded to 2 decimals): a = 0.58, b = -0.18
Best-fit equation: y = 0.58 * exp(-0.18 * x)

Model Predictions (rounded to 3 decimals):
x =   1 → y = 48.446 %
x =   2 → y = 40.639 %
x =   3 → y = 34.09 %
x =   4 → y = 28.597 %
x =   5 → y = 23.988 %
x =   6 → y = 20.123 %
x =   7 → y = 16.88 %
x =   8 → y = 14.16 %
x =   9 → y = 11.878 %
x =  10 → y = 9.964 %
x =  11 → y = 8.358 %
x =  12 → y = 7.011 %
x =  13 → y = 5.882 %
x =  14 → y = 4.934 %
x =  15 → y = 4.139 %
x =  16 → y = 3.472 %
x =  17 → y = 2.912 %
x =  18 → y = 2.443 %
x =  19 → y = 2.049 %
x =  20 → y = 1.719 %

Sum of all predicted y-values from x = 1 to 20 (fraction): 2.917
Plot saved as exponential_fit_crowdfunding.png


In [24]:
# First Month - First Group_second equation
# Initial setup
initial_x = 20
z = 645860 * initial_x * 1.5 * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]

println("Iteration Results:")
println("x\t   y\t\t\t    t\t\t     r\t\t     updated_z")

for x in 30:-1:11
    y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
    t = y * 645860 * 0.5
    r = 645860 * 0.5 - (t)
    z -= (r + t)

    @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\n", x, y, t, r, z)

    # Store values
    push!(x_vals, x)
    push!(y_vals, y)
    push!(t_vals, t)
    push!(r_vals, r)
    push!(z_vals, z)
end

# Create a DataFrame from the results
results_df = DataFrame(x = x_vals, y = y_vals, t = t_vals, r = r_vals, z = z_vals)

Iteration Results:
x	   y			    t		     r		     updated_z
30	0.002966	957.78	321972.22	9364970.00
29	0.003536	1141.78	321788.22	9042040.00
28	0.004215	1361.11	321568.89	8719110.00
27	0.005025	1622.59	321307.41	8396180.00
26	0.005990	1934.29	320995.71	8073250.00
25	0.007140	2305.87	320624.13	7750320.00
24	0.008512	2748.84	320181.16	7427390.00
23	0.010147	3276.90	319653.10	7104460.00
22	0.012097	3906.40	319023.60	6781530.00
21	0.014421	4656.82	318273.18	6458600.00
20	0.017191	5551.41	317378.59	6135670.00
19	0.020493	6617.85	316312.15	5812740.00
18	0.024430	7889.16	315040.84	5489810.00
17	0.029123	9404.69	313525.31	5166880.00
16	0.034718	11211.35	311718.65	4843950.00
15	0.041387	13365.08	309564.92	4521020.00
14	0.049337	15932.55	306997.45	4198090.00
13	0.058815	18993.24	303936.76	3875160.00
12	0.070114	22641.89	300288.11	3552230.00
11	0.083583	26991.45	295938.55	3229300.00


,x,y,t,r,z
,Int64,Float64,Float64,Float64,Float64
1,30,0.00296592,957.783,3.21972e5,9.36497e6
2,29,0.00353568,1141.78,3.21788e5,9.04204e6
3,28,0.00421489,1361.11,3.21569e5,8.71911e6
4,27,0.00502458,1622.59,3.21307e5,8.39618e6
5,26,0.00598981,1934.29,3.20996e5,8.07325e6
6,25,0.00714047,2305.87,3.20624e5,7.75032e6
7,24,0.00851218,2748.84,3.20181e5,7.42739e6
8,23,0.0101474,3276.9,3.19653e5,7.10446e6
9,22,0.0120967,3906.4,3.19024e5,6.78153e6


In [15]:
# All Months - Second Group _Second equation
# Initial setup
initial_x = 20
z = 645860 * initial_x * 1 * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]

println("Iteration Results:")
println("x\t   y\t\t\t    t\t\t     r\t\t     updated_z")

for x in 20:-1:1
    y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
    t = y * 645860 * 0.5
    r = 645860 * 0.5 - (t)
    z -= (r + t)

    @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\n", x, y, t, r, z)

    # Store values
    push!(x_vals, x)
    push!(y_vals, y)
    push!(t_vals, t)
    push!(r_vals, r)
    push!(z_vals, z)
end

# Create a DataFrame from the results
results_df = DataFrame(x = x_vals, y = y_vals, t = t_vals, r = r_vals, z = z_vals)

Iteration Results:
x	   y			    t		     r		     updated_z
20	0.017191	5551.41	317378.59	6135670.00
19	0.020493	6617.85	316312.15	5812740.00
18	0.024430	7889.16	315040.84	5489810.00
17	0.029123	9404.69	313525.31	5166880.00
16	0.034718	11211.35	311718.65	4843950.00
15	0.041387	13365.08	309564.92	4521020.00
14	0.049337	15932.55	306997.45	4198090.00
13	0.058815	18993.24	303936.76	3875160.00
12	0.070114	22641.89	300288.11	3552230.00
11	0.083583	26991.45	295938.55	3229300.00
10	0.099639	32176.58	290753.42	2906370.00
 9	0.118781	38357.79	284572.21	2583440.00
 8	0.141599	45726.42	277203.58	2260510.00
 7	0.168800	54510.58	268419.42	1937580.00
 6	0.201227	64982.21	257947.79	1614650.00
 5	0.239883	77465.46	245464.54	1291720.00
 4	0.285965	92346.78	230583.22	968790.00
 3	0.340900	110086.84	212843.16	645860.00
 2	0.406388	131234.81	191695.19	322930.00
 1	0.484456	156445.36	166484.64	0.00


,x,y,t,r,z
,Int64,Float64,Float64,Float64,Float64
1,20,0.0171908,5551.41,3.17379e5,6.13567e6
2,19,0.0204932,6617.85,3.16312e5,5.81274e6
3,18,0.0244299,7889.16,3.15041e5,5.48981e6
4,17,0.029123,9404.69,3.13525e5,5.16688e6
5,16,0.0347176,11211.4,3.11719e5,4.84395e6
6,15,0.0413869,13365.1,3.09565e5,4.52102e6
7,14,0.0493375,15932.6,3.06997e5,4.19809e6
8,13,0.0588153,18993.2,3.03937e5,3.87516e6
9,12,0.0701139,22641.9,3.00288e5,3.55223e6


In [26]:
#Second Month to the last month- First  - Second equation
# Initial setup
initial_x = 20
z1 = 645860 * initial_x * 0.5 * 0.5
z2 = 645860 * initial_x * 1 * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]
j_vals = Int[]

println("Iteration Results:")
println("x\t  y\t\t      t\t\t     r\t\t     z\t\t     j")

    j = 21  # Total iterations

    # Phase 1 (j = 21 to 11): x goes from 10 to 1
    for x in 10:-1:1
        y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
        t = y * 645860 * 0.5
        r = 645860 * 0.5 - t
        z1 -= (r + t)
        z = z1
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

    # Phase 2 (j = 10 to 1): x goes from 20 to 11
    for x in 20:-1:11
        y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
        t = y * 645860 * 0.5
        r = 645860 * 0.5 - t
        z2 -= (r + t)
        z = z2
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

# Create a DataFrame from the results
results_df = DataFrame(
    x = x_vals,
    y = y_vals,
    t = t_vals,
    r = r_vals,
    z = z_vals,
    j = j_vals
)

Iteration Results:
x	  y		      t		     r		     z		     j
10	0.099639	32176.58	290753.42	2906370.00	20
 9	0.118781	38357.79	284572.21	2583440.00	19
 8	0.141599	45726.42	277203.58	2260510.00	18
 7	0.168800	54510.58	268419.42	1937580.00	17
 6	0.201227	64982.21	257947.79	1614650.00	16
 5	0.239883	77465.46	245464.54	1291720.00	15
 4	0.285965	92346.78	230583.22	968790.00	14
 3	0.340900	110086.84	212843.16	645860.00	13
 2	0.406388	131234.81	191695.19	322930.00	12
 1	0.484456	156445.36	166484.64	0.00	11
20	0.017191	5551.41	317378.59	6135670.00	10
19	0.020493	6617.85	316312.15	5812740.00	 9
18	0.024430	7889.16	315040.84	5489810.00	 8
17	0.029123	9404.69	313525.31	5166880.00	 7
16	0.034718	11211.35	311718.65	4843950.00	 6
15	0.041387	13365.08	309564.92	4521020.00	 5
14	0.049337	15932.55	306997.45	4198090.00	 4
13	0.058815	18993.24	303936.76	3875160.00	 3
12	0.070114	22641.89	300288.11	3552230.00	 2
11	0.083583	26991.45	295938.55	3229300.00	 1


,x,y,t,r,z,j
,Int64,Float64,Float64,Float64,Float64,Int64
1,10,0.0996395,32176.6,2.90753e5,2.90637e6,20
2,9,0.118781,38357.8,2.84572e5,2.58344e6,19
3,8,0.141599,45726.4,2.77204e5,2.26051e6,18
4,7,0.1688,54510.6,2.68419e5,1.93758e6,17
5,6,0.201227,64982.2,2.57948e5,1.61465e6,16
6,5,0.239883,77465.5,2.45465e5,1.29172e6,15
7,4,0.285965,92346.8,2.30583e5,968790.0,14
8,3,0.3409,1.10087e5,2.12843e5,645860.0,13
9,2,0.406388,1.31235e5,1.91695e5,322930.0,12


In [28]:
#last month- First Group
# Initial setup
initial_x = 20
z1 = 645860 * initial_x * 0.5 * 0.5
z2 = 645860 * initial_x * 0.5 * 0.5

# Create arrays to store iteration results
x_vals = Int[]
y_vals = Float64[]
t_vals = Float64[]
r_vals = Float64[]
z_vals = Float64[]
j_vals = Int[]

println("Iteration Results:")
println("x\t  y\t\t      t\t\t     r\t\t     z\t\t     j")

    j = 21  # Total iterations

    # Phase 1 (j = 21 to 11): x goes from 10 to 1
    for x in 10:-1:1
        y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
        t = y * 645860 * 0.5
        r = 645860 * 0.5 - t
        z1 -= (r + t)
        z = z1
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

    # Phase 2 (j = 10 to 1): x goes from 20 to 11
    for x in 10:-1:1
        y = 0.5775211856783368 * exp(-0.17571864924550018 * x)
        t = y * 645860 * 0.5
        r = 645860 * 0.5 - t
        z2 -= (r + t)
        z = z2
        j -= 1

        @printf("%2d\t%.6f\t%.2f\t%.2f\t%.2f\t%2d\n", x, y, t, r, z, j)

        push!(x_vals, x)
        push!(y_vals, y)
        push!(t_vals, t)
        push!(r_vals, r)
        push!(z_vals, z)
        push!(j_vals, j)
    end

# Create a DataFrame from the results
results_df = DataFrame(
    x = x_vals,
    y = y_vals,
    t = t_vals,
    r = r_vals,
    z = z_vals,
    j = j_vals
)

Iteration Results:
x	  y		      t		     r		     z		     j
10	0.099639	32176.58	290753.42	2906370.00	20
 9	0.118781	38357.79	284572.21	2583440.00	19
 8	0.141599	45726.42	277203.58	2260510.00	18
 7	0.168800	54510.58	268419.42	1937580.00	17
 6	0.201227	64982.21	257947.79	1614650.00	16
 5	0.239883	77465.46	245464.54	1291720.00	15
 4	0.285965	92346.78	230583.22	968790.00	14
 3	0.340900	110086.84	212843.16	645860.00	13
 2	0.406388	131234.81	191695.19	322930.00	12
 1	0.484456	156445.36	166484.64	0.00	11
10	0.099639	32176.58	290753.42	2906370.00	10
 9	0.118781	38357.79	284572.21	2583440.00	 9
 8	0.141599	45726.42	277203.58	2260510.00	 8
 7	0.168800	54510.58	268419.42	1937580.00	 7
 6	0.201227	64982.21	257947.79	1614650.00	 6
 5	0.239883	77465.46	245464.54	1291720.00	 5
 4	0.285965	92346.78	230583.22	968790.00	 4
 3	0.340900	110086.84	212843.16	645860.00	 3
 2	0.406388	131234.81	191695.19	322930.00	 2
 1	0.484456	156445.36	166484.64	0.00	 1


,x,y,t,r,z,j
,Int64,Float64,Float64,Float64,Float64,Int64
1,10,0.0996395,32176.6,2.90753e5,2.90637e6,20
2,9,0.118781,38357.8,2.84572e5,2.58344e6,19
3,8,0.141599,45726.4,2.77204e5,2.26051e6,18
4,7,0.1688,54510.6,2.68419e5,1.93758e6,17
5,6,0.201227,64982.2,2.57948e5,1.61465e6,16
6,5,0.239883,77465.5,2.45465e5,1.29172e6,15
7,4,0.285965,92346.8,2.30583e5,968790.0,14
8,3,0.3409,1.10087e5,2.12843e5,645860.0,13
9,2,0.406388,1.31235e5,1.91695e5,322930.0,12


In [9]:
x = 1 # x defines number of working day in a month

coinsallocated = 655040 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
final_result_row = ()

for x in 1:20 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 20,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end

    allocated_mocos += final_result_row.coins_allocated
    consuming_mocos += final_result_row.coins_needed   

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = final_result_row.coins_allocated

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("monthly_result.csv", monthly_result_row, ',')
    
end
println(monthly_result_row)

     0     9.487045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.593022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.298590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.164937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.277848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     6.994080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     6.994080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.203913e+02   

In [10]:
x = 20 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 20:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)

     0     9.487045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.593022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.298590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.164937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.277848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     6.994080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     6.994080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.203913e+02   

In [11]:
using CSV
using DataFrames

# Load the original CSV file
df = CSV.read("Monthly Results with Crowdfunding.csv", DataFrame)

# Get column names
cols = names(df)

# Select the first 5 columns and the last column
selected_cols = vcat(cols[1:5], cols[end])
filtered_df = df[:, selected_cols]

# Compute column 7: coins_consummed = coins_needed - crowdfunding
filtered_df.coins_consummed = filtered_df[!, 3] .- filtered_df[!, 6]

# Compute column 8: market_delta_wi_crowd = coins_allocated - coins_consummed
filtered_df.market_delta_wi_crowd = filtered_df[!, 3] .- filtered_df.coins_consummed

# Compute column 9: ratio_of_less_consumption = coins_consummed / coins_needed
filtered_df.ratio_of_less_consumption = filtered_df.coins_consummed ./ filtered_df[!, 4]

# Compute column 10: updated_mocomp = ratio_of_less_consumption * mocomp
filtered_df.updated_mocomp = filtered_df.ratio_of_less_consumption .* filtered_df[!, 2]

# Add new column: day_num = 1 to 20
filtered_df.day_num = 1:20

# Save to new CSV file
CSV.write("Filtered_Monthly_Results.csv", filtered_df)

"Filtered_Monthly_Results.csv"

In [12]:
using CSV
using DataFrames

# === Load both CSV files ===
final_df = CSV.read("Filtered_Monthly_Results.csv", DataFrame)
source_df = CSV.read("Monthly Results with Crowdfunding.csv", DataFrame)

# === Extract ratio column ===
ratio_col = final_df.ratio_of_less_consumption

# === Get columns 6 to end from the source file ===
cols_to_multiply = source_df[:, 6:end-1]  # Exclude the last column
col_names = names(cols_to_multiply)

# === Multiply each column by the ratio and collect results ===
scaled_cols = DataFrame()
for name in col_names
    scaled_cols[!, name] = cols_to_multiply[!, name] .* ratio_col
end

# === Append results to final_df starting from column 12 ===
final_df = hcat(final_df, scaled_cols)

# === Save final result ===
CSV.write("Filtered_Monthly_Results_With_Extras.csv", final_df)


"Filtered_Monthly_Results_With_Extras.csv"

In [13]:
using CSV
using DataFrames

# === Load the original data ===
df = CSV.read("Filtered_Monthly_Results_With_Extras.csv", DataFrame)

# === Construct the full repeated DataFrame ===
# First 21 rows: original data
# Rows 22+: original data repeated twice (so, 42 rows to append)
df_repeats = vcat(df, df)  # Two copies
df_combined = vcat(df, df_repeats)  # Original + 2 repetitions

# === Save the first part (rows 1–21) with headers ===
CSV.write("Filtered_Monthly_Results_With_Extras_Expanded.csv", df_combined[1:21, :])

# === Save rows 22 onward (2× repeats) without headers ===
CSV.write("temp_append.csv", df_combined[22:end, :], writeheader=false)

# === Append temp content to the main file ===
open("Filtered_Monthly_Results_With_Extras_Expanded.csv", "a") do io_target
    open("temp_append.csv", "r") do io_source
        for line in eachline(io_source)
            println(io_target, line)
        end
    end
end

# === Remove temp file
rm("temp_append.csv")

println("Data repeated 2 times and appended starting from row 22 without headers.")

Data repeated 2 times and appended starting from row 22 without headers.


In [14]:
using CSV
using DataFrames

# === Load the expanded file ===
df = CSV.read("Filtered_Monthly_Results_With_Extras_Expanded.csv", DataFrame)

# === Drop the 2nd and 9th columns ===
select!(df, Not([2, 5, 9]))

# === Rename the 10th column to "mocomp" ===
rename!(df, names(df)[7] => "mocomp")
rename!(df, names(df)[6] => "market_delta")

# === Bring the 8th column (currently at position 8) to the first column ===
col_names = names(df)
reordered_cols = vcat(col_names[8], col_names[1:7], col_names[9:end])
df = df[:, reordered_cols]

# === Save the updated DataFrame back to file ===
CSV.write("Final_Filtered_Monthly_Results_Expanded.csv", df)

println("Columns updated: 2nd and 9th removed, 10th renamed to 'mocomp', and 8th column moved to first position.")

Columns updated: 2nd and 9th removed, 10th renamed to 'mocomp', and 8th column moved to first position.


In [15]:
using CSV
using DataFrames

# === Load the file ===
df = CSV.read("Final_Filtered_Monthly_Results_Expanded.csv", DataFrame)

# === Get column names except the first one ===
cols_to_modify = names(df)[3:end]

# === Divide selected columns by 2, skipping the first row ===
foreach(col -> df[1:end, col] .= df[1:end, col] ./ 2, cols_to_modify)

# === Save the updated DataFrame ===
CSV.write("Second_Group_3Months_Results.csv", df)

println("All columns except the first one have been divided by 2, excluding the first row.")

All columns except the first one have been divided by 2, excluding the first row.


In [16]:
x = 10 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 10:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("first_one_monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("First one Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)

     0     9.487045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.593022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.298590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.164937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.277848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     6.994080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     6.994080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.203913e+02   

In [8]:
using CSV
using DataFrames

# === Load the file ===
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# === Separate the first 10 rows ===
second_10_rows = df[11:20, :]

# === Keep the rest of the data (from row 11 onward) ===
remaining_rows = df[11:end, :]

# === Append the first 10 rows to the end ===
new_df = vcat(remaining_rows, second_10_rows)

# === Update first column values for first and last 10 rows ===
new_df[1:20, 1] .= 1:20 
new_df[21:40, 1] .= 1:20                # First 20 rows
new_df[41:60, 1] .= 1:20                # Last 20 rows

# === Save the updated DataFrame ===
CSV.write("First_one_Group_3Months_Results.csv", new_df)

println("First 10 rows moved to the end, and first/last 10 rows of column 1 updated to 1–10.")

First 10 rows moved to the end, and first/last 10 rows of column 1 updated to 1–10.


In [18]:
x = 30 # x defines number of working day in a month

coinsallocated = 655040 #670250 #655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

crowdfunding = 0
totalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

totalcardem = 0
totalbusdem = 0
totalbikedem = 0
totaldem = 0

monthly_mocperkgCO2 = 0
last_mocomp = 0
monthly_coins_allocated = 0
monthly_coins_needed = 0
monthly_market_delta = 0
last_cardem = 0
last_busdem = 0
last_bikedem = 0
last_flows_car = 0
last_flows_bus = 0
last_flows_bike = 0
monthly_mocos_car = 0
monthly_mocos_bus = 0
monthly_mocos_bike = 0
monthly_emissions_car = 0
monthly_emissions_bus = 0
monthly_total_tt = 0
monthly_total_tt_c = 0 
monthly_total_tt_b = 0
monthly_total_tt_r = 0
monthly_total_dist_car = 0
monthly_total_dist_bus = 0
monthly_total_dist_rad = 0

mocperkg = [1.0]
        # 2.5,
        # 5.0,
        # 7.5]

monthly_result_row = ()
day_result_row = ()
final_result_row = ()
daily_results_df = DataFrame()
monthly_result_df= DataFrame()

for x in 30:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        day_result_row = ( 
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index]
        )
    end

    allocated_mocos += day_result_row.coins_allocated
    
    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69))) # 20 working days in a month
    crowdfunding = exp_model(x) * (655040)
    #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
    totalcrowdfunding += crowdfunding

    coinsallocated = day_result_row.coins_allocated #- crowdfunding

    for mocperkgCO2 in mocperkg
        
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = true
                                )

        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 

        # Save the row of minimum delta in a separate file
        final_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end
    # Merge crowdfunding into your daily row
    daily_row_with_crowdfunding = merge(final_result_row, (crowdfunding = crowdfunding,))
    daily_results_df = DataFrame([daily_row_with_crowdfunding])
    push!(monthly_result_df, daily_results_df[1, :])

    consuming_mocos += final_result_row.coins_needed

    available_allocated_mocos = allocated_mocos #+ crowdfunding
    available_consuming_mocos = consuming_mocos #+ crowdfunding

    nextcardem = final_result_row.cardem
    nextbusdem = final_result_row.busdem
    nextbikedem = final_result_row.bikedem
    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += (final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem)

    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    last_mocomp = final_result_row.mocomp
    available_monthly_coins_allocated = available_allocated_mocos
    available_monthly_coins_needed = available_consuming_mocos 
    monthly_market_delta += final_result_row.market_delta
    last_cardem = final_result_row.cardem 
    last_busdem = final_result_row.busdem
    last_bikedem = final_result_row.bikedem
    last_flows_car = final_result_row.flows_car
    last_flows_bus = final_result_row.flows_bus
    last_flows_bike = final_result_row.flows_bike
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c 
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    first_available_allocated_mocos = available_allocated_mocos
    first_available_consuming_mocos = available_consuming_mocos
    first_crowdfunding = crowdfunding
    monthly_crowdfunding = totalcrowdfunding

    monthly_result_row = ( 
        monthly_mocperkgCO2 = monthly_mocperkgCO2, last_mocomp, available_monthly_coins_allocated, available_monthly_coins_needed, monthly_market_delta,
        last_cardem, last_busdem, last_bikedem, 
        last_flows_car, last_flows_bus, last_flows_bike, 
        monthly_mocos_car, monthly_mocos_bus, monthly_mocos_bike, monthly_emissions_car, monthly_emissions_bus, 
        monthly_total_tt, monthly_total_tt_c, monthly_total_tt_b, monthly_total_tt_r, 
        monthly_total_dist_car, monthly_total_dist_bus, monthly_total_dist_rad,
        totalcardem, totalbusdem, totalbikedem, totaldem, first_crowdfunding, monthly_crowdfunding,
        first_available_allocated_mocos, first_available_consuming_mocos
    )

    coinsallocated = 655040 #654860 #670250

    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp

    # Write header and data to CSV
    writedlm("first_two_monthly_result.csv", monthly_result_row, ',')
    
end
CSV.write("First two Monthly Results with Crowdfunding.csv", monthly_result_df)
println(monthly_result_row)

     0     9.487045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.593022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.298590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.164937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.277848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     6.994080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     6.994080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.203913e+02   

In [19]:
using CSV
using DataFrames

# Load the original CSV file
df = CSV.read("First two Monthly Results with Crowdfunding.csv", DataFrame)

# Get column names
cols = names(df)

# Select the first 5 columns and the last column
selected_cols = vcat(cols[1:5], cols[end])
filtered_df = df[:, selected_cols]

# Compute column 7: coins_consummed = coins_needed - crowdfunding
filtered_df.coins_consummed = filtered_df[!, 3] .- filtered_df[!, 6]

# Compute column 8: market_delta_wi_crowd = coins_allocated - coins_consummed
filtered_df.market_delta_wi_crowd = filtered_df[!, 3] .- filtered_df.coins_consummed

# Compute column 9: ratio_of_less_consumption = coins_consummed / coins_needed
filtered_df.ratio_of_less_consumption = filtered_df.coins_consummed ./ filtered_df[!, 4]

# Compute column 10: updated_mocomp = ratio_of_less_consumption * mocomp
filtered_df.updated_mocomp = filtered_df.ratio_of_less_consumption .* filtered_df[!, 2]

# Add new column: day_num = 1 to 20 and 1 to 10
filtered_df.day_num = vcat(1:20, 1:10)

# Save to new CSV file
CSV.write("First_Filtered_Monthly_Results.csv", filtered_df)

"First_Filtered_Monthly_Results.csv"

In [20]:
using CSV
using DataFrames

# === Load both CSV files ===
final_df = CSV.read("First_Filtered_Monthly_Results.csv", DataFrame)
source_df = CSV.read("First two Monthly Results with Crowdfunding.csv", DataFrame)

# === Extract ratio column ===
ratio_col = final_df.ratio_of_less_consumption

# === Get columns 6 to end from the source file ===
cols_to_multiply = source_df[:, 6:end-1]  # Exclude the last column
col_names = names(cols_to_multiply)

# === Multiply each column by the ratio and collect results ===
scaled_cols = DataFrame()
for name in col_names
    scaled_cols[!, name] = cols_to_multiply[!, name] .* ratio_col
end

# === Append results to final_df starting from column 12 ===
final_df = hcat(final_df, scaled_cols)

# === Save final result ===
CSV.write("First_Filtered_Monthly_Results_With_Extras.csv", final_df)

"First_Filtered_Monthly_Results_With_Extras.csv"

In [21]:
using CSV
using DataFrames

# === Load the file ===
df = CSV.read("First_Filtered_Monthly_Results_With_Extras.csv", DataFrame)

# === Separate the first 10 rows ===
second_20_rows = df[11:30, :]
second_10_rows = df[21:30, :]
# === Keep the rest of the data (from row 11 onward) ===
remaining_rows = df[1:end, :]

# === Append the first 10 rows to the end ===
first_df = vcat(remaining_rows, second_20_rows)
new_df = vcat(first_df, second_10_rows)
# === Update first column values for first and last 10 rows ===
new_df[end-9:end, 11] .= 11:20                # Last 10 rows

# === Save the updated DataFrame ===
CSV.write("First_Filtered_Monthly_Results_With_Extras_Expanded.csv", new_df)

println("First 10 rows moved to the end, and first/last 10 rows of column 1 updated to 1–10.")

First 10 rows moved to the end, and first/last 10 rows of column 1 updated to 1–10.


In [22]:
using CSV
using DataFrames

# === Load the expanded file ===
df = CSV.read("First_Filtered_Monthly_Results_With_Extras_Expanded.csv", DataFrame)

# === Drop the 2nd and 9th columns ===
select!(df, Not([2, 5, 9]))

# === Rename the 10th column to "mocomp" ===
rename!(df, names(df)[7] => "mocomp")
rename!(df, names(df)[6] => "market_delta")

# === Bring the 8th column (currently at position 8) to the first column ===
col_names = names(df)
reordered_cols = vcat(col_names[8], col_names[1:7], col_names[9:end])
df = df[:, reordered_cols]

# === Save the updated DataFrame back to file ===
CSV.write("Final_First_Filtered_Monthly_Results_With_Extras_Expanded.csv", df)

println("Columns updated: 2nd and 9th removed, 10th renamed to 'mocomp', and 8th column moved to first position.")

Columns updated: 2nd and 9th removed, 10th renamed to 'mocomp', and 8th column moved to first position.


In [ ]:
using CSV
using DataFrames

# === Load the file ===
df = CSV.read("Final_First_Filtered_Monthly_Results_With_Extras_Expanded.csv", DataFrame)

# === Get column names except the first one ===
cols_to_modify = names(df)[3:end]

# === Divide selected columns by 2, skipping the first row ===
foreach(col -> df[1:end, col] .= df[1:end, col] ./ 2, cols_to_modify)

# === Save the updated DataFrame ===
CSV.write("First_Two_Group_3Months_Results.csv", df)

println("All columns except the first one have been divided by 2, excluding the first row.")

All columns except the first one have been divided by 2, excluding the first row.


In [109]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
allocated = df[!, Symbol("coins_allocated")]
needed = df[!, Symbol("coins_needed")]
consumed = df[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:340000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day in Market",
    lw = 2,
    legend = :bottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 340000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed,
    label = "Consumed Coins",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed,
    label = "Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 345000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 345000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 345000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "32months_allocated_consumed_needed.png")
println("Plot saved as 32months_allocated_consumed_needed.png")

Plot saved as 32months_allocated_consumed_needed.png


In [114]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
crowdfunding = df[!, Symbol("crowdfunding")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 0:10000:180000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    crowdfunding,
    label = "Crowdfunding",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of Crowdfunded MobiltyCoins",
    lw = 2,
    legend = :top,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (0, 180000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line

scatter!(
    plt,
    days,
    crowdfunding,
    label = "Crowdfunded Coins Data",
    marker = (:circle, 5),
    color = :red,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [0, 180000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 185000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 185000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 185000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "32months_crowdfunding.png")
println("Plot saved as 32months_crowdfunding.png")

Plot saved as 32months_crowdfunding.png


In [139]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
cardem = df[!, Symbol("cardem")]
busdem = df[!, Symbol("busdem")]
bikedem = df[!, Symbol("bikedem")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 50000:10000:280000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    cardem,
    label = "Car Demand",
    xlabel = "3 Months' Working Days",
    ylabel = "Different Type of Transport Mode Demand (Car, PT, Bike)",
    lw = 2,
    legend = :bottomleft,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (50000, 280000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    busdem,
    label = "PT Demand",
    lw = 2,
    color = :blue,
    #linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    bikedem,
    label = "Bike Demand",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    cardem,
    label = "Car Demand per Day",
    marker = (:circle, 5),
    color = :red,
)

scatter!(
    plt,
    days,
    busdem,
    label = "PT Demand per Day",
    marker = (:diamond, 5),
    color = :blue,
)

scatter!(
    plt,
    days,
    bikedem,
    label = "Bike Demand per Day",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [50000, 280000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 285000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 285000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 285000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "32months_demand.png")
println("Plot saved as 32months_demand.png")

Plot saved as 32months_demand.png


In [129]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
mocomp = df[!, Symbol("mocomp")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 2:0.1:5
ytick_labels = [mod(y,0.5) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    mocomp,
    label = "MobiltyCoin Market Price",
    xlabel = "3 Months' Working Days",
    ylabel = "MobiltyCoin Market Price (Coins)",
    lw = 2,
    legend = :bottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (2, 5),
    minorgrid = true,
    size = (2000, 750),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :red,
)

# Add PT Demand as a second line

scatter!(
    plt,
    days,
    mocomp,
    label = "MobiltyCoin Market Price Data",
    marker = (:circle, 5),
    color = :blue,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [2, 5], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 5.1, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 5.1, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 5.1, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "32months_mocomp.png")
println("Plot saved as 32months_mocomp.png")

Plot saved as 32months_mocomp.png


In [115]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
allocated = df[!, Symbol("coins_allocated")]
needed = df[!, Symbol("coins_needed")]
consumed = df[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:340000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day in Market",
    lw = 2,
    legend = :outerbottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 340000),
    minorgrid = true,
    size = (1950, 1100),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 10Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed,
    label = "Consumed Coins",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed,
    label = "Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 345000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 345000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 345000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "311months_allocated_consumed_needed.png")
println("Plot saved as 311months_allocated_consumed_needed.png")

Plot saved as 311months_allocated_consumed_needed.png


In [116]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("First_Two_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
allocated = df[!, Symbol("coins_allocated")]
needed = df[!, Symbol("coins_needed")]
consumed = df[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:340000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day in Market",
    lw = 2,
    legend = :bottomleft,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 340000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed,
    label = "Consumed Coins",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed,
    label = "Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 345000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 345000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 345000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "312months_allocated_consumed_needed.png")
println("Plot saved as 312months_allocated_consumed_needed.png")

Plot saved as 312months_allocated_consumed_needed.png


In [117]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df1 = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df2 = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df1[!, Symbol("day_num")]
allocated = df1[!, Symbol("coins_allocated")]
needed = df1[!, Symbol("coins_needed")]
consumed1 = df1[!, Symbol("coins_consummed")]
consumed2 = df2[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:340000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day for Two Groups",
    lw = 2,
    legend = :outerbottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 340000),
    minorgrid = true,
    size = (2000, 1150),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 10Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed1,
    label = "1st Group Consumed Coins",
    lw = 2,
    color = :orange,
    #linestyle = :dot,
)

plot!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins",
    lw = 2,
    color = :green,
    linestyle = :dot,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed1,
    label = "1st Group Consumed Coins Data",
    marker = (:rect, 6),
    color = :orange,
)

scatter!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 345000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 345000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 345000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "321months_allocated_consumed_needed.png")
println("Plot saved as 321months_allocated_consumed_needed.png")

Plot saved as 321months_allocated_consumed_needed.png


In [ ]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
crowdfunding = df[!, Symbol("crowdfunding")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 0:10000:180000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    crowdfunding,
    label = "Crowdfunding",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of Crowdfunded MobiltyCoins",
    lw = 2,
    legend = :top,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (0, 180000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line

scatter!(
    plt,
    days,
    crowdfunding,
    label = "Crowdfunded Coins Data",
    marker = (:circle, 5),
    color = :red,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [0, 180000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 185000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 185000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 185000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "32months_crowdfunding.png")
println("Plot saved as 32months_crowdfunding.png")

In [118]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df1 = CSV.read("First_Two_Group_3Months_Results.csv", DataFrame)
df2 = CSV.read("Second_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df1[!, Symbol("day_num")]
allocated = df1[!, Symbol("coins_allocated")]
needed = df1[!, Symbol("coins_needed")]
consumed1 = df1[!, Symbol("coins_consummed")]
consumed2 = df2[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:340000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day for Two Groups",
    lw = 2,
    legend = :outerbottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 340000),
    minorgrid = true,
    size = (2000, 1150),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 10Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed1,
    label = "1st Group Consumed Coins",
    lw = 2,
    color = :brown,
    #linestyle = :dot,
)

plot!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins",
    lw = 2,
    color = :green,
    linestyle = :dot,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed1,
    label = "1st Group Consumed Coins Data",
    marker = (:pentagon, 6),
    color = :brown,
)

scatter!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 345000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 345000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 345000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "322months_allocated_consumed_needed.png")
println("Plot saved as 322months_allocated_consumed_needed.png")

Plot saved as 322months_allocated_consumed_needed.png


In [119]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
df1 = CSV.read("First_Two_Group_3Months_Results.csv", DataFrame)
df2 = CSV.read("Second_Group_3Months_Results.csv", DataFrame)
df3 = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df1[!, Symbol("day_num")]
allocated = df1[!, Symbol("coins_allocated")]
needed = df1[!, Symbol("coins_needed")]
consumed1 = df1[!, Symbol("coins_consummed")]
consumed2 = df2[!, Symbol("coins_consummed")]
consumed3 = df3[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 140000:10000:330000
ytick_labels = [mod(y, 10000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    consumed3,
    label = "1st Group (1st Allocation Type)\nConsumed Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day\nfor Two Groups and Two Different Allocation Types)",
    lw = 2,
    legend = :outerbottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (140000, 330000),
    minorgrid = true,
    size = (2000, 1100),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 10Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :orange,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed1,
    label = "1st Group (2nd Allocation Type)\nConsumed Coins",
    lw = 2,
    color = :brown,
    linestyle = :dash,
)

plot!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins",
    lw = 2,
    color = :green,
    linestyle = :dot,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    consumed3,
    label = "1st Group (1st Allocation Type)\nConsumed Coins Data",
    marker = (:rect, 6),
    color = :orange,
)

scatter!(
    plt,
    days,
    consumed1,
    label = "1st Group (2nd Allocation Type)\nConsumed Coins Data",
    marker = (:pentagon, 5),
    color = :brown,
)

scatter!(
    plt,
    days,
    consumed2,
    label = "2nd Group Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [140000, 340000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 335000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 335000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 335000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "3212months_allocated_consumed_needed.png")
println("Plot saved as 3212months_allocated_consumed_needed.png")

Plot saved as 3212months_allocated_consumed_needed.png


In [121]:
using CSV
using DataFrames
using Plots

# ==========================
# READ YOUR CSV FILE
# ==========================
# Adjust the path to your CSV file
#df = CSV.read("First_one_Group_3Months_Results.csv", DataFrame)
df = CSV.read("Final_Filtered_Monthly_Results_Expanded.csv", DataFrame)

# ==========================
# EXTRACT COLUMNS
# ==========================
days = 1:60 
x_values = df[!, Symbol("day_num")]
allocated = df[!, Symbol("coins_allocated")]
needed = df[!, Symbol("coins_needed")]
consumed = df[!, Symbol("coins_consummed")]

# ==========================
# PLOT
# ==========================

# Prepare y-tick labels (optional)
ytick_positions = 280000:10000:660000
ytick_labels = [mod(y, 20000) == 0 ? string(y) : "" for y in ytick_positions]
yticks = (ytick_positions, ytick_labels)

# Prepare x-tick labels (optional)
xtick_positions = 1:60  # These are the positions on the x-axis
xtick_labels = x_values # These are your repeated 1–20 values from the DataFrame
xticks = (xtick_positions, string.(xtick_labels))


# Create the plot
plt = plot(
    days,
    allocated,
    label = "Allocated Coins",
    xlabel = "3 Months' Working Days",
    ylabel = "Number of MobiltyCoins Allocated and Consumed per Day in Market",
    lw = 2,
    legend = :bottom,
    legendorientation = :horizontal,
    title = "",
    xticks = xticks,
    yticks = yticks,
    xlims = (0.5, 60.5),
    ylims = (280000, 660000),
    minorgrid = true,
    size = (2000, 850),
    titlefont = 16,
    guidefontsize = 16,
    tickfontsize = 12,
    legendfontsize = 12,
    margin = 12Plots.mm,
    grid = true,
    framestyle = :box,
    fontfamily = "Computer Modern",
    background_color = :white,
    foreground_color_subplot = :black,
    color = :blue,
)

# Add PT Demand as a second line
plot!(
    plt,
    days,
    needed,
    label = "Needed Coins",
    lw = 2,
    color = :red,
    linestyle = :dash,
)

# Add bike Demand as a second line
plot!(
    plt,
    days,
    consumed,
    label = "Consumed Coins",
    lw = 2,
    color = :green,
)

# Optionally add scatter markers
scatter!(
    plt,
    days,
    allocated,
    label = "Allocated Coins Data",
    marker = (:circle, 6),
    color = :blue,
)

scatter!(
    plt,
    days,
    needed,
    label = "Needed Coins Data",
    marker = (:diamond, 4),
    color = :red,
)

scatter!(
    plt,
    days,
    consumed,
    label = "Consumed Coins Data",
    marker = (:utriangle, 5),
    color = :forestgreen,
)

# Add vertical lines to separate months
for x in [20.5, 40.5]
    plot!([x, x], [280000, 660000], lc = :gray, ls = :dash, lw = 1.5, label = "")
end

# Add annotations for months
annotate!([
    (10, 670000, text("Month 1", :black, "Computer Modern", 16, :center)),
    (30, 670000, text("Month 2", :black, "Computer Modern", 16, :center)),
    (50, 670000, text("Month 3", :black, "Computer Modern", 16, :center))
])


# Save the plot
savefig(plt, "3months_allocated_consumed_needed.png")
println("Plot saved as 3months_allocated_consumed_needed.png")

Plot saved as 3months_allocated_consumed_needed.png


In [9]:
using DataFrames
using CSV
using DelimitedFiles
using Plots

# Initialize storage for daily data
daily_results = DataFrame(
    Day = Int[],
    AllocatedMoCos = Float64[],
    ConsumedMoCos = Float64[],
    Crowdfunding = Float64[],
    TotalDemand = Float64[],
    EmissionsCar = Float64[],
    EmissionsBus = Float64[],
    EmissionsTotal = Float64[],
    MarketDelta = Float64[],
)

# reset monthly aggregates
totalcrowdfunding = 0.0
allocated_mocos = 0.0
consuming_mocos = 0.0

# Variables to accumulate monthly sums
monthly_mocperkgCO2 = 0.0
monthly_market_delta = 0.0
monthly_mocos_car = 0.0
monthly_mocos_bus = 0.0
monthly_mocos_bike = 0.0
monthly_emissions_car = 0.0
monthly_emissions_bus = 0.0
monthly_total_tt = 0.0
monthly_total_tt_c = 0.0
monthly_total_tt_b = 0.0
monthly_total_tt_r = 0.0
monthly_total_dist_car = 0.0
monthly_total_dist_bus = 0.0
monthly_total_dist_rad = 0.0
totalcardem = 0.0
totalbusdem = 0.0
totalbikedem = 0.0
totaldem = 0.0
first_crowdfunding = 0.0
monthly_crowdfunding = 0.0
first_available_allocated_mocos = 0.0
first_available_consuming_mocos = 0.0

coinsallocated = 645860
initialMP = 1
mocperkg = [1.0]

for x in 1:20
    for mocperkgCO2 in mocperkg

        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)

        result = optimize(local_objective, 0.0, 10.0, Brent();
                                iterations = 15,
                                store_trace = true,
                                show_trace = false
                                )
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)

        min_delta_index = findfirst(==(optimized_value), deltaarr)

        final_result_row = (
            mocperkgCO2 = mocperkgCO2arr[min_delta_index],
            mocomp = MParr[min_delta_index],
            coins_allocated = coinsallocatedarr[min_delta_index],
            coins_needed = coinsneededarr[min_delta_index],
            market_delta = deltaarr[min_delta_index],
            cardem = cardemarr[min_delta_index],
            busdem = busdemarr[min_delta_index],
            bikedem = bikedemarr[min_delta_index],
            flows_car = flowscararr[min_delta_index],
            flows_bus = flowsbusarr[min_delta_index],
            flows_bike = flowsbikearr[min_delta_index],
            mocos_car = mocoscararr[min_delta_index],
            mocos_bus = mocosbusarr[min_delta_index],
            mocos_bike = mocosbikearr[min_delta_index],
            emissions_car = emissionscararr[min_delta_index],
            emissions_bus = emissionsbusarr[min_delta_index],
            total_tt = totaltt[min_delta_index],
            total_tt_c = totalttc[min_delta_index],
            total_tt_b = totalttb[min_delta_index],
            total_tt_r = totalttr[min_delta_index],
            total_dist_car = totaldcarr[min_delta_index],
            total_dist_bus = totaldbarr[min_delta_index],
            total_dist_rad = totaldrarr[min_delta_index]
        )
    end

    allocated_mocos += final_result_row.coins_allocated
    consuming_mocos += final_result_row.coins_needed

    exp_model(x) = 897.27 / (1 + exp(0.22 * (x + 32.69)))
    crowdfunding = exp_model(x) * allocated_mocos
    totalcrowdfunding += crowdfunding

    nextdem = final_result_row.cardem + final_result_row.busdem + final_result_row.bikedem

    # Aggregate for monthly sums
    monthly_mocperkgCO2 += final_result_row.mocperkgCO2
    monthly_market_delta += final_result_row.market_delta
    monthly_mocos_car += final_result_row.mocos_car
    monthly_mocos_bus += final_result_row.mocos_bus
    monthly_mocos_bike += final_result_row.mocos_bike
    monthly_emissions_car += final_result_row.emissions_car
    monthly_emissions_bus += final_result_row.emissions_bus
    monthly_total_tt += final_result_row.total_tt
    monthly_total_tt_c += final_result_row.total_tt_c
    monthly_total_tt_b += final_result_row.total_tt_b
    monthly_total_tt_r += final_result_row.total_tt_r
    monthly_total_dist_car += final_result_row.total_dist_car
    monthly_total_dist_bus += final_result_row.total_dist_bus
    monthly_total_dist_rad += final_result_row.total_dist_rad
    totalcardem += final_result_row.cardem
    totalbusdem += final_result_row.busdem
    totalbikedem += final_result_row.bikedem
    totaldem += nextdem

    if x == 1
        first_available_allocated_mocos = allocated_mocos
        first_available_consuming_mocos = consuming_mocos
        first_crowdfunding = crowdfunding
    end

    monthly_crowdfunding = totalcrowdfunding

    emissions_total = final_result_row.emissions_car + final_result_row.emissions_bus

    push!(daily_results, (
        Day = x,
        AllocatedMoCos = round(allocated_mocos, digits=2),
        ConsumedMoCos = round(consuming_mocos, digits=2),
        Crowdfunding = round(crowdfunding, digits=2),
        TotalDemand = round(nextdem, digits=2),
        EmissionsCar = round(final_result_row.emissions_car, digits=2),
        EmissionsBus = round(final_result_row.emissions_bus, digits=2),
        EmissionsTotal = round(emissions_total, digits=2),
        MarketDelta = round(final_result_row.market_delta, digits=2),
    ))

    coinsallocated = final_result_row.coins_allocated
    mocperkgCO2 = final_result_row.mocperkgCO2
    initialMP = final_result_row.mocomp
end

# Save daily CSV for plotting
CSV.write("daily_results.csv", daily_results)

# Save the monthly result as a LaTeX table
monthly_result_row = (
    mocperkgCO2 = monthly_mocperkgCO2,
    last_mocomp = final_result_row.mocomp,
    available_monthly_coins_allocated = allocated_mocos,
    available_monthly_coins_needed = consuming_mocos,
    monthly_market_delta = monthly_market_delta,
    last_cardem = final_result_row.cardem,
    last_busdem = final_result_row.busdem,
    last_bikedem = final_result_row.bikedem,
    last_flows_car = final_result_row.flows_car,
    last_flows_bus = final_result_row.flows_bus,
    last_flows_bike = final_result_row.flows_bike,
    monthly_mocos_car = monthly_mocos_car,
    monthly_mocos_bus = monthly_mocos_bus,
    monthly_mocos_bike = monthly_mocos_bike,
    monthly_emissions_car = monthly_emissions_car,
    monthly_emissions_bus = monthly_emissions_bus,
    monthly_total_tt = monthly_total_tt,
    monthly_total_tt_c = monthly_total_tt_c,
    monthly_total_tt_b = monthly_total_tt_b,
    monthly_total_tt_r = monthly_total_tt_r,
    monthly_total_dist_car = monthly_total_dist_car,
    monthly_total_dist_bus = monthly_total_dist_bus,
    monthly_total_dist_rad = monthly_total_dist_rad,
    totalcardem = totalcardem,
    totalbusdem = totalbusdem,
    totalbikedem = totalbikedem,
    totaldem = totaldem,
    first_crowdfunding = first_crowdfunding,
    monthly_crowdfunding = monthly_crowdfunding,
    first_available_allocated_mocos = first_available_allocated_mocos,
    first_available_consuming_mocos = first_available_consuming_mocos
)

# Export LaTeX table with ALL variables
open("monthly_result_table.tex", "w") do io
    println(io, "\\begin{tabular}{|l|r|}")
    println(io, "\\hline")
    println(io, "\\textbf{Variable} & \\textbf{Value} \\\\ \\hline")
    for (k, v) in pairs(monthly_result_row)
        println(io, "\$\\texttt{$(k)}\$ & \$$(round(v, digits=2))\$ \\\\ \\hline")
    end
    println(io, "\\end{tabular}")
end

println("Monthly LaTeX table written to monthly_result_table.tex.")

# --- Plots ---

plot(
    daily_results.Day,
    [daily_results.AllocatedMoCos daily_results.ConsumedMoCos],
    labels = ["Allocated MoCos" "Consumed MoCos"],
    xlabel = "Day",
    ylabel = "MoCos",
    title = "Allocated vs Consumed MoCos Over Time",
    lw = 2,
)
savefig("plot_allocated_consumed.pdf")

plot(
    daily_results.Day,
    daily_results.Crowdfunding,
    label = "Crowdfunding",
    xlabel = "Day",
    ylabel = "Crowdfunding (€)",
    title = "Crowdfunding Over Time",
    lw = 2,
)
savefig("plot_crowdfunding.pdf")

plot(
    daily_results.Day,
    daily_results.TotalDemand,
    label = "Total Demand",
    xlabel = "Day",
    ylabel = "Passengers",
    title = "Total Demand Over Time",
    lw = 2,
)
savefig("plot_total_demand.pdf")

plot(
    daily_results.Day,
    daily_results.EmissionsTotal,
    label = "Total Emissions (kgCO₂)",
    xlabel = "Day",
    ylabel = "Emissions (kg CO₂)",
    title = "Daily Total Emissions",
    lw = 2,
)
savefig("plot_total_emissions.pdf")

plot(
    daily_results.Day,
    daily_results.MarketDelta,
    label = "Market Delta",
    xlabel = "Day",
    ylabel = "Delta",
    title = "Market Delta Over Time",
    lw = 2,
)
savefig("plot_market_delta.pdf")

println("Plots saved.")

Monthly LaTeX table written to monthly_result_table.tex.
Plots saved.


In [10]:
# 2 Groups of allocated mobility coins at the beginning of the month
# first group received mobility coins which is necessary for a half month travelling which is half of the population
firstgroupmocos = monthly_result_row.first_available_allocated_mocos * 0.5
firstmonthlygroupmocos = monthly_result_row.first_available_allocated_mocos * 1
firstgroupmocoslastmonth = monthly_result_row.first_available_allocated_mocos * 0.5
secondgroupmocos = monthly_result_row.first_available_allocated_mocos * 1

InitialTotalCoinsforBothGroups = firstgroupmocos + secondgroupmocos

#firstgroupparticipants = 0.5 * monthly_result_row.totaldem
#secondgroupparticipants = 0.5 * monthly_result_row.totaldem

x = 60 # number of working days in 3 months (from 2025 to 2030)
m1 = 10 # number of working days in first and last allocation of first group
m2 = 20 # number of working days in second allocation of first group and all allocation of second group

firstcoinsallocated = firstgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6
secondcoinsallocated = secondgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6

dailycoinsallocated = 655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6

first_allocated_mocos = 0
first_consuming_mocos = 0

allocated_mocos = 0
consuming_mocos = 0

firstgroupcrowdfunding = 0
secondgroupcrowdfunding = 0

firstgrouptotalcrowdfunding = 0
secondgrouptotalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

firsttotalcardem = 0
firsttotalbusdem = 0
firsttotalbikedem = 0
firsttotaldem = 0

secondtotalcardem = 0
secondtotalbusdem = 0
secondtotalbikedem = 0
secondtotaldem = 0

firstlasttotalcardem = 0
firstlasttotalbusdem = 0
firstlasttotalbikedem = 0
firstlasttotaldem = 0

secondlasttotalcardem = 0
secondlasttotalbusdem = 0
secondlasttotalbikedem = 0
secondlasttotaldem = 0

firstmonthly_mocperkgCO2 = 0
firstlast_mocomp = 0   
firstmonthly_mocperkgCO2 = 0 
firstlast_mocomp = 0 
firstmonthly_coins_allocated = 0 
firstmonthly_coins_needed = 0 
firstmonthly_market_delta = 0 
firstlast_cardem = 0 
firstlast_busdem = 0 
firstlast_bikedem = 0 
firstlast_flows_car = 0 
firstlast_flows_bus = 0 
firstlast_flows_bike = 0 
firstmonthly_mocos_car = 0 
firstmonthly_mocos_bus = 0 
firstmonthly_mocos_bike = 0 
firstmonthly_emissions_car = 0 
firstmonthly_emissions_bus = 0 
firstmonthly_total_tt = 0 
firstmonthly_total_tt_c = 0 
firstmonthly_total_tt_b = 0 
firstmonthly_total_tt_r = 0 
firstmonthly_total_dist_car = 0 
firstmonthly_total_dist_bus = 0 
firstmonthly_total_dist_rad = 0
firstmonthly_lastday_market_delta = 0
lastavailable_firstgroupmocos = 0
lastfirstgroup_crowdfunding = 0
monthlyfirstgroup_crowdfunding = 0
firstgroup_totalcardem = 0
firstgroup_totalbusdem = 0
firstgroup_totalbikedem = 0
firstgroup_totaldem = 0

secondmonthly_mocperkgCO2 = 0 
secondlast_mocomp = 0 
secondmonthly_coins_allocated = 0 
secondmonthly_coins_needed = 0 
secondmonthly_market_delta = 0 
secondlast_cardem = 0 
secondlast_busdem = 0 
secondlast_bikedem = 0 
secondlast_flows_car = 0 
secondlast_flows_bus = 0 
secondlast_flows_bike = 0 
secondmonthly_mocos_car = 0 
secondmonthly_mocos_bus = 0 
secondmonthly_mocos_bike = 0 
secondmonthly_emissions_car = 0 
secondmonthly_emissions_bus = 0 
secondmonthly_total_tt = 0 
secondmonthly_total_tt_c = 0 
secondmonthly_total_tt_b = 0 
secondmonthly_total_tt_r = 0 
secondmonthly_total_dist_car = 0 
secondmonthly_total_dist_bus = 0 
secondmonthly_total_dist_rad = 0
secondmonthly_lastday_market_delta = 0
lastavailable_secondgroupmocos = 0
lastsecondgroup_crowdfunding = 0
monthlysecondgroup_crowdfunding = 0
secondgroup_totalcardem = 0
secondgroup_totalbusdem = 0
secondgroup_totalbikedem = 0
secondgroup_totaldem = 0

mocperkg = [1.0]
    # 2.5,
    # 5.0,
    # 7.5]
daily_result_row = ()
first_group_results_df = DataFrame()
first_group_of_multi_monthly_results_df= DataFrame()
first_group_of_multi_monthly_result_row = ()

second_group_results_df = DataFrame()
second_group_of_multi_monthly_results_df= DataFrame()
second_group_of_multi_monthly_result_row = ()

for x in 60:-1:1 # 20 working days in a month during 3 month from 2025 to 2030
    #for coinsallocated in coinsallocatedarr1
    for m1 in 10:-1:1 # 10 working days in a month for first group
        if x == 60
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 20,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.27 / (1 + exp(0.22 * (m1 + 32.69))) # 10 first days of 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstgroupmocos -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstgroupmocos
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
           dailycoinsallocated = daily_result_row.coins_allocated
    
           mocperkgCO2 = daily_result_row.mocperkgCO2
           initialMP = daily_result_row.mocomp
        end
    end

    for m1 in 20:-1:1 # 10 working days in a month for first group
        if x % 20 == 10
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.27 / (1 + exp(0.22 * (m1 + 32.69))) # 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstmonthlygroupmocos -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstmonthlygroupmocos
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
            dailycoinsallocated = daily_result_row.coins_allocated
    
            mocperkgCO2 = daily_result_row.mocperkgCO2
            initialMP = daily_result_row.mocomp
        end
    end

    for m1 in 10:-1:1 # 10 working days in a month for first group
        if x == 10
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.2730544491106 / (1 + exp(0.22186006853514756 * (m1 + 32.68639136103844))) # 10 last days of 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstgroupmocoslastmonth -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstgroupmocoslastmonth
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
            dailycoinsallocated = daily_result_row.coins_allocated
    
            mocperkgCO2 = daily_result_row.mocperkgCO2
            initialMP = daily_result_row.mocomp
        end
    end
    
    if x % 20 == 0

    first_group_of_multi_monthly_result_row = ( 
        firstmonthly_mocperkgCO2 = firstmonthly_mocperkgCO2, firstlast_mocomp, firstmonthly_coins_allocated, firstmonthly_coins_needed, firstmonthly_lastday_market_delta,
        firstmonthly_market_delta, firstlast_cardem, firstlast_busdem, firstlast_bikedem, 
        firstlast_flows_car, firstlast_flows_bus, firstlast_flows_bike, 
        firstmonthly_mocos_car, firstmonthly_mocos_bus, firstmonthly_mocos_bike, firstmonthly_emissions_car, firstmonthly_emissions_bus, 
        firstmonthly_total_tt, firstmonthly_total_tt_c, firstmonthly_total_tt_b, firstmonthly_total_tt_r, 
        firstmonthly_total_dist_car, firstmonthly_total_dist_bus, firstmonthly_total_dist_rad,
        firstlasttotalcardem, firstlasttotalbusdem, firstlasttotalbikedem, firstlasttotaldem,
        firstcoinsallocated, lastavailable_firstgroupmocos, lastfirstgroup_crowdfunding, monthlyfirstgroup_crowdfunding,
        firstgroup_totalcardem, firstgroup_totalbusdem, firstgroup_totalbikedem, firstgroup_totaldem #first_group_first_month_difference_of_demand
    )

    first_group_of_multi_monthly_result = DataFrame([first_group_of_multi_monthly_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(first_group_of_multi_monthly_results_df, first_group_of_multi_monthly_result[1, :])

    # Reorder columns to put :day at the front
    #select!(first_group_results_df, [:day; setdiff(names(first_group_results_df), [:day])])
    if x == 60
    # Calculate totals for numeric columns
    numeric_cols = names(first_group_results_df)[findall(eltype.(eachcol(first_group_results_df)) .== Number)]
    # Identify numeric columns
    numeric_cols = names(first_group_results_df)[findall(x -> eltype(x) <: Number, eachcol(first_group_results_df))]
    totals_row = [sum(first_group_results_df[!, col]) for col in numeric_cols]

    # Create a new row for totals
    first_group_totals_row = Dict{Symbol, Any}()

    for col in names(first_group_results_df)
        if col in numeric_cols
            first_group_totals_row[col] = sum(skipmissing(first_group_results_df[!, col]))
        else
            first_group_totals_row[col] = "Total"  # Or `missing`, or keep the column label
        end
    end

    # Create a new row for totals
    #first_group_totals_df = DataFrame()
    #for col in names(first_group_results_df)
    #   if col in numeric_cols
    #      first_group_totals_df[!, col] = [sum(first_group_results_df[!, col])]
    #else
        #first_group_totals_df[!, col] = [missing]  # or "Total" if you want label text
   # end
   #end

    # Convert the :day column to Any to allow mix of Int and String
    #first_group_results_df.day = convert(Vector{Any}, first_group_results_df.day)
    #first_group_totals_df.day = convert(Vector{Any}, first_group_totals_df.day)

    # Append the totals row to the results
    #append!(first_group_results_df, first_group_totals_df)
    # Append the totals row to the DataFrame
    push!(first_group_results_df, first_group_totals_row)
    end
    end

    # Optional: Save to CSV
    CSV.write("first_group_monthly_results_summary.csv", first_group_results_df)
    CSV.write("First_Groupe_of_Multi_Tradable_Scheme_Monthly_Result.csv", first_group_of_multi_monthly_results_df)

    println(first_group_of_multi_monthly_result_row)

    for m2 in 20:-1:1 # 20 working days in a month for second group
        if x % 20 == 0
        #for coinsallocated in coinsallocatedarr1
        for mocperkgCO2 in mocperkg
    
            local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
            # Optimizing the objective function
            # You can choose different methods available in Optim, here BFGS is used as an example
            result = optimize(local_objective, 0.0, 10.0, Brent();
                        iterations = 15,
                        store_trace = true,
                        show_trace = true
                        )
        
            # Extracting the optimized value of mocomp
            optimized_mocomp = Optim.minimizer(result)
            optimized_value = Optim.minimum(result)
            #iter = iterations(result)
            #traceres = trace(result)
            # Find the row where delta is minimized
            min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
            println("Optimization happened at row: ", min_delta_index) 
        
            # Save the row of minimum delta in a separate file
            daily_result_row = ( 
                mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                mocomp = MParr[min_delta_index], 
                coins_allocated = coinsallocatedarr[min_delta_index], 
                coins_needed = coinsneededarr[min_delta_index], 
                market_delta = deltaarr[min_delta_index], 
                cardem = cardemarr[min_delta_index], 
                busdem = busdemarr[min_delta_index], 
                bikedem = bikedemarr[min_delta_index], 
                flows_car = flowscararr[min_delta_index], 
                flows_bus = flowsbusarr[min_delta_index], 
                flows_bike = flowsbikearr[min_delta_index], 
                mocos_car = mocoscararr[min_delta_index], 
                mocos_bus = mocosbusarr[min_delta_index], 
                mocos_bike = mocosbikearr[min_delta_index], 
                emissions_car = emissionscararr[min_delta_index], 
                emissions_bus = emissionsbusarr[min_delta_index], 
                total_tt = totaltt[min_delta_index], 
                total_tt_c = totalttc[min_delta_index], 
                total_tt_b = totalttb[min_delta_index], 
                total_tt_r = totalttr[min_delta_index], 
                total_dist_car = totaldcarr[min_delta_index], 
                total_dist_bus = totaldbarr[min_delta_index], 
                total_dist_rad = totaldrarr[min_delta_index]
            )
        end
        allocated_mocos += daily_result_row.coins_allocated
        consuming_mocos += daily_result_row.coins_needed
        exp_model(m2) = 897.2730544491106 / (1 + exp(0.22186006853514756 * (m2 + 32.68639136103844))) # 20 working days in a month
        secondgroupcrowdfunding = exp_model(m2) * (daily_result_row.coins_allocated)
        #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
        secondgrouptotalcrowdfunding += secondgroupcrowdfunding
        secondgroupactualdailycoinsallocated = daily_result_row.coins_allocated - secondgroupcrowdfunding
    
        # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
        second_group_daily_row = DataFrame([daily_result_row])
        #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
        push!(second_group_results_df, second_group_daily_row[1, :])
          
        secondgroupmocos -= (secondgroupactualdailycoinsallocated + secondgroupcrowdfunding)
    
        secondnextcardem = daily_result_row.cardem
        secondnextbusdem = daily_result_row.busdem
        secondnextbikedem = daily_result_row.bikedem
        secondnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
    
        available_allocated_mocos = allocated_mocos #+ crowdfunding
        available_consuming_mocos = consuming_mocos #+ crowdfunding

        secondlasttotalcardem = secondnextcardem
        secondlasttotalbusdem = secondnextbusdem
        secondlasttotalbikedem = secondnextbikedem
        secondlasttotaldem = secondnextdem

        secondtotalcardem += secondnextcardem
        secondtotalbusdem += secondnextcardem
        secondtotalbikedem += secondnextcardem
        secondtotaldem += secondnextcardem

        #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem

        secondmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
        secondlast_mocomp = daily_result_row.mocomp
        secondmonthly_coins_allocated += daily_result_row.coins_allocated
        secondmonthly_coins_needed += daily_result_row.coins_needed
        secondmonthly_lastday_market_delta = daily_result_row.market_delta
        secondmonthly_market_delta = allocated_mocos - consuming_mocos
        secondlast_cardem = daily_result_row.cardem
        secondlast_busdem = daily_result_row.busdem
        secondlast_bikedem = daily_result_row.bikedem
        secondlast_flows_car = daily_result_row.flows_car
        secondlast_flows_bus = daily_result_row.flows_bus
        secondlast_flows_bike = daily_result_row.flows_bike
        secondmonthly_mocos_car += (daily_result_row.mocos_car)
        secondmonthly_mocos_bus += (daily_result_row.mocos_bus)
        secondmonthly_mocos_bike += (daily_result_row.mocos_bike)
        secondmonthly_emissions_car += (daily_result_row.emissions_car)
        secondmonthly_emissions_bus += (daily_result_row.emissions_bus)
        secondmonthly_total_tt += (daily_result_row.total_tt)
        secondmonthly_total_tt_c += (daily_result_row.total_tt_c) 
        secondmonthly_total_tt_b += (daily_result_row.total_tt_r)
        secondmonthly_total_dist_car += (daily_result_row.total_dist_car)
        secondmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
        secondmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
        seconddailycoinsallocated = daily_result_row.coins_allocated
        #secondgroupparticipants = secondgroupparticipants
        lastavailable_secondgroupmocos = secondgroupmocos
        lastsecondgroup_crowdfunding = secondgroupcrowdfunding
        monthlysecondgroup_crowdfunding = secondgrouptotalcrowdfunding
        secondgroup_totalcardem = secondtotalcardem
        secondgroup_totalbusdem = secondtotalbusdem
        secondgroup_totalbikedem = secondtotalbikedem
        secondgroup_totaldem = secondtotaldem

       dailycoinsallocated = daily_result_row.coins_allocated

       mocperkgCO2 = daily_result_row.mocperkgCO2
       initialMP = daily_result_row.mocomp
    end
    end
    
    if x % 20 == 0

    second_group_of_multi_monthly_result_row = ( 
        secondmonthly_mocperkgCO2 = secondmonthly_mocperkgCO2, secondlast_mocomp, secondmonthly_coins_allocated, secondmonthly_coins_needed, secondmonthly_lastday_market_delta,
        secondmonthly_market_delta, secondlast_cardem, secondlast_busdem, secondlast_bikedem, 
        secondlast_flows_car, secondlast_flows_bus, secondlast_flows_bike, 
        secondmonthly_mocos_car, secondmonthly_mocos_bus, secondmonthly_mocos_bike, secondmonthly_emissions_car, secondmonthly_emissions_bus, 
        secondmonthly_total_tt, secondmonthly_total_tt_c, secondmonthly_total_tt_b, secondmonthly_total_tt_r, 
        secondmonthly_total_dist_car, secondmonthly_total_dist_bus, secondmonthly_total_dist_rad,
        secondlasttotalcardem, secondlasttotalbusdem, secondlasttotalbikedem, secondlasttotaldem,
        secondcoinsallocated, lastavailable_secondgroupmocos, lastsecondgroup_crowdfunding, monthlysecondgroup_crowdfunding,
        secondgroup_totalcardem, secondgroup_totalbusdem, secondgroup_totalbikedem, secondgroup_totaldem #first_group_first_month_difference_of_demand
    )


    second_group_of_multi_monthly_result = DataFrame([second_group_of_multi_monthly_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(second_group_of_multi_monthly_results_df, second_group_of_multi_monthly_result[1, :])

    # Reorder columns to put :day at the front
    #select!(first_group_results_df, [:day; setdiff(names(first_group_results_df), [:day])])
    if x == 60
    # Calculate totals for numeric columns
    numeric_cols = names(second_group_results_df)[findall(eltype.(eachcol(second_group_results_df)) .== Number)]
    # Identify numeric columns
    numeric_cols = names(second_group_results_df)[findall(x -> eltype(x) <: Number, eachcol(second_group_results_df))]
    totals_row = [sum(second_group_results_df[!, col]) for col in numeric_cols]

    # Create a new row for totals
    second_group_totals_row = Dict{Symbol, Any}()

    for col in names(second_group_results_df)
        if col in numeric_cols
            second_group_totals_row[col] = sum(skipmissing(second_group_results_df[!, col]))
        else
            second_group_totals_row[col] = "Total"  # Or `missing`, or keep the column label
        end
    end

    # Create a new row for totals
    #first_group_totals_df = DataFrame()
    #for col in names(first_group_results_df)
    #   if col in numeric_cols
    #      first_group_totals_df[!, col] = [sum(first_group_results_df[!, col])]
    #else
        #first_group_totals_df[!, col] = [missing]  # or "Total" if you want label text
   # end
   #end

    # Convert the :day column to Any to allow mix of Int and String
    #first_group_results_df.day = convert(Vector{Any}, first_group_results_df.day)
    #first_group_totals_df.day = convert(Vector{Any}, first_group_totals_df.day)

    # Append the totals row to the results
    #append!(first_group_results_df, first_group_totals_df)
    # Append the totals row to the DataFrame
    push!(second_group_results_df, second_group_totals_row)
end
end
    # Optional: Save to CSV
    CSV.write("second_group_monthly_results_summary.csv", second_group_results_df)
    CSV.write("Second_Groupe_of_Multi_Tradable_Scheme_Monthly_Result.csv", second_group_of_multi_monthly_results_df)

    println(second_group_of_multi_monthly_result_row)
end

     0     9.497045e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 10.0
 * minimizer: 3.819660112501051
     1     5.603022e+03              NaN
 * x_lower: 3.819660112501051
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 6.180339887498947
     2     3.308590e+03              NaN
 * x_lower: 6.180339887498947
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 7.639320225002102
     3     2.174937e+03              NaN
 * x_lower: 7.639320225002102
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 8.541019662496845
     4     8.377848e+02              NaN
 * x_lower: 8.541019662496845
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.098300562505257
     5     7.094080e+02              NaN
 * x_lower: 9.098300562505257
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     6     7.094080e+02              NaN
 * x_lower: 9.35342706928507
 * best bound: lower
 * x_upper: 10.0
 * minimizer: 9.442719099991587
     7     4.303913e+02   

Excessive output truncated after 524305 bytes.

     7     4.303913e+02              NaN
 * x_lower: 9.442719099991587


In [ ]:
# 2 Groups of allocated mobility coins at the beginning of the month
# first group received mobility coins which is necessary for a half month travelling which is half of the population
firstgroupmocos = monthly_result_row.first_available_allocated_mocos * 0.5
firstmonthlygroupmocos = monthly_result_row.first_available_allocated_mocos * 1
firstgroupmocoslastmonth = monthly_result_row.first_available_allocated_mocos * 0.5
secondgroupmocos = monthly_result_row.first_available_allocated_mocos * 1

InitialTotalCoinsforBothGroups = firstgroupmocos + secondgroupmocos

#firstgroupparticipants = 0.5 * monthly_result_row.totaldem
#secondgroupparticipants = 0.5 * monthly_result_row.totaldem

x = 60 # number of working days in 3 months (from 2025 to 2030)
m1 = 10 # number of working days in first and last allocation of first group
m2 = 20 # number of working days in second allocation of first group and all allocation of second group

firstcoinsallocated = firstgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6
secondcoinsallocated = secondgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6

dailycoinsallocated = 655030 #645860 # 1.1369356986281308e6 # 1.164475393364052e6

first_allocated_mocos = 0
first_consuming_mocos = 0

allocated_mocos = 0
consuming_mocos = 0

firstgroupcrowdfunding = 0
secondgroupcrowdfunding = 0

firstgrouptotalcrowdfunding = 0
secondgrouptotalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

firsttotalcardem = 0
firsttotalbusdem = 0
firsttotalbikedem = 0
firsttotaldem = 0

secondtotalcardem = 0
secondtotalbusdem = 0
secondtotalbikedem = 0
secondtotaldem = 0

firstlasttotalcardem = 0
firstlasttotalbusdem = 0
firstlasttotalbikedem = 0
firstlasttotaldem = 0

secondlasttotalcardem = 0
secondlasttotalbusdem = 0
secondlasttotalbikedem = 0
secondlasttotaldem = 0

firstmonthly_mocperkgCO2 = 0
firstlast_mocomp = 0   
firstmonthly_mocperkgCO2 = 0 
firstlast_mocomp = 0 
firstmonthly_coins_allocated = 0 
firstmonthly_coins_needed = 0 
firstmonthly_market_delta = 0 
firstlast_cardem = 0 
firstlast_busdem = 0 
firstlast_bikedem = 0 
firstlast_flows_car = 0 
firstlast_flows_bus = 0 
firstlast_flows_bike = 0 
firstmonthly_mocos_car = 0 
firstmonthly_mocos_bus = 0 
firstmonthly_mocos_bike = 0 
firstmonthly_emissions_car = 0 
firstmonthly_emissions_bus = 0 
firstmonthly_total_tt = 0 
firstmonthly_total_tt_c = 0 
firstmonthly_total_tt_b = 0 
firstmonthly_total_tt_r = 0 
firstmonthly_total_dist_car = 0 
firstmonthly_total_dist_bus = 0 
firstmonthly_total_dist_rad = 0
firstmonthly_lastday_market_delta = 0
lastavailable_firstgroupmocos = 0
lastfirstgroup_crowdfunding = 0
monthlyfirstgroup_crowdfunding = 0
firstgroup_totalcardem = 0
firstgroup_totalbusdem = 0
firstgroup_totalbikedem = 0
firstgroup_totaldem = 0

secondmonthly_mocperkgCO2 = 0 
secondlast_mocomp = 0 
secondmonthly_coins_allocated = 0 
secondmonthly_coins_needed = 0 
secondmonthly_market_delta = 0 
secondlast_cardem = 0 
secondlast_busdem = 0 
secondlast_bikedem = 0 
secondlast_flows_car = 0 
secondlast_flows_bus = 0 
secondlast_flows_bike = 0 
secondmonthly_mocos_car = 0 
secondmonthly_mocos_bus = 0 
secondmonthly_mocos_bike = 0 
secondmonthly_emissions_car = 0 
secondmonthly_emissions_bus = 0 
secondmonthly_total_tt = 0 
secondmonthly_total_tt_c = 0 
secondmonthly_total_tt_b = 0 
secondmonthly_total_tt_r = 0 
secondmonthly_total_dist_car = 0 
secondmonthly_total_dist_bus = 0 
secondmonthly_total_dist_rad = 0
secondmonthly_lastday_market_delta = 0
lastavailable_secondgroupmocos = 0
lastsecondgroup_crowdfunding = 0
monthlysecondgroup_crowdfunding = 0
secondgroup_totalcardem = 0
secondgroup_totalbusdem = 0
secondgroup_totalbikedem = 0
secondgroup_totaldem = 0

mocperkg = [1.0]
    # 2.5,
    # 5.0,
    # 7.5]
daily_result_row = ()
first_group_results_df = DataFrame()
first_group_of_multi_monthly_results_df= DataFrame()
first_group_of_multi_monthly_result_row = ()

second_group_results_df = DataFrame()
second_group_of_multi_monthly_results_df= DataFrame()
second_group_of_multi_monthly_result_row = ()

for x in 60:-1:1 # 20 working days in a month during 3 month from 2025 to 2030
    #for coinsallocated in coinsallocatedarr1
    for m1 in 10:-1:1 # 10 working days in a month for first group
        if x == 60
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 20,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.27 / (1 + exp(0.22 * (m1 + 32.69))) # 10 first days of 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstgroupmocos -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstgroupmocos
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
           dailycoinsallocated = daily_result_row.coins_allocated
    
           mocperkgCO2 = daily_result_row.mocperkgCO2
           initialMP = daily_result_row.mocomp
        end
    end

    for m1 in 20:-1:1 # 10 working days in a month for first group
        if x % 20 == 10
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.27 / (1 + exp(0.22 * (m1 + 32.69))) # 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstmonthlygroupmocos -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstmonthlygroupmocos
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
            dailycoinsallocated = daily_result_row.coins_allocated
    
            mocperkgCO2 = daily_result_row.mocperkgCO2
            initialMP = daily_result_row.mocomp
        end
    end

    for m1 in 10:-1:1 # 10 working days in a month for first group
        if x == 10
            #for coinsallocated in coinsallocatedarr1
            for mocperkgCO2 in mocperkg
        
                local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
                # Optimizing the objective function
                # You can choose different methods available in Optim, here BFGS is used as an example
                result = optimize(local_objective, 0.0, 10.0, Brent();
                            iterations = 15,
                            store_trace = true,
                            show_trace = true
                            )
            
                # Extracting the optimized value of mocomp
                optimized_mocomp = Optim.minimizer(result)
                optimized_value = Optim.minimum(result)
                #iter = iterations(result)
                #traceres = trace(result)
                # Find the row where delta is minimized
                min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
                println("Optimization happened at row: ", min_delta_index) 
            
                # Save the row of minimum delta in a separate file
                daily_result_row = ( 
                    mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                    mocomp = MParr[min_delta_index], 
                    coins_allocated = coinsallocatedarr[min_delta_index], 
                    coins_needed = coinsneededarr[min_delta_index], 
                    market_delta = deltaarr[min_delta_index], 
                    cardem = cardemarr[min_delta_index], 
                    busdem = busdemarr[min_delta_index], 
                    bikedem = bikedemarr[min_delta_index], 
                    flows_car = flowscararr[min_delta_index], 
                    flows_bus = flowsbusarr[min_delta_index], 
                    flows_bike = flowsbikearr[min_delta_index], 
                    mocos_car = mocoscararr[min_delta_index], 
                    mocos_bus = mocosbusarr[min_delta_index], 
                    mocos_bike = mocosbikearr[min_delta_index], 
                    emissions_car = emissionscararr[min_delta_index], 
                    emissions_bus = emissionsbusarr[min_delta_index], 
                    total_tt = totaltt[min_delta_index], 
                    total_tt_c = totalttc[min_delta_index], 
                    total_tt_b = totalttb[min_delta_index], 
                    total_tt_r = totalttr[min_delta_index], 
                    total_dist_car = totaldcarr[min_delta_index], 
                    total_dist_bus = totaldbarr[min_delta_index], 
                    total_dist_rad = totaldrarr[min_delta_index]
                )
            end
            first_allocated_mocos += daily_result_row.coins_allocated
            first_consuming_mocos += daily_result_row.coins_needed
            exp_model(m1) = 897.2730544491106 / (1 + exp(0.22186006853514756 * (m1 + 32.68639136103844))) # 10 last days of 20 working days in a month
            firstgroupcrowdfunding = exp_model(m1) * (daily_result_row.coins_allocated)
            #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
            firstgrouptotalcrowdfunding += firstgroupcrowdfunding
            firstgroupactualdailycoinsallocated = daily_result_row.coins_allocated - firstgroupcrowdfunding
        
            # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
            first_group_daily_row = DataFrame([daily_result_row])
            #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
            push!(first_group_results_df, first_group_daily_row[1, :])
              
            firstgroupmocoslastmonth -= (firstgroupactualdailycoinsallocated + firstgroupcrowdfunding)
        
            firstnextcardem = daily_result_row.cardem
            firstnextbusdem = daily_result_row.busdem
            firstnextbikedem = daily_result_row.bikedem
            firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
        
            available_allocated_mocos = first_allocated_mocos #+ crowdfunding
            available_consuming_mocos = first_consuming_mocos #+ crowdfunding
    
            firstlasttotalcardem = firstnextcardem
            firstlasttotalbusdem = firstnextbusdem
            firstlasttotalbikedem = firstnextbikedem
            firstlasttotaldem = firstnextdem
    
            firsttotalcardem += firstnextcardem
            firsttotalbusdem += firstnextcardem
            firsttotalbikedem += firstnextcardem
            firsttotaldem += firstnextcardem
    
            #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem
    
            firstmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
            firstlast_mocomp = daily_result_row.mocomp
            firstmonthly_coins_allocated += daily_result_row.coins_allocated
            firstmonthly_coins_needed += daily_result_row.coins_needed
            firstmonthly_lastday_market_delta = daily_result_row.market_delta
            firstmonthly_market_delta = first_allocated_mocos - first_consuming_mocos
            firstlast_cardem = daily_result_row.cardem
            firstlast_busdem = daily_result_row.busdem
            firstlast_bikedem = daily_result_row.bikedem
            firstlast_flows_car = daily_result_row.flows_car
            firstlast_flows_bus = daily_result_row.flows_bus
            firstlast_flows_bike = daily_result_row.flows_bike
            firstmonthly_mocos_car += (daily_result_row.mocos_car)
            firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
            firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
            firstmonthly_emissions_car += (daily_result_row.emissions_car)
            firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
            firstmonthly_total_tt += (daily_result_row.total_tt)
            firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
            firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
            firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
            firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
            firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
            firstdailycoinsallocated = daily_result_row.coins_allocated
            #firstgroupparticipants = firstgroupparticipants
            lastavailable_firstgroupmocos = firstgroupmocoslastmonth
            lastfirstgroup_crowdfunding = firstgroupcrowdfunding
            monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
            firstgroup_totalcardem = firsttotalcardem
            firstgroup_totalbusdem = firsttotalbusdem
            firstgroup_totalbikedem = firsttotalbikedem
            firstgroup_totaldem = firsttotaldem
    
            dailycoinsallocated = daily_result_row.coins_allocated
    
            mocperkgCO2 = daily_result_row.mocperkgCO2
            initialMP = daily_result_row.mocomp
        end
    end
    
    if x % 20 == 0

    first_group_of_multi_monthly_result_row = ( 
        firstmonthly_mocperkgCO2 = firstmonthly_mocperkgCO2, firstlast_mocomp, firstmonthly_coins_allocated, firstmonthly_coins_needed, firstmonthly_lastday_market_delta,
        firstmonthly_market_delta, firstlast_cardem, firstlast_busdem, firstlast_bikedem, 
        firstlast_flows_car, firstlast_flows_bus, firstlast_flows_bike, 
        firstmonthly_mocos_car, firstmonthly_mocos_bus, firstmonthly_mocos_bike, firstmonthly_emissions_car, firstmonthly_emissions_bus, 
        firstmonthly_total_tt, firstmonthly_total_tt_c, firstmonthly_total_tt_b, firstmonthly_total_tt_r, 
        firstmonthly_total_dist_car, firstmonthly_total_dist_bus, firstmonthly_total_dist_rad,
        firstlasttotalcardem, firstlasttotalbusdem, firstlasttotalbikedem, firstlasttotaldem,
        firstcoinsallocated, lastavailable_firstgroupmocos, lastfirstgroup_crowdfunding, monthlyfirstgroup_crowdfunding,
        firstgroup_totalcardem, firstgroup_totalbusdem, firstgroup_totalbikedem, firstgroup_totaldem #first_group_first_month_difference_of_demand
    )

    first_group_of_multi_monthly_result = DataFrame([first_group_of_multi_monthly_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(first_group_of_multi_monthly_results_df, first_group_of_multi_monthly_result[1, :])

    # Reorder columns to put :day at the front
    #select!(first_group_results_df, [:day; setdiff(names(first_group_results_df), [:day])])
    if x == 60
    # Calculate totals for numeric columns
    numeric_cols = names(first_group_results_df)[findall(eltype.(eachcol(first_group_results_df)) .== Number)]
    # Identify numeric columns
    numeric_cols = names(first_group_results_df)[findall(x -> eltype(x) <: Number, eachcol(first_group_results_df))]
    totals_row = [sum(first_group_results_df[!, col]) for col in numeric_cols]

    # Create a new row for totals
    first_group_totals_row = Dict{Symbol, Any}()

    for col in names(first_group_results_df)
        if col in numeric_cols
            first_group_totals_row[col] = sum(skipmissing(first_group_results_df[!, col]))
        else
            first_group_totals_row[col] = "Total"  # Or `missing`, or keep the column label
        end
    end

    # Create a new row for totals
    #first_group_totals_df = DataFrame()
    #for col in names(first_group_results_df)
    #   if col in numeric_cols
    #      first_group_totals_df[!, col] = [sum(first_group_results_df[!, col])]
    #else
        #first_group_totals_df[!, col] = [missing]  # or "Total" if you want label text
   # end
   #end

    # Convert the :day column to Any to allow mix of Int and String
    #first_group_results_df.day = convert(Vector{Any}, first_group_results_df.day)
    #first_group_totals_df.day = convert(Vector{Any}, first_group_totals_df.day)

    # Append the totals row to the results
    #append!(first_group_results_df, first_group_totals_df)
    # Append the totals row to the DataFrame
    push!(first_group_results_df, first_group_totals_row)
    end
    end

    # Optional: Save to CSV
    CSV.write("first_group_monthly_results_summary.csv", first_group_results_df)
    CSV.write("First_Groupe_of_Multi_Tradable_Scheme_Monthly_Result.csv", first_group_of_multi_monthly_results_df)

    println(first_group_of_multi_monthly_result_row)

    for m2 in 20:-1:1 # 20 working days in a month for second group
        if x % 20 == 0
        #for coinsallocated in coinsallocatedarr1
        for mocperkgCO2 in mocperkg
    
            local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
            # Optimizing the objective function
            # You can choose different methods available in Optim, here BFGS is used as an example
            result = optimize(local_objective, 0.0, 10.0, Brent();
                        iterations = 15,
                        store_trace = true,
                        show_trace = true
                        )
        
            # Extracting the optimized value of mocomp
            optimized_mocomp = Optim.minimizer(result)
            optimized_value = Optim.minimum(result)
            #iter = iterations(result)
            #traceres = trace(result)
            # Find the row where delta is minimized
            min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
            println("Optimization happened at row: ", min_delta_index) 
        
            # Save the row of minimum delta in a separate file
            daily_result_row = ( 
                mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                mocomp = MParr[min_delta_index], 
                coins_allocated = coinsallocatedarr[min_delta_index], 
                coins_needed = coinsneededarr[min_delta_index], 
                market_delta = deltaarr[min_delta_index], 
                cardem = cardemarr[min_delta_index], 
                busdem = busdemarr[min_delta_index], 
                bikedem = bikedemarr[min_delta_index], 
                flows_car = flowscararr[min_delta_index], 
                flows_bus = flowsbusarr[min_delta_index], 
                flows_bike = flowsbikearr[min_delta_index], 
                mocos_car = mocoscararr[min_delta_index], 
                mocos_bus = mocosbusarr[min_delta_index], 
                mocos_bike = mocosbikearr[min_delta_index], 
                emissions_car = emissionscararr[min_delta_index], 
                emissions_bus = emissionsbusarr[min_delta_index], 
                total_tt = totaltt[min_delta_index], 
                total_tt_c = totalttc[min_delta_index], 
                total_tt_b = totalttb[min_delta_index], 
                total_tt_r = totalttr[min_delta_index], 
                total_dist_car = totaldcarr[min_delta_index], 
                total_dist_bus = totaldbarr[min_delta_index], 
                total_dist_rad = totaldrarr[min_delta_index]
            )
        end
        allocated_mocos += daily_result_row.coins_allocated
        consuming_mocos += daily_result_row.coins_needed
        exp_model(m2) = 897.2730544491106 / (1 + exp(0.22186006853514756 * (m2 + 32.68639136103844))) # 20 working days in a month
        secondgroupcrowdfunding = exp_model(m2) * (daily_result_row.coins_allocated)
        #traded_mocos_without_transfering = allocated_mocos - consuming_mocos - crowdfunding
        secondgrouptotalcrowdfunding += secondgroupcrowdfunding
        secondgroupactualdailycoinsallocated = daily_result_row.coins_allocated - secondgroupcrowdfunding
    
        # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
        second_group_daily_row = DataFrame([daily_result_row])
        #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
        push!(second_group_results_df, second_group_daily_row[1, :])
          
        secondgroupmocos -= (secondgroupactualdailycoinsallocated + secondgroupcrowdfunding)
    
        secondnextcardem = daily_result_row.cardem
        secondnextbusdem = daily_result_row.busdem
        secondnextbikedem = daily_result_row.bikedem
        secondnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)
    
        available_allocated_mocos = allocated_mocos #+ crowdfunding
        available_consuming_mocos = consuming_mocos #+ crowdfunding

        secondlasttotalcardem = secondnextcardem
        secondlasttotalbusdem = secondnextbusdem
        secondlasttotalbikedem = secondnextbikedem
        secondlasttotaldem = secondnextdem

        secondtotalcardem += secondnextcardem
        secondtotalbusdem += secondnextcardem
        secondtotalbikedem += secondnextcardem
        secondtotaldem += secondnextcardem

        #first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem

        secondmonthly_mocperkgCO2 = daily_result_row.mocperkgCO2
        secondlast_mocomp = daily_result_row.mocomp
        secondmonthly_coins_allocated += daily_result_row.coins_allocated
        secondmonthly_coins_needed += daily_result_row.coins_needed
        secondmonthly_lastday_market_delta = daily_result_row.market_delta
        secondmonthly_market_delta = allocated_mocos - consuming_mocos
        secondlast_cardem = daily_result_row.cardem
        secondlast_busdem = daily_result_row.busdem
        secondlast_bikedem = daily_result_row.bikedem
        secondlast_flows_car = daily_result_row.flows_car
        secondlast_flows_bus = daily_result_row.flows_bus
        secondlast_flows_bike = daily_result_row.flows_bike
        secondmonthly_mocos_car += (daily_result_row.mocos_car)
        secondmonthly_mocos_bus += (daily_result_row.mocos_bus)
        secondmonthly_mocos_bike += (daily_result_row.mocos_bike)
        secondmonthly_emissions_car += (daily_result_row.emissions_car)
        secondmonthly_emissions_bus += (daily_result_row.emissions_bus)
        secondmonthly_total_tt += (daily_result_row.total_tt)
        secondmonthly_total_tt_c += (daily_result_row.total_tt_c) 
        secondmonthly_total_tt_b += (daily_result_row.total_tt_r)
        secondmonthly_total_dist_car += (daily_result_row.total_dist_car)
        secondmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
        secondmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
        seconddailycoinsallocated = daily_result_row.coins_allocated
        #secondgroupparticipants = secondgroupparticipants
        lastavailable_secondgroupmocos = secondgroupmocos
        lastsecondgroup_crowdfunding = secondgroupcrowdfunding
        monthlysecondgroup_crowdfunding = secondgrouptotalcrowdfunding
        secondgroup_totalcardem = secondtotalcardem
        secondgroup_totalbusdem = secondtotalbusdem
        secondgroup_totalbikedem = secondtotalbikedem
        secondgroup_totaldem = secondtotaldem

       dailycoinsallocated = daily_result_row.coins_allocated

       mocperkgCO2 = daily_result_row.mocperkgCO2
       initialMP = daily_result_row.mocomp
    end
    end
    
    if x % 20 == 0

    second_group_of_multi_monthly_result_row = ( 
        secondmonthly_mocperkgCO2 = secondmonthly_mocperkgCO2, secondlast_mocomp, secondmonthly_coins_allocated, secondmonthly_coins_needed, secondmonthly_lastday_market_delta,
        secondmonthly_market_delta, secondlast_cardem, secondlast_busdem, secondlast_bikedem, 
        secondlast_flows_car, secondlast_flows_bus, secondlast_flows_bike, 
        secondmonthly_mocos_car, secondmonthly_mocos_bus, secondmonthly_mocos_bike, secondmonthly_emissions_car, secondmonthly_emissions_bus, 
        secondmonthly_total_tt, secondmonthly_total_tt_c, secondmonthly_total_tt_b, secondmonthly_total_tt_r, 
        secondmonthly_total_dist_car, secondmonthly_total_dist_bus, secondmonthly_total_dist_rad,
        secondlasttotalcardem, secondlasttotalbusdem, secondlasttotalbikedem, secondlasttotaldem,
        secondcoinsallocated, lastavailable_secondgroupmocos, lastsecondgroup_crowdfunding, monthlysecondgroup_crowdfunding,
        secondgroup_totalcardem, secondgroup_totalbusdem, secondgroup_totalbikedem, secondgroup_totaldem #first_group_first_month_difference_of_demand
    )


    second_group_of_multi_monthly_result = DataFrame([second_group_of_multi_monthly_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(second_group_of_multi_monthly_results_df, second_group_of_multi_monthly_result[1, :])

    # Reorder columns to put :day at the front
    #select!(first_group_results_df, [:day; setdiff(names(first_group_results_df), [:day])])
    if x == 60
    # Calculate totals for numeric columns
    numeric_cols = names(second_group_results_df)[findall(eltype.(eachcol(second_group_results_df)) .== Number)]
    # Identify numeric columns
    numeric_cols = names(second_group_results_df)[findall(x -> eltype(x) <: Number, eachcol(second_group_results_df))]
    totals_row = [sum(second_group_results_df[!, col]) for col in numeric_cols]

    # Create a new row for totals
    second_group_totals_row = Dict{Symbol, Any}()

    for col in names(second_group_results_df)
        if col in numeric_cols
            second_group_totals_row[col] = sum(skipmissing(second_group_results_df[!, col]))
        else
            second_group_totals_row[col] = "Total"  # Or `missing`, or keep the column label
        end
    end

    # Create a new row for totals
    #first_group_totals_df = DataFrame()
    #for col in names(first_group_results_df)
    #   if col in numeric_cols
    #      first_group_totals_df[!, col] = [sum(first_group_results_df[!, col])]
    #else
        #first_group_totals_df[!, col] = [missing]  # or "Total" if you want label text
   # end
   #end

    # Convert the :day column to Any to allow mix of Int and String
    #first_group_results_df.day = convert(Vector{Any}, first_group_results_df.day)
    #first_group_totals_df.day = convert(Vector{Any}, first_group_totals_df.day)

    # Append the totals row to the results
    #append!(first_group_results_df, first_group_totals_df)
    # Append the totals row to the DataFrame
    push!(second_group_results_df, second_group_totals_row)
end
end
    # Optional: Save to CSV
    CSV.write("second_group_monthly_results_summary.csv", second_group_results_df)
    CSV.write("Second_Groupe_of_Multi_Tradable_Scheme_Monthly_Result.csv", second_group_of_multi_monthly_results_df)

    println(second_group_of_multi_monthly_result_row)
end

In [ ]:
# 2 Groups of allocated mobility coins at the beginning of the month
# first group received mobility coins which is necessary for one and a half month travelling which is half of the population
firstgroupmocos = monthly_result_row.first_available_allocated_mocos * 0.5
firstgroupparticipants = 0.5 * monthly_result_row.totaldem 

x = 30 # number of working days in 1.5 month

firstcoinsallocated = firstgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6

dailycoinsallocated = 645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

firstgroupcrowdfunding = 0

firstgrouptotalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

firsttotalcardem = 0
firsttotalbusdem = 0
firsttotalbikedem = 0
firsttotaldem = 0

firstlasttotalcardem = 0
firstlasttotalbusdem = 0
firstlasttotalbikedem = 0
firstlasttotaldem = 0

firstmonthly_mocperkgCO2 = 0
firstlast_mocomp = 0   
firstmonthly_mocperkgCO2 = 0 
firstlast_mocomp = 0 
firstmonthly_coins_allocated = 0 
firstmonthly_coins_needed = 0 
firstmonthly_market_delta = 0 
firstlast_cardem = 0 
firstlast_busdem = 0 
firstlast_bikedem = 0 
firstlast_flows_car = 0 
firstlast_flows_bus = 0 
firstlast_flows_bike = 0 
firstmonthly_mocos_car = 0 
firstmonthly_mocos_bus = 0 
firstmonthly_mocos_bike = 0 
firstmonthly_emissions_car = 0 
firstmonthly_emissions_bus = 0 
firstmonthly_total_tt = 0 
firstmonthly_total_tt_c = 0 
firstmonthly_total_tt_b = 0 
firstmonthly_total_tt_r = 0 
firstmonthly_total_dist_car = 0 
firstmonthly_total_dist_bus = 0 
firstmonthly_total_dist_rad = 0 

mocperkg = [1.0]
    # 2.5,
    # 5.0,
    # 7.5]
daily_result_row = ()
first_group_results_df = DataFrame()
first_group_of_multi_first_month_results_df= DataFrame()
first_group_of_multi_first_month_result_row = ()

for x in 30:-1:11 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
    
    local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, 1.0, 1.0, Brent();
                iterations = 15,
                store_trace = true,
                show_trace = true
                )

    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)
    # Find the row where delta is minimized
    min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
    println("Optimization happened at row: ", min_delta_index) 

    # Save the row of minimum delta in a separate file
    daily_result_row = ( 
        mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
        mocomp = MParr[min_delta_index], 
        coins_allocated = coinsallocatedarr[min_delta_index], 
        coins_needed = coinsneededarr[min_delta_index], 
        market_delta = deltaarr[min_delta_index], 
        cardem = cardemarr[min_delta_index], 
        busdem = busdemarr[min_delta_index], 
        bikedem = bikedemarr[min_delta_index], 
        flows_car = flowscararr[min_delta_index], 
        flows_bus = flowsbusarr[min_delta_index], 
        flows_bike = flowsbikearr[min_delta_index], 
        mocos_car = mocoscararr[min_delta_index], 
        mocos_bus = mocosbusarr[min_delta_index], 
        mocos_bike = mocosbikearr[min_delta_index], 
        emissions_car = emissionscararr[min_delta_index], 
        emissions_bus = emissionsbusarr[min_delta_index], 
        total_tt = totaltt[min_delta_index], 
        total_tt_c = totalttc[min_delta_index], 
        total_tt_b = totalttb[min_delta_index], 
        total_tt_r = totalttr[min_delta_index], 
        total_dist_car = totaldcarr[min_delta_index], 
        total_dist_bus = totaldbarr[min_delta_index], 
        total_dist_rad = totaldrarr[min_delta_index]
    )
    end

    fisrtgroupdailycoinsallocated = 645860

    exp_model(x) = 897.2730544491106 / (1 + exp(0.22186006853514756 * (x + 32.68639136103844))) # 20 working days in a month
    firstgroupcrowdfunding = exp_model(x) * (firstgroupmocos)
    firstgrouptotalcrowdfunding += firstgroupcrowdfunding
    fisrtgroupactualdailycoinsallocated = fisrtgroupdailycoinsallocated - firstgroupcrowdfunding

    # Create daily row with :day label after assigning values to first_group_actual_daily_result_row (Convert NamedTuple to DataFrame row and push!):
    first_group_daily_row = DataFrame([daily_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(first_group_results_df, first_group_daily_row[1, :])
      
    firstgroupmocos -= (fisrtgroupactualdailycoinsallocated + firstgroupcrowdfunding)

    firstnextcardem = daily_result_row.cardem
    firstnextbusdem = daily_result_row.busdem
    firstnextbikedem = daily_result_row.bikedem
    firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem)

    firstlasttotalcardem = firstnextcardem
    firstlasttotalbusdem = firstnextbusdem
    firstlasttotalbikedem = firstnextbikedem
    firstlasttotaldem = firstnextdem

    firsttotalcardem += firstnextcardem
    firsttotalbusdem += firstnextcardem
    firsttotalbikedem += firstnextcardem
    firsttotaldem += firstnextcardem

    first_group_first_month_difference_of_demand = firstgroupparticipants - firsttotaldem

    firstmonthly_mocperkgCO2 += (daily_result_row.mocperkgCO2)
    firstlast_mocomp = daily_result_row.mocomp
    firstmonthly_coins_allocated = monthly_result_row.first_available_allocated_mocos * 1.5
    firstmonthly_coins_needed += (daily_result_row.coins_needed)
    firstmonthly_lastday_market_delta = daily_result_row.market_delta
    firstlast_cardem = daily_result_row.cardem
    firstlast_busdem = daily_result_row.busdem
    firstlast_bikedem = daily_result_row.bikedem
    firstlast_flows_car = daily_result_row.flows_car
    firstlast_flows_bus = daily_result_row.flows_bus
    firstlast_flows_bike = daily_result_row.flows_bike
    firstmonthly_mocos_car += (daily_result_row.mocos_car)
    firstmonthly_mocos_bus += (daily_result_row.mocos_bus)
    firstmonthly_mocos_bike += (daily_result_row.mocos_bike)
    firstmonthly_emissions_car += (daily_result_row.emissions_car)
    firstmonthly_emissions_bus += (daily_result_row.emissions_bus)
    firstmonthly_total_tt += (daily_result_row.total_tt)
    firstmonthly_total_tt_c += (daily_result_row.total_tt_c) 
    firstmonthly_total_tt_b += (daily_result_row.total_tt_r)
    firstmonthly_total_dist_car += (daily_result_row.total_dist_car)
    firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus)
    firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad)
    firstcoinsallocated = firstcoinsallocated
    firstgroupparticipants = firstgroupparticipants
    lastavailable_firstgroupmocos = firstgroupmocos
    lastfirstgroup_crowdfunding = firstgroupcrowdfunding
    monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
    firstgroup_totalcardem = firsttotalcardem
    firstgroup_totalbusdem = firsttotalbusdem
    firstgroup_totalbikedem = firsttotalbikedem
    firstgroup_totaldem = firsttotaldem

    first_group_of_multi_first_month_result_row = ( 
    firstmonthly_mocperkgCO2 = firstmonthly_mocperkgCO2, firstlast_mocomp, firstmonthly_coins_allocated, firstmonthly_coins_needed, firstmonthly_lastday_market_delta,
    firstlast_cardem, firstlast_busdem, firstlast_bikedem, 
    firstlast_flows_car, firstlast_flows_bus, firstlast_flows_bike, 
    firstmonthly_mocos_car, firstmonthly_mocos_bus, firstmonthly_mocos_bike, firstmonthly_emissions_car, firstmonthly_emissions_bus, 
    firstmonthly_total_tt, firstmonthly_total_tt_c, firstmonthly_total_tt_b, firstmonthly_total_tt_r, 
    firstmonthly_total_dist_car, firstmonthly_total_dist_bus, firstmonthly_total_dist_rad,
    firstlasttotalcardem, firstlasttotalbusdem, firstlasttotalbikedem, firstlasttotaldem,
    firstcoinsallocated, firstgroupparticipants, lastavailable_firstgroupmocos, lastfirstgroup_crowdfunding, monthlyfirstgroup_crowdfunding,
    firstgroup_totalcardem, firstgroup_totalbusdem, firstgroup_totalbikedem, firstgroup_totaldem, first_group_first_month_difference_of_demand
    )


    first_group_of_multi_first_month_result = DataFrame([first_group_of_multi_first_month_result_row])
    #first_group_daily_row[!, :day] = [21 - x]  # So that day counts up from 1 to 20
    push!(first_group_of_multi_first_month_results_df, first_group_of_multi_first_month_result[1, :])


    dailycoinsallocated = fisrtgroupactualdailycoinsallocated

    mocperkgCO2 = daily_result_row.mocperkgCO2
    initialMP = daily_result_row.mocomp

end

# Reorder columns to put :day at the front
#select!(first_group_results_df, [:day; setdiff(names(first_group_results_df), [:day])])

# Calculate totals for numeric columns
numeric_cols = names(first_group_results_df)[findall(eltype.(eachcol(first_group_results_df)) .== Number)]
# Identify numeric columns
numeric_cols = names(first_group_results_df)[findall(x -> eltype(x) <: Number, eachcol(first_group_results_df))]
totals_row = [sum(first_group_results_df[!, col]) for col in numeric_cols]

# Create a new row for totals
first_group_totals_row = Dict{Symbol, Any}()

for col in names(first_group_results_df)
    if col in numeric_cols
        first_group_totals_row[col] = sum(skipmissing(first_group_results_df[!, col]))
    else
        first_group_totals_row[col] = "Total"  # Or `missing`, or keep the column label
    end
end

# Create a new row for totals
#first_group_totals_df = DataFrame()
#for col in names(first_group_results_df)
 #   if col in numeric_cols
  #      first_group_totals_df[!, col] = [sum(first_group_results_df[!, col])]
    #else
        #first_group_totals_df[!, col] = [missing]  # or "Total" if you want label text
   # end
#end

# Convert the :day column to Any to allow mix of Int and String
#first_group_results_df.day = convert(Vector{Any}, first_group_results_df.day)
#first_group_totals_df.day = convert(Vector{Any}, first_group_totals_df.day)

# Append the totals row to the results
#append!(first_group_results_df, first_group_totals_df)
# Append the totals row to the DataFrame
push!(first_group_results_df, first_group_totals_row)

# Optional: Save to CSV
CSV.write("first_group_first_month_results_summary.csv", first_group_results_df)
CSV.write("First_Groupe_of_Multi_Tradable_Scheme_First_Month_Result.csv", first_group_of_multi_first_month_results_df)

println(first_group_of_multi_first_month_result_row)

     0     1.649716e+03              NaN
 * x_lower: 1.0
 * best bound: initial
 * x_upper: 1.0
 * minimizer: 1.0
Optimization happened at row: 358
     0     1.420604e+04              NaN
 * x_lower: 1.0
 * best bound: initial
 * x_upper: 1.0
 * minimizer: 1.0
Optimization happened at row: 361


┌ Error: Error adding value to column :coins_allocated.
└ @ DataFrames /home/aamirhossein393/.julia/packages/DataFrames/S3ZFo/src/dataframerow/dataframerow.jl:346


InexactError: InexactError: Int64(630004.2474346015)

In [ ]:
# 2 Groups of allocated mobility coins at the beginning of the month
# second group received mobility coins which is necessary for one month travelling which is the other half of the population
secondgroupmocos = monthly_result_row.first_available_allocated_mocos * 0.5
secondgroupparticipants = 0.5 * monthly_result_row.totaldem 
x = 20 # number of working days in month

secondcoinsallocated = secondgroupmocos

coinsallocated = 645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

secondgroupcrowdfunding = 0
secondgrouptotalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

secondtotalcardem = 0
secondtotalbusdem = 0
secondtotalbikedem = 0
secondtotaldem = 0

secondlasttotalcardem = 0
secondlasttotalbusdem = 0
secondlasttotalbikedem = 0
secondlasttotaldem = 0

secondmonthly_mocperkgCO2 = 0 
secondlast_mocomp = 0 
secondmonthly_coins_allocated = 0 
secondmonthly_coins_needed = 0 
secondmonthly_market_delta = 0 
secondlast_cardem = 0 
secondlast_busdem = 0 
secondlast_bikedem = 0 
secondlast_flows_car = 0 
secondlast_flows_bus = 0 
secondlast_flows_bike = 0 
secondmonthly_mocos_car = 0 
secondmonthly_mocos_bus = 0 
secondmonthly_mocos_bike = 0 
secondmonthly_emissions_car = 0 
secondmonthly_emissions_bus = 0 
secondmonthly_total_tt = 0 
secondmonthly_total_tt_c = 0 
secondmonthly_total_tt_b = 0 
secondmonthly_total_tt_r = 0 
secondmonthly_total_dist_car = 0 
secondmonthly_total_dist_bus = 0 
secondmonthly_total_dist_rad = 0 

mocperkg = [1.0]
    # 2.5,
    # 5.0,
    # 7.5]
daily_result_row = ()
second_group_actual_daily_result_row = ()
second_group_results_df = DataFrame()
second_group_of_multi_first_month_result_row = ()

for x in 20:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
    
    local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, 0.0, 10.0, Brent();
                iterations = 15,
                store_trace = true,
                show_trace = true
                )

    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)
    # Find the row where delta is minimized
    min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
    println("Optimization happened at row: ", min_delta_index) 

    # Save the row of minimum delta in a separate file
    daily_result_row = ( 
        mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
        mocomp = MParr[min_delta_index], 
        coins_allocated = coinsallocatedarr[min_delta_index], 
        coins_needed = coinsneededarr[min_delta_index], 
        market_delta = deltaarr[min_delta_index], 
        cardem = cardemarr[min_delta_index], 
        busdem = busdemarr[min_delta_index], 
        bikedem = bikedemarr[min_delta_index], 
        flows_car = flowscararr[min_delta_index], 
        flows_bus = flowsbusarr[min_delta_index], 
        flows_bike = flowsbikearr[min_delta_index], 
        mocos_car = mocoscararr[min_delta_index], 
        mocos_bus = mocosbusarr[min_delta_index], 
        mocos_bike = mocosbikearr[min_delta_index], 
        emissions_car = emissionscararr[min_delta_index], 
        emissions_bus = emissionsbusarr[min_delta_index], 
        total_tt = totaltt[min_delta_index], 
        total_tt_c = totalttc[min_delta_index], 
        total_tt_b = totalttb[min_delta_index], 
        total_tt_r = totalttr[min_delta_index], 
        total_dist_car = totaldcarr[min_delta_index], 
        total_dist_bus = totaldbarr[min_delta_index], 
        total_dist_rad = totaldrarr[min_delta_index]
    )
    end

    exp_model(x) = 0.5775211856783368 * exp(-0.17571864924550018 * (x)) # 20 working days in a month
    firstgroupcrowdfunding = exp_model(x) * (firstgroupmocos)
    secondgroupcrowdfunding = exp_model(x) * (secondgroupmocos)

    firstgrouptotalcrowdfunding += firstgroupcrowdfunding
    secondgrouptotalcrowdfunding += secondgroupcrowdfunding

    fisrtgroupdailycoinsneeded = daily_result_row.coins_needed * 0.5 * 0.75
    secondgroupdailycoinsneeded = daily_result_row.coins_needed * 0.5 * 0.5

    fisrtgroupdailycoinsallocated = daily_result_row.coins_allocated * 0.5 * 0.75
    secondgroupdailycoinsallocated = daily_result_row.coins_allocated * 0.5 * 0.5

    fisrtgroupactualdailycoinsallocated = fisrtgroupdailycoinsallocated - firstgroupcrowdfunding
    secondgroupactualdailycoinsallocated = secondgroupdailycoinsallocated - secondgroupcrowdfunding

    coinsallocated = fisrtgroupactualdailycoinsallocated
    mocperkgCO2 = first_group_actual_daily_result_row.mocperkgCO2
    initialMP = first_group_actual_daily_result_row.mocomp

     #for first group actual coinsallocated in coinsallocatedarr1
     for mocperkgCO2 in mocperkg
    
        local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
        # Optimizing the objective function
        # You can choose different methods available in Optim, here BFGS is used as an example
        result = optimize(local_objective, 0.0, 10.0, Brent();
                    iterations = 15,
                    store_trace = true,
                    show_trace = true
                    )
    
        # Extracting the optimized value of mocomp
        optimized_mocomp = Optim.minimizer(result)
        optimized_value = Optim.minimum(result)
        #iter = iterations(result)
        #traceres = trace(result)
        # Find the row where delta is minimized
        min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
        println("Optimization happened at row: ", min_delta_index) 
    
        # Save the row of minimum delta in a separate file
        first_group_actual_daily_result_row = ( 
            mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
            mocomp = MParr[min_delta_index], 
            coins_allocated = coinsallocatedarr[min_delta_index], 
            coins_needed = coinsneededarr[min_delta_index], 
            market_delta = deltaarr[min_delta_index], 
            cardem = cardemarr[min_delta_index], 
            busdem = busdemarr[min_delta_index], 
            bikedem = bikedemarr[min_delta_index], 
            flows_car = flowscararr[min_delta_index], 
            flows_bus = flowsbusarr[min_delta_index], 
            flows_bike = flowsbikearr[min_delta_index], 
            mocos_car = mocoscararr[min_delta_index], 
            mocos_bus = mocosbusarr[min_delta_index], 
            mocos_bike = mocosbikearr[min_delta_index], 
            emissions_car = emissionscararr[min_delta_index], 
            emissions_bus = emissionsbusarr[min_delta_index], 
            total_tt = totaltt[min_delta_index], 
            total_tt_c = totalttc[min_delta_index], 
            total_tt_b = totalttb[min_delta_index], 
            total_tt_r = totalttr[min_delta_index], 
            total_dist_car = totaldcarr[min_delta_index], 
            total_dist_bus = totaldbarr[min_delta_index], 
            total_dist_rad = totaldrarr[min_delta_index]
        )
        end
        coinsallocated = secondgroupactualdailycoinsallocated
        mocperkgCO2 = second_group_actual_daily_result_row.mocperkgCO2
        initialMP = second_group_actual_daily_result_row.mocomp
         #for second group actual coinsallocated in coinsallocatedarr1
         for mocperkgCO2 in mocperkg
    
            local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
            # Optimizing the objective function
            # You can choose different methods available in Optim, here BFGS is used as an example
            result = optimize(local_objective, 0.0, 10.0, Brent();
                        iterations = 15,
                        store_trace = true,
                        show_trace = true
                        )
        
            # Extracting the optimized value of mocomp
            optimized_mocomp = Optim.minimizer(result)
            optimized_value = Optim.minimum(result)
            #iter = iterations(result)
            #traceres = trace(result)
            # Find the row where delta is minimized
            min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
            println("Optimization happened at row: ", min_delta_index) 
        
            # Save the row of minimum delta in a separate file
            second_group_actual_daily_result_row  = ( 
                mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
                mocomp = MParr[min_delta_index], 
                coins_allocated = coinsallocatedarr[min_delta_index], 
                coins_needed = coinsneededarr[min_delta_index], 
                market_delta = deltaarr[min_delta_index], 
                cardem = cardemarr[min_delta_index], 
                busdem = busdemarr[min_delta_index], 
                bikedem = bikedemarr[min_delta_index], 
                flows_car = flowscararr[min_delta_index], 
                flows_bus = flowsbusarr[min_delta_index], 
                flows_bike = flowsbikearr[min_delta_index], 
                mocos_car = mocoscararr[min_delta_index], 
                mocos_bus = mocosbusarr[min_delta_index], 
                mocos_bike = mocosbikearr[min_delta_index], 
                emissions_car = emissionscararr[min_delta_index], 
                emissions_bus = emissionsbusarr[min_delta_index], 
                total_tt = totaltt[min_delta_index], 
                total_tt_c = totalttc[min_delta_index], 
                total_tt_b = totalttb[min_delta_index], 
                total_tt_r = totalttr[min_delta_index], 
                total_dist_car = totaldcarr[min_delta_index], 
                total_dist_bus = totaldbarr[min_delta_index], 
                total_dist_rad = totaldrarr[min_delta_index]
            )
            end



    firstgroupmocos -= ((daily_result_row.coins_needed * 0.5 * 0.75) + firstgroupcrowdfunding)
    secondgroupmocos -= ((daily_result_row.coins_needed * 0.5 * 0.5) + secondgroupcrowdfunding)

    firstnextcardem = daily_result_row.cardem * 0.5
    firstnextbusdem = daily_result_row.busdem * 0.5
    firstnextbikedem = daily_result_row.bikedem * 0.5
    firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem) * 0.5

    secondnextcardem = daily_result_row.cardem * 0.5
    secondnextbusdem = daily_result_row.busdem * 0.5
    secondnextbikedem = daily_result_row.bikedem * 0.5
    secondnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem) * 0.5

    firstlasttotalcardem -= firstnextcardem
    firstlasttotalbusdem -= firstnextbusdem
    firstlasttotalbikedem -= firstnextbikedem
    firstlasttotaldem -= firstnextdem

    secondlasttotalcardem -= secondnextcardem
    secondlasttotalbusdem -= secondnextbusdem
    secondlasttotalbikedem -= secondnextbikedem
    secondlasttotaldem -= secondnextdem

    firstmonthly_mocperkgCO2 += (daily_result_row.mocperkgCO2 * 0.5)
    firstlast_mocomp = daily_result_row.mocomp
    firstmonthly_coins_allocated = monthly_result_row.first_available_allocated_mocos * 0.75
    firstmonthly_coins_needed += (daily_result_row.coins_needed * 0.5)
    firstmonthly_lastday_market_delta = monthly_result_row.first_available_allocated_mocos * 0.25 - daily_result_row.coins_needed * 0.5
    firstlast_cardem = daily_result_row.cardem * 0.5
    firstlast_busdem = daily_result_row.busdem * 0.5
    firstlast_bikedem = daily_result_row.bikedem * 0.5
    firstlast_flows_car = daily_result_row.flows_car * 0.5
    firstlast_flows_bus = daily_result_row.flows_bus * 0.5
    firstlast_flows_bike = daily_result_row.flows_bike * 0.5
    firstmonthly_mocos_car += (daily_result_row.mocos_car * 0.5)
    firstmonthly_mocos_bus += (daily_result_row.mocos_bus * 0.5)
    firstmonthly_mocos_bike += (daily_result_row.mocos_bike * 0.5)
    firstmonthly_emissions_car += (daily_result_row.emissions_car * 0.5)
    firstmonthly_emissions_bus += (daily_result_row.emissions_bus * 0.5)
    firstmonthly_total_tt += (daily_result_row.total_tt * 0.5)
    firstmonthly_total_tt_c += (daily_result_row.total_tt_c * 0.5) 
    firstmonthly_total_tt_b += (daily_result_row.total_tt_b * 0.5)
    firstmonthly_total_tt_r += (daily_result_row.total_tt_r * 0.5)
    firstmonthly_total_dist_car += (daily_result_row.total_dist_car * 0.5)
    firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus * 0.5)
    firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad * 0.5)
    firstcoinsallocated = firstcoinsallocated
    firstgroupparticipants = firstgroupparticipants
    lastavailable_firstgroupmocos = firstgroupmocos
    lastfirstgroup_crowdfunding = firstgroupcrowdfunding
    monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
    firstgroup_totalcardem = monthly_result_row.totalcardem / 2
    firstgroup_totalbusdem = monthly_result_row.totalbusdem / 2
    firstgroup_totalbikedem = monthly_result_row.totalbikedem / 2
    firstgroup_totaldem = monthly_result_row.totaldem / 2

    first_group_of_multi_first_month_result_row = ( 
    firstmonthly_mocperkgCO2 = firstmonthly_mocperkgCO2, firstlast_mocomp, firstmonthly_coins_allocated, firstmonthly_coins_needed, firstmonthly_lastday_market_delta,
    firstlast_cardem, firstlast_busdem, firstlast_bikedem, 
    firstlast_flows_car, firstlast_flows_bus, firstlast_flows_bike, 
    firstmonthly_mocos_car, firstmonthly_mocos_bus, firstmonthly_mocos_bike, firstmonthly_emissions_car, firstmonthly_emissions_bus, 
    firstmonthly_total_tt, firstmonthly_total_tt_c, firstmonthly_total_tt_b, firstmonthly_total_tt_r, 
    firstmonthly_total_dist_car, firstmonthly_total_dist_bus, firstmonthly_total_dist_rad,
    firstlasttotalcardem, firstlasttotalbusdem, firstlasttotalbikedem, firstlasttotaldem,
    firstcoinsallocated, firstgroupparticipants, lastavailable_firstgroupmocos, lastfirstgroup_crowdfunding, monthlyfirstgroup_crowdfunding,
    firstgroup_totalcardem, firstgroup_totalbusdem, firstgroup_totalbikedem, firstgroup_totaldem
    )

    secondmonthly_mocperkgCO2 += (daily_result_row.mocperkgCO2 * 0.5)
    secondlast_mocomp = daily_result_row.mocomp
    secondmonthly_coins_allocated = monthly_result_row.first_available_allocated_mocos * 0.5
    secondmonthly_coins_needed += (daily_result_row.coins_needed * 0.5)
    secondmonthly_lastday_market_delta = daily_result_row.market_delta * 0.5
    secondlast_cardem = daily_result_row.cardem * 0.5
    secondlast_busdem = daily_result_row.busdem * 0.5
    secondlast_bikedem = daily_result_row.bikedem * 0.5
    secondlast_flows_car = daily_result_row.flows_car * 0.5
    secondlast_flows_bus = daily_result_row.flows_bus * 0.5
    secondlast_flows_bike = daily_result_row.flows_bike * 0.5
    secondmonthly_mocos_car += (daily_result_row.mocos_car * 0.5)
    secondmonthly_mocos_bus += (daily_result_row.mocos_bus * 0.5)
    secondmonthly_mocos_bike += (daily_result_row.mocos_bike * 0.5)
    secondmonthly_emissions_car += (daily_result_row.emissions_car * 0.5)
    secondmonthly_emissions_bus += (daily_result_row.emissions_bus * 0.5)
    secondmonthly_total_tt += (daily_result_row.total_tt * 0.5)
    secondmonthly_total_tt_c += (daily_result_row.total_tt_c * 0.5)
    secondmonthly_total_tt_b += (daily_result_row.total_tt_b * 0.5)
    secondmonthly_total_tt_r += (daily_result_row.total_tt_r * 0.5)
    secondmonthly_total_dist_car += (daily_result_row.total_dist_car * 0.5)
    secondmonthly_total_dist_bus += (daily_result_row.total_dist_bus * 0.5)
    secondmonthly_total_dist_rad += (daily_result_row.total_dist_rad * 0.5)
    secondcoinsallocated = secondcoinsallocated
    secondgroupparticipants = secondgroupparticipants
    lastavailable_secondgroupmocos = secondgroupmocos
    lastsecondgroup_crowdfunding = secondgroupcrowdfunding
    monthlysecondgroup_crowdfunding = secondgrouptotalcrowdfunding
    secondgroup_totalcardem = monthly_result_row.totalcardem / 2
    secondgroup_totalbusdem = monthly_result_row.totalbusdem / 2
    secondgroup_totalbikedem = monthly_result_row.totalbikedem / 2
    secondgroup_totaldem = monthly_result_row.totaldem / 2

    second_group_of_multi_first_month_result_row = ( 
    secondmonthly_mocperkgCO2 = secondmonthly_mocperkgCO2, secondlast_mocomp, secondmonthly_coins_allocated, secondmonthly_coins_needed, secondmonthly_lastday_market_delta,
    secondlast_cardem, secondlast_busdem, secondlast_bikedem, 
    secondlast_flows_car, secondlast_flows_bus, secondlast_flows_bike, 
    secondmonthly_mocos_car, secondmonthly_mocos_bus, secondmonthly_mocos_bike, secondmonthly_emissions_car, secondmonthly_emissions_bus, 
    secondmonthly_total_tt, secondmonthly_total_tt_c, secondmonthly_total_tt_b, secondmonthly_total_tt_r, 
    secondmonthly_total_dist_car, secondmonthly_total_dist_bus, secondmonthly_total_dist_rad,
    secondlasttotalcardem, secondlasttotalbusdem, secondlasttotalbikedem, secondlasttotaldem,
    secondcoinsallocated, secondgroupparticipants, lastavailable_secondgroupmocos, lastsecondgroup_crowdfunding, monthlysecondgroup_crowdfunding,
    secondgroup_totalcardem, secondgroup_totalbusdem, secondgroup_totalbikedem, secondgroup_totaldem
    )
    
    coinsallocated = daily_result_row.coins_allocated

    mocperkgCO2 = daily_result_row.mocperkgCO2
    initialMP = daily_result_row.mocomp

    # Write header and data to CSV
    writedlm("First_Groupe_of_Multi_Tradable_Scheme_First_Month_Result.csv", first_group_of_multi_first_month_result_row, ',')
    writedlm("Second_Groupe_of_Multi_Tradable_Scheme_First_Month_Result.csv", second_group_of_multi_first_month_result_row, ',')

end

println(first_group_of_multi_first_month_result_row)
println(second_group_of_multi_first_month_result_row)

In [ ]:
# 2 Groups of allocated mobility coins at the beginning of the month
# first group received mobility coins which is necessary for first half month travelling from second month to the last month which is half of the population
firstgroupmocosfirsthalf = monthly_result_row.first_available_allocated_mocos * 0.75 - firstmonthly_coins_needed
# first group received mobility coins which is necessary for second half month travelling from second month to the last month which is half of the population
firstgroupmocosfirsthalf = monthly_result_row.first_available_allocated_mocos * 0.5
firstgroupparticipants = 0.5 * monthly_result_row.totaldem
# second group received mobility coins which is necessary for one month travelling which is the other half of the population
secondgroupmocos = monthly_result_row.first_available_allocated_mocos * 0.5
secondgroupparticipants = 0.5 * monthly_result_row.totaldem 
x = 20 # number of working days in month
y = 10 # number of working days in half month

firstcoinsallocated = firstgroupmocos # 1.1369356986281308e6 # 1.164475393364052e6
secondcoinsallocated = secondgroupmocos

coinsallocated = 645860 # 1.1369356986281308e6 # 1.164475393364052e6
allocated_mocos = 0
consuming_mocos = 0

firstgroupcrowdfunding = 0
secondgroupcrowdfunding = 

firstgrouptotalcrowdfunding = 0
secondgrouptotalcrowdfunding = 0

mocperkgCO2 = 1 # 1.05
initialMP = 1

firsttotalcardem = monthly_result_row.totalcardem / 2
firsttotalbusdem = monthly_result_row.totalbusdem / 2
firsttotalbikedem = monthly_result_row.totalbikedem / 2
firsttotaldem = monthly_result_row.totaldem / 2

firstlasttotalcardem = monthly_result_row.totalcardem / 2
firstlasttotalbusdem = monthly_result_row.totalbusdem / 2
firstlasttotalbikedem = monthly_result_row.totalbikedem / 2
firstlasttotaldem = monthly_result_row.totaldem / 2

secondtotalcardem = monthly_result_row.totalcardem / 2
secondtotalbusdem = monthly_result_row.totalbusdem / 2
secondtotalbikedem = monthly_result_row.totalbikedem / 2
secondtotaldem = monthly_result_row.totaldem / 2

secondlasttotalcardem = monthly_result_row.totalcardem / 2
secondlasttotalbusdem = monthly_result_row.totalbusdem / 2
secondlasttotalbikedem = monthly_result_row.totalbikedem / 2
secondlasttotaldem = monthly_result_row.totaldem / 2

firstmonthly_mocperkgCO2 = 0
firstlast_mocomp = 0   
firstmonthly_mocperkgCO2 = 0 
firstlast_mocomp = 0 
firstmonthly_coins_allocated = 0 
firstmonthly_coins_needed = 0 
firstmonthly_market_delta = 0 
firstlast_cardem = 0 
firstlast_busdem = 0 
firstlast_bikedem = 0 
firstlast_flows_car = 0 
firstlast_flows_bus = 0 
firstlast_flows_bike = 0 
firstmonthly_mocos_car = 0 
firstmonthly_mocos_bus = 0 
firstmonthly_mocos_bike = 0 
firstmonthly_emissions_car = 0 
firstmonthly_emissions_bus = 0 
firstmonthly_total_tt = 0 
firstmonthly_total_tt_c = 0 
firstmonthly_total_tt_b = 0 
firstmonthly_total_tt_r = 0 
firstmonthly_total_dist_car = 0 
firstmonthly_total_dist_bus = 0 
firstmonthly_total_dist_rad = 0 

secondmonthly_mocperkgCO2 = 0 
secondlast_mocomp = 0 
secondmonthly_coins_allocated = 0 
secondmonthly_coins_needed = 0 
secondmonthly_market_delta = 0 
secondlast_cardem = 0 
secondlast_busdem = 0 
secondlast_bikedem = 0 
secondlast_flows_car = 0 
secondlast_flows_bus = 0 
secondlast_flows_bike = 0 
secondmonthly_mocos_car = 0 
secondmonthly_mocos_bus = 0 
secondmonthly_mocos_bike = 0 
secondmonthly_emissions_car = 0 
secondmonthly_emissions_bus = 0 
secondmonthly_total_tt = 0 
secondmonthly_total_tt_c = 0 
secondmonthly_total_tt_b = 0 
secondmonthly_total_tt_r = 0 
secondmonthly_total_dist_car = 0 
secondmonthly_total_dist_bus = 0 
secondmonthly_total_dist_rad = 0 

mocperkg = [1.0]
    # 2.5,
    # 5.0,
    # 7.5]
daily_result_row = ()
first_group_of_multi_first_month_result_row = ()
second_group_of_multi_first_month_result_row = ()

for x in 20:-1:1 # 20 working days in a month
    #for coinsallocated in coinsallocatedarr1
    for mocperkgCO2 in mocperkg
    
    local_objective(initialMP) = objective_function_1(initialMP, mocperkgCO2)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, 0.0, 10.0, Brent();
                iterations = 15,
                store_trace = true,
                show_trace = true
                )

    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)
    # Find the row where delta is minimized
    min_delta_index = findfirst(==(optimized_value), deltaarr)  # Find row index of min delta
    println("Optimization happened at row: ", min_delta_index) 

    # Save the row of minimum delta in a separate file
    daily_result_row = ( 
        mocperkgCO2 = mocperkgCO2arr[min_delta_index], 
        mocomp = MParr[min_delta_index], 
        coins_allocated = coinsallocatedarr[min_delta_index], 
        coins_needed = coinsneededarr[min_delta_index], 
        market_delta = deltaarr[min_delta_index], 
        cardem = cardemarr[min_delta_index], 
        busdem = busdemarr[min_delta_index], 
        bikedem = bikedemarr[min_delta_index], 
        flows_car = flowscararr[min_delta_index], 
        flows_bus = flowsbusarr[min_delta_index], 
        flows_bike = flowsbikearr[min_delta_index], 
        mocos_car = mocoscararr[min_delta_index], 
        mocos_bus = mocosbusarr[min_delta_index], 
        mocos_bike = mocosbikearr[min_delta_index], 
        emissions_car = emissionscararr[min_delta_index], 
        emissions_bus = emissionsbusarr[min_delta_index], 
        total_tt = totaltt[min_delta_index], 
        total_tt_c = totalttc[min_delta_index], 
        total_tt_b = totalttb[min_delta_index], 
        total_tt_r = totalttr[min_delta_index], 
        total_dist_car = totaldcarr[min_delta_index], 
        total_dist_bus = totaldbarr[min_delta_index], 
        total_dist_rad = totaldrarr[min_delta_index]
    )
    end

    exp_model(x) = 0.5775211856783368 * exp(-0.17571864924550018 * (x)) # 20 working days in a month
    firstgroupcrowdfunding = exp_model(x) * (firstgroupmocos)
    secondgroupcrowdfunding = exp_model(x) * (secondgroupmocos)

    firstgrouptotalcrowdfunding += firstgroupcrowdfunding
    secondgrouptotalcrowdfunding += secondgroupcrowdfunding

    firstgroupmocos -= ((daily_result_row.coins_needed * 0.5 * 0.75) + firstgroupcrowdfunding)
    secondgroupmocos -= ((daily_result_row.coins_needed * 0.5 * 0.5) + secondgroupcrowdfunding)

    firstnextcardem = daily_result_row.cardem * 0.5
    firstnextbusdem = daily_result_row.busdem * 0.5
    firstnextbikedem = daily_result_row.bikedem * 0.5
    firstnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem) * 0.5

    secondnextcardem = daily_result_row.cardem * 0.5
    secondnextbusdem = daily_result_row.busdem * 0.5
    secondnextbikedem = daily_result_row.bikedem * 0.5
    secondnextdem = (daily_result_row.cardem + daily_result_row.busdem + daily_result_row.bikedem) * 0.5

    firstlasttotalcardem -= firstnextcardem
    firstlasttotalbusdem -= firstnextbusdem
    firstlasttotalbikedem -= firstnextbikedem
    firstlasttotaldem -= firstnextdem

    secondlasttotalcardem -= secondnextcardem
    secondlasttotalbusdem -= secondnextbusdem
    secondlasttotalbikedem -= secondnextbikedem
    secondlasttotaldem -= secondnextdem

    firstmonthly_mocperkgCO2 += (daily_result_row.mocperkgCO2 * 0.5)
    firstlast_mocomp = daily_result_row.mocomp
    firstmonthly_coins_allocated = monthly_result_row.first_available_allocated_mocos * 0.75
    firstmonthly_coins_needed += (daily_result_row.coins_needed * 0.5)
    firstmonthly_lastday_market_delta = monthly_result_row.first_available_allocated_mocos * 0.25 - daily_result_row.coins_needed * 0.5
    firstlast_cardem = daily_result_row.cardem * 0.5
    firstlast_busdem = daily_result_row.busdem * 0.5
    firstlast_bikedem = daily_result_row.bikedem * 0.5
    firstlast_flows_car = daily_result_row.flows_car * 0.5
    firstlast_flows_bus = daily_result_row.flows_bus * 0.5
    firstlast_flows_bike = daily_result_row.flows_bike * 0.5
    firstmonthly_mocos_car += (daily_result_row.mocos_car * 0.5)
    firstmonthly_mocos_bus += (daily_result_row.mocos_bus * 0.5)
    firstmonthly_mocos_bike += (daily_result_row.mocos_bike * 0.5)
    firstmonthly_emissions_car += (daily_result_row.emissions_car * 0.5)
    firstmonthly_emissions_bus += (daily_result_row.emissions_bus * 0.5)
    firstmonthly_total_tt += (daily_result_row.total_tt * 0.5)
    firstmonthly_total_tt_c += (daily_result_row.total_tt_c * 0.5) 
    firstmonthly_total_tt_b += (daily_result_row.total_tt_b * 0.5)
    firstmonthly_total_tt_r += (daily_result_row.total_tt_r * 0.5)
    firstmonthly_total_dist_car += (daily_result_row.total_dist_car * 0.5)
    firstmonthly_total_dist_bus += (daily_result_row.total_dist_bus * 0.5)
    firstmonthly_total_dist_rad += (daily_result_row.total_dist_rad * 0.5)
    firstcoinsallocated = firstcoinsallocated
    firstgroupparticipants = firstgroupparticipants
    lastavailable_firstgroupmocos = firstgroupmocos
    lastfirstgroup_crowdfunding = firstgroupcrowdfunding
    monthlyfirstgroup_crowdfunding = firstgrouptotalcrowdfunding
    firstgroup_totalcardem = monthly_result_row.totalcardem / 2
    firstgroup_totalbusdem = monthly_result_row.totalbusdem / 2
    firstgroup_totalbikedem = monthly_result_row.totalbikedem / 2
    firstgroup_totaldem = monthly_result_row.totaldem / 2

    first_group_of_multi_first_month_result_row = ( 
    firstmonthly_mocperkgCO2 = firstmonthly_mocperkgCO2, firstlast_mocomp, firstmonthly_coins_allocated, firstmonthly_coins_needed, firstmonthly_lastday_market_delta,
    firstlast_cardem, firstlast_busdem, firstlast_bikedem, 
    firstlast_flows_car, firstlast_flows_bus, firstlast_flows_bike, 
    firstmonthly_mocos_car, firstmonthly_mocos_bus, firstmonthly_mocos_bike, firstmonthly_emissions_car, firstmonthly_emissions_bus, 
    firstmonthly_total_tt, firstmonthly_total_tt_c, firstmonthly_total_tt_b, firstmonthly_total_tt_r, 
    firstmonthly_total_dist_car, firstmonthly_total_dist_bus, firstmonthly_total_dist_rad,
    firstlasttotalcardem, firstlasttotalbusdem, firstlasttotalbikedem, firstlasttotaldem,
    firstcoinsallocated, firstgroupparticipants, lastavailable_firstgroupmocos, lastfirstgroup_crowdfunding, monthlyfirstgroup_crowdfunding,
    firstgroup_totalcardem, firstgroup_totalbusdem, firstgroup_totalbikedem, firstgroup_totaldem
    )

    secondmonthly_mocperkgCO2 += (daily_result_row.mocperkgCO2 * 0.5)
    secondlast_mocomp = daily_result_row.mocomp
    secondmonthly_coins_allocated = monthly_result_row.first_available_allocated_mocos * 0.5
    secondmonthly_coins_needed += (daily_result_row.coins_needed * 0.5)
    secondmonthly_lastday_market_delta = daily_result_row.market_delta * 0.5
    secondlast_cardem = daily_result_row.cardem * 0.5
    secondlast_busdem = daily_result_row.busdem * 0.5
    secondlast_bikedem = daily_result_row.bikedem * 0.5
    secondlast_flows_car = daily_result_row.flows_car * 0.5
    secondlast_flows_bus = daily_result_row.flows_bus * 0.5
    secondlast_flows_bike = daily_result_row.flows_bike * 0.5
    secondmonthly_mocos_car += (daily_result_row.mocos_car * 0.5)
    secondmonthly_mocos_bus += (daily_result_row.mocos_bus * 0.5)
    secondmonthly_mocos_bike += (daily_result_row.mocos_bike * 0.5)
    secondmonthly_emissions_car += (daily_result_row.emissions_car * 0.5)
    secondmonthly_emissions_bus += (daily_result_row.emissions_bus * 0.5)
    secondmonthly_total_tt += (daily_result_row.total_tt * 0.5)
    secondmonthly_total_tt_c += (daily_result_row.total_tt_c * 0.5)
    secondmonthly_total_tt_b += (daily_result_row.total_tt_b * 0.5)
    secondmonthly_total_tt_r += (daily_result_row.total_tt_r * 0.5)
    secondmonthly_total_dist_car += (daily_result_row.total_dist_car * 0.5)
    secondmonthly_total_dist_bus += (daily_result_row.total_dist_bus * 0.5)
    secondmonthly_total_dist_rad += (daily_result_row.total_dist_rad * 0.5)
    secondcoinsallocated = secondcoinsallocated
    secondgroupparticipants = secondgroupparticipants
    lastavailable_secondgroupmocos = secondgroupmocos
    lastsecondgroup_crowdfunding = secondgroupcrowdfunding
    monthlysecondgroup_crowdfunding = secondgrouptotalcrowdfunding
    secondgroup_totalcardem = monthly_result_row.totalcardem / 2
    secondgroup_totalbusdem = monthly_result_row.totalbusdem / 2
    secondgroup_totalbikedem = monthly_result_row.totalbikedem / 2
    secondgroup_totaldem = monthly_result_row.totaldem / 2

    second_group_of_multi_first_month_result_row = ( 
    secondmonthly_mocperkgCO2 = secondmonthly_mocperkgCO2, secondlast_mocomp, secondmonthly_coins_allocated, secondmonthly_coins_needed, secondmonthly_lastday_market_delta,
    secondlast_cardem, secondlast_busdem, secondlast_bikedem, 
    secondlast_flows_car, secondlast_flows_bus, secondlast_flows_bike, 
    secondmonthly_mocos_car, secondmonthly_mocos_bus, secondmonthly_mocos_bike, secondmonthly_emissions_car, secondmonthly_emissions_bus, 
    secondmonthly_total_tt, secondmonthly_total_tt_c, secondmonthly_total_tt_b, secondmonthly_total_tt_r, 
    secondmonthly_total_dist_car, secondmonthly_total_dist_bus, secondmonthly_total_dist_rad,
    secondlasttotalcardem, secondlasttotalbusdem, secondlasttotalbikedem, secondlasttotaldem,
    secondcoinsallocated, secondgroupparticipants, lastavailable_secondgroupmocos, lastsecondgroup_crowdfunding, monthlysecondgroup_crowdfunding,
    secondgroup_totalcardem, secondgroup_totalbusdem, secondgroup_totalbikedem, secondgroup_totaldem
    )
    
    coinsallocated = daily_result_row.coins_allocated

    mocperkgCO2 = daily_result_row.mocperkgCO2
    initialMP = daily_result_row.mocomp

    # Write header and data to CSV
    writedlm("First_Groupe_of_Multi_Tradable_Scheme_First_Month_Result.csv", first_group_of_multi_first_month_result_row, ',')
    writedlm("Second_Groupe_of_Multi_Tradable_Scheme_First_Month_Result.csv", second_group_of_multi_first_month_result_row, ',')

end

println(first_group_of_multi_first_month_result_row)
println(second_group_of_multi_first_month_result_row)

In [ ]:
# Given data points
x_data = [7, 5, 3]  # Remaining days
y_data = [0.1688, 0.1658, 0.3409]  # Crowdfunding amount

# Logistic growth model: y = L / (1 + exp(-k(x - x0)))
logistic_model(x, p) = p[1] / (1 + exp.(-p[2] * (x .- p[3])))

# Initial guess for parameters [L, k, x0]
p0 = [1.0, 0.2, 5.0]  # L = 1, k = 0.2, x0 = 5

# Fit the logistic model to the data
fit = curve_fit(logistic_model, x_data, y_data, p0)

# Extract the estimated parameters
L_fit, k_fit, x0_fit = fit.param

println("Logistic Model Parameters: L = ", L_fit, ", k = ", k_fit, ", x0 = ", x0_fit)
println("Best-fit equation: y = ", L_fit, " / (1 + exp(-", k_fit, " * (x - ", x0_fit, ")))")

# Compute model values for x = 1 to 20
x_values = 1:20
y_values = logistic_model.(x_values, [L_fit, k_fit, x0_fit])

# Print the results
println("\nModel Predictions:")
for (x, y) in zip(x_values, y_values)
    println("x = $x → y = $y")
end

BACKUP BELOW

In [7]:
#coinsallocated = 1000000 # 1.164475393364052e6

mocperkgCO2 = 1 # 1.05
initialMP = 1

# link_travel_timec = link_travel_timecsq
# link_travel_timeb = link_travel_timebsq
# link_travel_timer = link_travel_timersq

borders = [0.0, 
           0.2,
           0.4,
           0.6,
           0.8,
           1.0,
           1.2,
           1.4,
           1.6,
           1.8,
           2.0,
           2.2,
           2.4,
           2.6,
           2.8,
           3.0] #16

# coinsallocatedarr1 = [955775.58]#,
#                     # 946217.82,
#                     # 927293.46,
#                     # 899474.66,
#                     # 863495.67,
#                     # 820320.89]


#for coinsallocated in coinsallocatedarr1
for border in borders
        
    local_objective(initialMP) = objective_function_1(initialMP, coinsallocated)
    # Optimizing the objective function
    # You can choose different methods available in Optim, here BFGS is used as an example
    result = optimize(local_objective, border, border, Brent();
                                                    iterations = 1,
                                                    store_trace = true,
                                                    show_trace = true
                                                    )


    # Extracting the optimized value of mocomp
    optimized_mocomp = Optim.minimizer(result)
    optimized_value = Optim.minimum(result)
    #iter = iterations(result)
    #traceres = trace(result)

end

     0     4.111417e+03              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 0.0
 * minimizer: 0.0
     0     6.458600e+05              NaN
 * x_lower: 0.2
 * best bound: initial
 * x_upper: 0.2
 * minimizer: 0.2
     0     6.458600e+05              NaN
 * x_lower: 0.4
 * best bound: initial
 * x_upper: 0.4
 * minimizer: 0.4
     0     6.458600e+05              NaN
 * x_lower: 0.6
 * best bound: initial
 * x_upper: 0.6
 * minimizer: 0.6
     0     6.458600e+05              NaN
 * x_lower: 0.8
 * best bound: initial
 * x_upper: 0.8
 * minimizer: 0.8
     0     6.458600e+05              NaN
 * x_lower: 1.0
 * best bound: initial
 * x_upper: 1.0
 * minimizer: 1.0
     0     6.458600e+05              NaN
 * x_lower: 1.2
 * best bound: initial
 * x_upper: 1.2
 * minimizer: 1.2
     0     6.458600e+05              NaN
 * x_lower: 1.4
 * best bound: initial
 * x_upper: 1.4
 * minimizer: 1.4
     0     6.458600e+05              NaN
 * x_lower: 1.6
 * best bound: initial
 * x_upp

In [15]:
# calculate flow on ring links
counter = 0
ringflowb = 0
avgflowb = 0
# update free flow travel time of bus ring links
for i in 1:length(ta_datab.init_node)
    init_node = ta_datab.init_node[i]
    term_node = ta_datab.term_node[i]
    
    # Check if the combination exists in ringlinks
    match = filter(row -> row[:init_node] == init_node && row[:term_node] == term_node, ring_links)
    
    # If a match is found, multiply the capacity by 0.7
    if nrow(match) > 0
        counter += 1
        ringflowb += link_flowb[i]
    end
    avgflowb = ringflow / counter
end

println(ringflowsbus)

# calculate travel time on link
ringtimeb = 0
# update free flow travel time of bus ring links
for i in 1:length(ta_datab.init_node)
    init_node = ta_datab.init_node[i]
    term_node = ta_datab.term_node[i]
    
    # Check if the combination exists in ringlinks
    match = filter(row -> row[:init_node] == init_node && row[:term_node] == term_node, ring_links)
    
    # If a match is found, multiply the capacity by 0.7
    if nrow(match) > 0
        ringtimeb += link_travel_timebsq[i] * link_flowb[i]
    end
end

println(ringtimeb)

# calculate emissions on ring
ringemissionsb = 0
# update free flow travel time of bus ring links
for i in 1:length(ta_datab.init_node)
    init_node = ta_datab.init_node[i]
    term_node = ta_datab.term_node[i]
    
    # Check if the combination exists in ringlinks
    match = filter(row -> row[:init_node] == init_node && row[:term_node] == term_node, ring_links)
    
    # If a match is found, multiply the capacity by 0.7
    if nrow(match) > 0
        ringemissionsb += ta_datab.link_length[i] * busext * link_flowb[i]
    end
end

println(ringtimeb)

UndefVarError: UndefVarError: ring_links not defined

In [16]:
0     1.928753e+05              NaN
 * x_lower: 0.0
 * best bound: initial
 * x_upper: 0.0

ErrorException: syntax: extra token "192875.3" after end of expression

In [14]:
link_travel_timec = link_travel_timecsq
link_travel_timeb = link_travel_timebsq
link_travel_timer = link_travel_timersq
println("done")

done
